## **POKY ModelFree Analysis Notebook**

by **Woonghee Lee, Ph.D.** (woonghee.lee@ucdenver.edu; POKY Team, Department of Chemistry, University of Colorado Denver)

December 27, 2022


Reference: https://comdnmr.nysbc.org/comd-nmr-dissem/comd-nmr-software/software/modelfree

*This notebook requires POKY BUILD 08/25/2022l or newer for the convenient data import.*

POKY download: https://poky.clas.ucdenver.edu

*Please email Prof. Arther G. Palmer III (agp6@columbia.edu) and let him know you use ModelFree 4. By that way, you will be updated with bug reports and updates.*



*   ModelFree 4: A program for optimizing “Lipari-Szabo model free” parameters to heteronuclear relaxation data.

*A. M. Mandel, M. Akke and A. G. Palmer, Backbone dynamics of Escherichia coli ribonuclease HI: Correlations with structure and function in an active enzyme, J. Mol. Biol. 246, 144-163 (1995).*

*   FASTModelFree 1.3: A graphical front-end for running the ModelFree program. Originally written by Roger Cole and J. Patrick Loria (Yale University). The current version has been updated by A.G. Palmer and M. Rance. In this notebook, FASTModelFree 1.3wl that includes some corrections by Woonghee Lee is used.

*R. Cole and J.P. Loria, FAST-Modelfree: a program for rapid automated analysis of solution NMR spin-relaxation data, J. Biomol. NMR 26, 203-213 (2003).*

*   mfoutParser 1.4.2: A python library and notebook for reading and interpreting the mfout files produced by the ModelFree program. This package replaces the now unsupported STARBASE tools for reading mfout files. mfoutParser is written by Michelle Gill and Arthur Palmer (v1.4.1). In this notebook, we use 1.4.2 that includes some corrections made by Woonghee Lee.

# **Configure the form below and Run all from Runtime menu above.**

In [ ]:
#@title ### **Configure FASTModelFree**
FMFcfg='''
tensor $TENSOR
cutoff 0.95
Fcutoff 0.80
optimize Yes
maxloop 10
almost1 20
S2cutoff 0.7
seed 1985
numsim 300
jobname $JOBNAME
gamma -2.710
rNH 1.02
N15CSA -160
tm 6.463
tmMin 6
tmMax 7
tmGrid 5
tmConv 0.001
Dratio 0.844
DratioMin 0.8
DratioMax 0.9
DratioGrid 5
DratioConv 0.001
Theta 15.558
ThetaMin 0
ThetaMax 40
ThetaGrid 5
ThetaConv 0.001
Phi -213.771
PhiMin 0
PhiMax 360
PhiGrid 20
PhiConv 0.001
model1only No
mpdb mfpdb
file{0}{R1} fmfR1.txt
file{0}{R2} fmfR2.txt
file{0}{NOE} fmfNOE.txt
file{0}{field} $MAGNETICFIELD
'''

import os
import re
import time
import random
from google.colab import files

#@markdown Job name is optional.
job_name = '' #@param {type:"string"}
#@markdown Ctrl+c in the "rh" tool from POKY (Peak Height Analysis) and paste here.
T1_data = '' #@param {type:"string"}
#@markdown Ctrl+c in the "rh" tool from POKY (Peak Height Analysis) and paste here.
T2_data = '' #@param {type:"string"}
#@markdown Ctrl+c in the "np" tool from POKY (Perturbation Analysis; Data Height(ratio/error)) and paste here.
NOE_data = '' #@param {type:"string"}
#@markdown Magnetic Field Strength of the NMR Spectrometer.
Magnetic_Field = '600' #@param {type:"string"}
#@markdown Diffusion tensor. Axially Symmetric requires a PDB file.
diffusion_tensor = "Axially Symmetric" #@param ["Axially Symmetric", "isotropic"]
#@markdown PDB File (required for Axially Symmetric). Either upload a file or paste from clipboard.
upload_PDB_file = True #@param {type:"boolean"}
#@markdown or
PDB_data = "" #@param {type:"string"}
input_format = "double field"
#@markdown Check here if you do a just test run (not your data).
test = False #@param {type:"boolean"}

if upload_PDB_file and not test:
  uploaded = files.upload()
  pdb_file = uploaded.keys()[0]
  PDB_data = uploaded[pdb_file]

if '' in [T1_data, T2_data, NOE_data] and not test:
  print('Please set input data.')
  assert False

if PDB_data == '' and diffusion_tensor == 'Axially Symmetric' and not test:
  print('Please set a PDB data.')
  assert False


# **Run all from Runtime menu above.**

In [ ]:
#@title **Data Preprocessing**
# Test Data
if test: 
  PDB_data='''ATOM      1  N   LYS     1       4.373 -18.825   8.887  1.00 23.56           N  
ATOM      2  CA  LYS     1       4.991 -17.631   8.181  1.00 21.72           C  
ATOM      3  C   LYS     1       3.882 -16.568   8.135  1.00 17.32           C  
ATOM      4  O   LYS     1       2.729 -16.864   8.363  1.00 16.89           O  
ATOM      5  CB  LYS     1       5.527 -18.137   6.904  1.00 27.53           C  
ATOM      6  CG  LYS     1       5.923 -17.414   5.683  1.00 33.17           C  
ATOM      7  CD  LYS     1       6.836 -18.257   4.763  1.00 37.31           C  
ATOM      8  CE  LYS     1       7.506 -17.495   3.637  1.00 39.40           C  
ATOM      9  NZ  LYS     1       8.740 -16.752   3.976  1.00 40.47           N  
ATOM     10 1H   LYS     1       3.381 -18.907   8.619  1.00  0.00           H  
ATOM     11 2H   LYS     1       4.873 -19.683   8.614  1.00  0.00           H  
ATOM     12  HA  LYS     1       5.764 -17.262   8.871  1.00 22.07           H  
ATOM     13 1HB  LYS     1       6.452 -18.726   7.265  1.00 27.68           H  
ATOM     14 2HB  LYS     1       4.861 -18.992   6.555  1.00 27.60           H  
ATOM     15 1HG  LYS     1       5.038 -17.191   5.052  1.00 33.14           H  
ATOM     16 2HG  LYS     1       6.413 -16.446   5.854  1.00 33.35           H  
ATOM     17 1HD  LYS     1       7.618 -18.723   5.380  1.00 37.26           H  
ATOM     18 2HD  LYS     1       6.226 -19.052   4.335  1.00 37.19           H  
ATOM     19 1HE  LYS     1       7.810 -18.218   2.859  1.00 39.57           H  
ATOM     20 2HE  LYS     1       6.812 -16.787   3.184  1.00 39.56           H  
ATOM     21 1HZ  LYS     1       8.925 -16.835   4.986  1.00  0.00           H  
ATOM     22 2HZ  LYS     1       9.533 -17.146   3.450  1.00  0.00           H  
ATOM     23  N   GLU     2       4.332 -15.366   7.883  1.00 13.96           N  
ATOM     24  CA  GLU     2       3.354 -14.241   7.791  1.00 12.35           C  
ATOM     25  C   GLU     2       2.282 -14.509   6.738  1.00 11.37           C  
ATOM     26  O   GLU     2       2.596 -14.935   5.638  1.00 12.34           O  
ATOM     27  CB  GLU     2       4.144 -13.032   7.392  1.00 12.64           C  
ATOM     28  CG  GLU     2       3.366 -11.706   7.456  1.00 11.88           C  
ATOM     29  CD  GLU     2       4.231 -10.577   6.999  1.00 12.29           C  
ATOM     30  OE1 GLU     2       5.086 -10.685   6.141  1.00 13.27           O  
ATOM     31  OE2 GLU     2       4.031  -9.415   7.558  1.00 12.32           O  
ATOM     32  H   GLU     2       5.282 -15.142   7.698  1.00 15.24           H  
ATOM     33  HA  GLU     2       2.901 -14.099   8.741  1.00 12.57           H  
ATOM     34 1HB  GLU     2       5.032 -12.954   8.003  1.00 12.41           H  
ATOM     35 2HB  GLU     2       4.491 -13.152   6.349  1.00 12.27           H  
ATOM     36 1HG  GLU     2       2.497 -11.766   6.850  1.00 11.55           H  
ATOM     37 2HG  GLU     2       3.042 -11.488   8.462  1.00 11.82           H  
ATOM     38  N   THR     3       1.061 -14.232   7.139  1.00 11.69           N  
ATOM     39  CA  THR     3      -0.091 -14.422   6.212  1.00 12.47           C  
ATOM     40  C   THR     3      -0.103 -13.201   5.207  1.00 11.52           C  
ATOM     41  O   THR     3       0.461 -12.188   5.427  1.00 11.28           O  
ATOM     42  CB  THR     3      -1.413 -14.544   6.924  1.00 13.91           C  
ATOM     43  OG1 THR     3      -1.692 -13.247   7.483  1.00 14.71           O  
ATOM     44  CG2 THR     3      -1.486 -15.634   8.006  1.00 14.39           C  
ATOM     45  H   THR     3       0.834 -13.872   8.041  1.00 12.05           H  
ATOM     46  HA  THR     3       0.103 -15.288   5.634  1.00 12.74           H  
ATOM     47  HB  THR     3      -2.201 -14.753   6.162  1.00 13.83           H  
ATOM     48  HG1 THR     3      -1.279 -13.086   8.345  1.00 14.34           H  
ATOM     49 1HG2 THR     3      -0.528 -15.673   8.540  1.00 14.04           H  
ATOM     50 2HG2 THR     3      -2.293 -15.494   8.682  1.00 13.72           H  
ATOM     51 3HG2 THR     3      -1.574 -16.634   7.513  1.00 14.47           H  
ATOM     52  N   ALA     4      -0.789 -13.489   4.110  1.00 11.64           N  
ATOM     53  CA  ALA     4      -0.911 -12.464   3.056  1.00 11.31           C  
ATOM     54  C   ALA     4      -1.688 -11.266   3.640  1.00 10.85           C  
ATOM     55  O   ALA     4      -1.319 -10.133   3.322  1.00 10.82           O  
ATOM     56  CB  ALA     4      -1.525 -13.077   1.783  1.00 12.77           C  
ATOM     57  H   ALA     4      -1.271 -14.351   3.951  1.00 12.74           H  
ATOM     58  HA  ALA     4       0.070 -12.126   2.784  1.00 11.15           H  
ATOM     59 1HB  ALA     4      -0.909 -13.911   1.453  1.00 12.50           H  
ATOM     60 2HB  ALA     4      -2.510 -13.433   1.972  1.00 12.03           H  
ATOM     61 3HB  ALA     4      -1.507 -12.367   0.969  1.00 12.31           H  
ATOM     62  N   ALA     5      -2.660 -11.483   4.468  1.00 10.36           N  
ATOM     63  CA  ALA     5      -3.432 -10.417   5.070  1.00 10.22           C  
ATOM     64  C   ALA     5      -2.543  -9.539   5.943  1.00 10.43           C  
ATOM     65  O   ALA     5      -2.642  -8.300   5.932  1.00 10.21           O  
ATOM     66  CB  ALA     5      -4.639 -10.881   5.861  1.00 10.84           C  
ATOM     67  H   ALA     5      -2.925 -12.436   4.712  1.00 11.37           H  
ATOM     68  HA  ALA     5      -3.808  -9.802   4.256  1.00 10.34           H  
ATOM     69 1HB  ALA     5      -5.292 -11.494   5.260  1.00 10.89           H  
ATOM     70 2HB  ALA     5      -4.356 -11.414   6.753  1.00 10.74           H  
ATOM     71 3HB  ALA     5      -5.220 -10.024   6.214  1.00 11.08           H  
ATOM     72  N   ALA     6      -1.693 -10.213   6.736  1.00  9.74           N  
ATOM     73  CA  ALA     6      -0.774  -9.510   7.633  1.00  9.66           C  
ATOM     74  C   ALA     6       0.259  -8.742   6.842  1.00  8.34           C  
ATOM     75  O   ALA     6       0.642  -7.621   7.222  1.00  9.15           O  
ATOM     76  CB  ALA     6      -0.084 -10.529   8.556  1.00 10.47           C  
ATOM     77  H   ALA     6      -1.642 -11.203   6.719  1.00 10.48           H  
ATOM     78  HA  ALA     6      -1.320  -8.847   8.272  1.00  9.48           H  
ATOM     79 1HB  ALA     6       0.253 -11.369   7.960  1.00  9.89           H  
ATOM     80 2HB  ALA     6       0.760 -10.067   9.029  1.00 10.10           H  
ATOM     81 3HB  ALA     6      -0.793 -10.867   9.288  1.00 10.15           H  
ATOM     82  N   LYS     7       0.766  -9.310   5.775  1.00  8.11           N  
ATOM     83  CA  LYS     7       1.734  -8.682   4.907  1.00  8.55           C  
ATOM     84  C   LYS     7       1.103  -7.398   4.331  1.00  8.43           C  
ATOM     85  O   LYS     7       1.741  -6.353   4.234  1.00  8.42           O  
ATOM     86  CB  LYS     7       2.218  -9.619   3.841  1.00 11.63           C  
ATOM     87  CG  LYS     7       3.238  -8.984   2.872  1.00 15.78           C  
ATOM     88  CD  LYS     7       3.630 -10.007   1.842  1.00 20.82           C  
ATOM     89  CE  LYS     7       4.374  -9.376   0.665  0.75 24.29           C  
ATOM     90  NZ  LYS     7       4.668 -10.467  -0.306  0.67 26.23           N  
ATOM     91  H   LYS     7       0.432 -10.224   5.481  1.00  8.76           H  
ATOM     92  HA  LYS     7       2.596  -8.391   5.481  1.00  8.28           H  
ATOM     93 1HB  LYS     7       2.708 -10.486   4.302  1.00 11.46           H  
ATOM     94 2HB  LYS     7       1.363  -9.978   3.272  1.00 11.40           H  
ATOM     95 1HG  LYS     7       2.831  -8.092   2.382  1.00 15.74           H  
ATOM     96 2HG  LYS     7       4.124  -8.663   3.414  1.00 15.77           H  
ATOM     97 1HD  LYS     7       4.285 -10.754   2.297  1.00 20.42           H  
ATOM     98 2HD  LYS     7       2.752 -10.532   1.442  1.00 20.40           H  
ATOM     99 1HE  LYS     7       3.756  -8.642   0.165  0.75 24.07           H  
ATOM    100 2HE  LYS     7       5.319  -8.959   0.977  0.75 23.99           H  
ATOM    101 1HZ  LYS     7       4.129 -11.307  -0.051  1.00  0.00           H  
ATOM    102 2HZ  LYS     7       4.403 -10.163  -1.253  1.00  0.00           H  
ATOM    103  N   PHE     8      -0.152  -7.483   3.967  1.00  8.12           N  
ATOM    104  CA  PHE     8      -0.840  -6.259   3.413  1.00  7.81           C  
ATOM    105  C   PHE     8      -0.835  -5.149   4.425  1.00  7.47           C  
ATOM    106  O   PHE     8      -0.589  -3.958   4.130  1.00  8.29           O  
ATOM    107  CB  PHE     8      -2.237  -6.641   2.904  1.00  8.16           C  
ATOM    108  CG  PHE     8      -3.031  -5.449   2.419  1.00  8.09           C  
ATOM    109  CD1 PHE     8      -3.028  -5.106   1.069  1.00  8.66           C  
ATOM    110  CD2 PHE     8      -3.734  -4.636   3.305  1.00  8.80           C  
ATOM    111  CE1 PHE     8      -3.700  -3.987   0.592  1.00  8.94           C  
ATOM    112  CE2 PHE     8      -4.392  -3.496   2.875  1.00  9.34           C  
ATOM    113  CZ  PHE     8      -4.340  -3.182   1.515  1.00  9.32           C  
ATOM    114  H   PHE     8      -0.693  -8.309   4.044  1.00  8.50           H  
ATOM    115  HA  PHE     8      -0.334  -6.007   2.519  1.00  7.34           H  
ATOM    116 1HB  PHE     8      -2.076  -7.349   2.092  1.00  7.55           H  
ATOM    117 2HB  PHE     8      -2.742  -7.150   3.686  1.00  7.82           H  
ATOM    118  HD1 PHE     8      -2.486  -5.709   0.411  1.00  8.64           H  
ATOM    119  HD2 PHE     8      -3.707  -4.902   4.351  1.00  8.92           H  
ATOM    120  HE1 PHE     8      -3.665  -3.748  -0.425  1.00  8.69           H  
ATOM    121  HE2 PHE     8      -4.903  -2.892   3.517  1.00  8.95           H  
ATOM    122  HZ  PHE     8      -4.882  -2.339   1.156  1.00  9.31           H  
ATOM    123  N   GLU     9      -1.188  -5.445   5.675  1.00  7.10           N  
ATOM    124  CA  GLU     9      -1.203  -4.464   6.721  1.00  7.85           C  
ATOM    125  C   GLU     9       0.172  -3.824   6.901  1.00  8.11           C  
ATOM    126  O   GLU     9       0.326  -2.611   7.037  1.00  8.42           O  
ATOM    127  CB  GLU     9      -1.673  -4.921   8.068  1.00 10.56           C  
ATOM    128  CG  GLU     9      -3.068  -5.422   8.102  1.00 14.00           C  
ATOM    129  CD  GLU     9      -3.617  -5.879   9.447  1.00 17.53           C  
ATOM    130  OE1 GLU     9      -4.538  -6.738   9.375  1.00 19.71           O  
ATOM    131  OE2 GLU     9      -3.065  -5.227  10.446  1.00 18.71           O  
ATOM    132  H   GLU     9      -1.401  -6.405   5.914  1.00  7.90           H  
ATOM    133  HA  GLU     9      -1.882  -3.667   6.385  1.00  8.54           H  
ATOM    134 1HB  GLU     9      -0.982  -5.645   8.472  1.00 10.41           H  
ATOM    135 2HB  GLU     9      -1.608  -4.042   8.762  1.00 10.11           H  
ATOM    136 1HG  GLU     9      -3.735  -4.609   7.795  1.00 13.84           H  
ATOM    137 2HG  GLU     9      -3.286  -6.240   7.426  1.00 14.06           H  
ATOM    138  N   ARG    10       1.205  -4.697   6.960  1.00  7.12           N  
ATOM    139  CA  ARG    10       2.565  -4.222   7.132  1.00  7.81           C  
ATOM    140  C   ARG    10       3.064  -3.344   5.994  1.00  7.06           C  
ATOM    141  O   ARG    10       3.729  -2.343   6.204  1.00  7.60           O  
ATOM    142  CB  ARG    10       3.538  -5.432   7.349  1.00  7.68           C  
ATOM    143  CG  ARG    10       5.009  -5.044   7.548  1.00  8.35           C  
ATOM    144  CD  ARG    10       5.839  -6.241   7.948  1.00  9.90           C  
ATOM    145  NE  ARG    10       5.806  -7.314   6.969  1.00 11.02           N  
ATOM    146  CZ  ARG    10       6.558  -7.425   5.922  1.00 13.78           C  
ATOM    147  NH1 ARG    10       7.444  -6.467   5.555  1.00 15.63           N  
ATOM    148  NH2 ARG    10       6.452  -8.422   5.093  1.00 15.05           N  
ATOM    149  H   ARG    10       1.053  -5.675   6.858  1.00  7.67           H  
ATOM    150  HA  ARG    10       2.630  -3.647   8.041  1.00  7.59           H  
ATOM    151 1HB  ARG    10       3.142  -5.984   8.159  1.00  7.69           H  
ATOM    152 2HB  ARG    10       3.422  -6.059   6.468  1.00  6.90           H  
ATOM    153 1HG  ARG    10       5.409  -4.621   6.697  1.00  7.99           H  
ATOM    154 2HG  ARG    10       5.093  -4.322   8.322  1.00  7.95           H  
ATOM    155 1HD  ARG    10       6.810  -5.983   8.231  1.00  9.40           H  
ATOM    156 2HD  ARG    10       5.370  -6.628   8.837  1.00  9.49           H  
ATOM    157 1HH1 ARG    10       8.015  -6.577   4.722  1.00 15.96           H  
ATOM    158 2HH1 ARG    10       7.615  -5.642   6.128  1.00 14.60           H  
ATOM    159 1HH2 ARG    10       7.040  -8.488   4.252  1.00 15.94           H  
ATOM    160 2HH2 ARG    10       5.818  -9.190   5.218  1.00 14.77           H  
ATOM    161  N   GLN    11       2.790  -3.742   4.794  1.00  7.89           N  
ATOM    162  CA  GLN    11       3.283  -3.008   3.645  1.00  7.79           C  
ATOM    163  C   GLN    11       2.477  -1.748   3.325  1.00  8.26           C  
ATOM    164  O   GLN    11       3.057  -0.835   2.775  1.00  9.02           O  
ATOM    165  CB  GLN    11       3.295  -3.872   2.388  0.66  9.73           C  
ATOM    166  CG  GLN    11       4.097  -5.119   2.418  0.66 13.09           C  
ATOM    167  CD  GLN    11       4.253  -5.721   1.008  0.66 15.62           C  
ATOM    168  OE1 GLN    11       3.363  -5.685   0.218  0.66 17.24           O  
ATOM    169  NE2 GLN    11       5.452  -6.242   0.816  0.66 17.88           N  
ATOM    170  H   GLN    11       2.257  -4.589   4.605  1.00  7.87           H  
ATOM    171  HA  GLN    11       4.311  -2.735   3.795  1.00  8.51           H  
ATOM    172 1HB  GLN    11       2.240  -4.111   2.167  0.66  9.50           H  
ATOM    173 2HB  GLN    11       3.641  -3.226   1.553  0.66  9.47           H  
ATOM    174 1HG  GLN    11       5.059  -5.052   2.861  0.66 13.01           H  
ATOM    175 2HG  GLN    11       3.633  -5.953   2.925  0.66 12.99           H  
ATOM    176 1HE2 GLN    11       5.690  -6.630  -0.085  0.66 17.25           H  
ATOM    177 2HE2 GLN    11       6.188  -6.238   1.499  0.66 16.91           H  
ATOM    178  N   HIS    12       1.173  -1.736   3.628  1.00  7.46           N  
ATOM    179  CA  HIS    12       0.305  -0.690   3.111  1.00  8.30           C  
ATOM    180  C   HIS    12      -0.446   0.130   4.107  1.00  9.12           C  
ATOM    181  O   HIS    12      -0.997   1.165   3.660  1.00 11.15           O  
ATOM    182  CB  HIS    12      -0.648  -1.252   2.052  1.00  8.14           C  
ATOM    183  CG  HIS    12       0.008  -1.972   0.902  1.00  7.84           C  
ATOM    184  ND1 HIS    12       0.785  -1.255   0.004  1.00  8.55           N  
ATOM    185  CD2 HIS    12       0.036  -3.286   0.566  1.00  9.41           C  
ATOM    186  CE1 HIS    12       1.271  -2.171  -0.839  1.00  9.03           C  
ATOM    187  NE2 HIS    12       0.829  -3.387  -0.544  1.00  9.11           N  
ATOM    188  H   HIS    12       0.743  -2.565   3.996  1.00  8.05           H  
ATOM    189  HA  HIS    12       0.938  -0.003   2.521  1.00  8.28           H  
ATOM    190 1HB  HIS    12      -1.391  -1.897   2.531  1.00  8.22           H  
ATOM    191 2HB  HIS    12      -1.183  -0.397   1.620  1.00  8.28           H  
ATOM    192  HD1 HIS    12       0.993  -0.260  -0.034  1.00  8.89           H  
ATOM    193  HD2 HIS    12      -0.430  -4.122   1.026  1.00  8.97           H  
ATOM    194  HE1 HIS    12       1.888  -1.926  -1.675  1.00  8.87           H  
ATOM    195  N   MET    13      -0.616  -0.285   5.330  1.00  8.33           N  
ATOM    196  CA  MET    13      -1.468   0.498   6.224  1.00  8.15           C  
ATOM    197  C   MET    13      -0.718   1.448   7.121  1.00  8.39           C  
ATOM    198  O   MET    13       0.234   1.034   7.767  1.00  9.50           O  
ATOM    199  CB  MET    13      -2.332  -0.470   7.084  1.00  8.24           C  
ATOM    200  CG  MET    13      -3.394  -1.252   6.353  1.00 10.01           C  
ATOM    201  SD  MET    13      -4.613  -0.232   5.483  1.00 11.53           S  
ATOM    202  CE  MET    13      -5.194   0.808   6.863  1.00 11.55           C  
ATOM    203  H   MET    13      -0.216  -1.151   5.635  1.00  8.57           H  
ATOM    204  HA  MET    13      -2.198   1.032   5.649  1.00  7.91           H  
ATOM    205 1HB  MET    13      -1.645  -1.210   7.525  1.00  8.05           H  
ATOM    206 2HB  MET    13      -2.719   0.049   7.941  1.00  8.26           H  
ATOM    207 1HG  MET    13      -3.047  -1.858   5.503  1.00  9.93           H  
ATOM    208 2HG  MET    13      -3.971  -1.868   7.033  1.00  9.71           H  
ATOM    209 1HE  MET    13      -5.023   0.285   7.781  1.00 10.78           H  
ATOM    210 2HE  MET    13      -4.625   1.707   6.802  1.00 11.31           H  
ATOM    211 3HE  MET    13      -6.249   0.926   6.668  1.00 11.43           H  
ATOM    212  N   ASP    14      -1.181   2.692   7.226  1.00  8.07           N  
ATOM    213  CA  ASP    14      -0.645   3.600   8.198  1.00  9.03           C  
ATOM    214  C   ASP    14      -1.833   4.445   8.752  1.00  9.01           C  
ATOM    215  O   ASP    14      -1.998   5.587   8.429  1.00 10.31           O  
ATOM    216  CB  ASP    14       0.506   4.465   7.737  1.00  9.96           C  
ATOM    217  CG  ASP    14       1.010   5.285   8.944  1.00 11.19           C  
ATOM    218  OD1 ASP    14       0.694   5.013  10.081  1.00 11.23           O  
ATOM    219  OD2 ASP    14       1.777   6.256   8.573  1.00 12.91           O  
ATOM    220  H   ASP    14      -2.016   2.960   6.729  1.00  8.28           H  
ATOM    221  HA  ASP    14      -0.269   3.062   9.048  1.00  8.70           H  
ATOM    222 1HB  ASP    14       1.319   3.884   7.370  1.00  9.97           H  
ATOM    223 2HB  ASP    14       0.220   5.137   6.949  1.00  9.82           H  
ATOM    224  N   SER    15      -2.544   3.805   9.685  1.00  9.49           N  
ATOM    225  CA  SER    15      -3.691   4.454  10.354  1.00 10.43           C  
ATOM    226  C   SER    15      -3.258   5.464  11.393  1.00 11.26           C  
ATOM    227  O   SER    15      -4.156   6.123  11.947  1.00 13.77           O  
ATOM    228  CB  SER    15      -4.544   3.359  11.039  1.00 12.08           C  
ATOM    229  OG  SER    15      -4.905   2.370  10.093  1.00 14.80           O  
ATOM    230  H   SER    15      -2.355   2.861   9.953  1.00 10.67           H  
ATOM    231  HA  SER    15      -4.338   4.895   9.644  1.00 10.36           H  
ATOM    232 1HB  SER    15      -4.029   2.933  11.889  1.00 10.99           H  
ATOM    233 2HB  SER    15      -5.452   3.853  11.375  1.00 12.18           H  
ATOM    234  HG  SER    15      -4.709   1.505  10.435  1.00 15.24           H  
ATOM    235  N   SER    16      -2.027   5.560  11.717  1.00 11.47           N  
ATOM    236  CA  SER    16      -1.461   6.432  12.762  1.00 12.94           C  
ATOM    237  C   SER    16      -1.255   7.836  12.302  1.00 12.71           C  
ATOM    238  O   SER    16      -0.983   8.736  13.201  1.00 13.54           O  
ATOM    239  CB  SER    16      -0.129   5.896  13.351  1.00 15.90           C  
ATOM    240  OG  SER    16       1.012   6.275  12.483  1.00 19.61           O  
ATOM    241  H   SER    16      -1.316   4.980  11.283  1.00 12.00           H  
ATOM    242  HA  SER    16      -2.131   6.394  13.603  1.00 13.23           H  
ATOM    243 1HB  SER    16       0.043   6.393  14.311  1.00 16.27           H  
ATOM    244 2HB  SER    16      -0.163   4.846  13.478  1.00 15.70           H  
ATOM    245  HG  SER    16       1.634   5.545  12.569  1.00 19.07           H  
ATOM    246  N   THR    17      -1.320   8.114  11.034  1.00 12.34           N  
ATOM    247  CA  THR    17      -1.139   9.460  10.521  1.00 12.98           C  
ATOM    248  C   THR    17      -2.227   9.692   9.474  1.00 12.89           C  
ATOM    249  O   THR    17      -2.671   8.758   8.851  1.00 13.19           O  
ATOM    250  CB  THR    17       0.268   9.764   9.947  1.00 15.75           C  
ATOM    251  OG1 THR    17       0.425   8.960   8.763  1.00 17.53           O  
ATOM    252  CG2 THR    17       1.357   9.362  10.890  1.00 18.04           C  
ATOM    253  H   THR    17      -1.520   7.369  10.388  1.00 12.67           H  
ATOM    254  HA  THR    17      -1.256  10.167  11.324  1.00 13.22           H  
ATOM    255  HB  THR    17       0.308  10.834   9.683  1.00 15.24           H  
ATOM    256  HG1 THR    17       0.284   8.011   8.940  1.00 16.75           H  
ATOM    257 1HG2 THR    17       1.240   9.876  11.845  1.00 17.65           H  
ATOM    258 2HG2 THR    17       1.378   8.293  11.045  1.00 17.49           H  
ATOM    259 3HG2 THR    17       2.368   9.642  10.477  1.00 17.81           H  
ATOM    260  N   SER    18      -2.646  10.943   9.316  1.00 12.52           N  
ATOM    261  CA  SER    18      -3.646  11.333   8.359  1.00 13.54           C  
ATOM    262  C   SER    18      -3.116  11.546   6.940  1.00 12.12           C  
ATOM    263  O   SER    18      -3.852  11.497   5.972  1.00 12.10           O  
ATOM    264  CB  SER    18      -4.424  12.621   8.767  1.00 17.36           C  
ATOM    265  OG  SER    18      -5.210  12.150   9.876  1.00 23.94           O  
ATOM    266  H   SER    18      -2.227  11.685   9.876  1.00 14.06           H  
ATOM    267  HA  SER    18      -4.440  10.596   8.326  1.00 13.86           H  
ATOM    268 1HB  SER    18      -3.743  13.395   9.058  1.00 17.09           H  
ATOM    269 2HB  SER    18      -5.091  12.892   7.978  1.00 17.08           H  
ATOM    270  HG  SER    18      -4.700  12.385  10.673  1.00 22.64           H  
ATOM    271  N   ALA    19      -1.790  11.782   6.845  1.00 11.95           N  
ATOM    272  CA  ALA    19      -1.057  12.065   5.651  1.00 12.67           C  
ATOM    273  C   ALA    19       0.463  12.129   6.042  1.00 11.83           C  
ATOM    274  O   ALA    19       0.826  12.261   7.166  1.00 12.93           O  
ATOM    275  CB  ALA    19      -1.476  13.385   5.001  1.00 13.21           C  
ATOM    276  H   ALA    19      -1.304  11.808   7.741  1.00 12.07           H  
ATOM    277  HA  ALA    19      -1.158  11.307   4.908  1.00 12.99           H  
ATOM    278 1HB  ALA    19      -2.061  13.945   5.721  1.00 13.12           H  
ATOM    279 2HB  ALA    19      -0.617  13.965   4.703  1.00 13.38           H  
ATOM    280 3HB  ALA    19      -2.054  13.216   4.107  1.00 12.99           H  
ATOM    281  N   ALA    20       1.244  12.051   4.998  1.00 11.71           N  
ATOM    282  CA  ALA    20       2.729  12.141   5.174  1.00 12.75           C  
ATOM    283  C   ALA    20       3.038  13.617   5.645  1.00 14.23           C  
ATOM    284  O   ALA    20       2.510  14.498   4.975  1.00 16.18           O  
ATOM    285  CB  ALA    20       3.319  11.891   3.808  1.00 12.39           C  
ATOM    286  H   ALA    20       0.939  11.968   4.051  1.00 12.29           H  
ATOM    287  HA  ALA    20       3.044  11.455   5.885  1.00 12.43           H  
ATOM    288 1HB  ALA    20       2.976  12.586   3.064  1.00 11.90           H  
ATOM    289 2HB  ALA    20       4.407  11.958   3.857  1.00 12.43           H  
ATOM    290 3HB  ALA    20       3.068  10.881   3.486  1.00 12.18           H  
ATOM    291  N   SER    21       3.847  13.744   6.623  1.00 15.51           N  
ATOM    292  CA  SER    21       4.115  15.139   7.156  1.00 17.49           C  
ATOM    293  C   SER    21       5.398  15.809   6.700  1.00 16.75           C  
ATOM    294  O   SER    21       5.657  17.008   6.961  1.00 17.43           O  
ATOM    295  CB  SER    21       4.145  14.986   8.692  1.00 20.70           C  
ATOM    296  OG  SER    21       5.022  13.923   9.098  1.00 24.68           O  
ATOM    297  H   SER    21       4.228  12.982   7.138  1.00 15.44           H  
ATOM    298  HA  SER    21       3.277  15.749   6.953  1.00 17.31           H  
ATOM    299 1HB  SER    21       4.477  15.927   9.119  1.00 21.24           H  
ATOM    300 2HB  SER    21       3.149  14.735   9.031  1.00 20.94           H  
ATOM    301  HG  SER    21       5.906  14.265   9.209  1.00 24.05           H  
ATOM    302  N   SER    22       6.255  15.029   6.010  1.00 14.63           N  
ATOM    303  CA  SER    22       7.530  15.571   5.531  1.00 13.35           C  
ATOM    304  C   SER    22       8.054  14.722   4.379  1.00 11.91           C  
ATOM    305  O   SER    22       7.592  13.597   4.105  1.00 11.23           O  
ATOM    306  CB  SER    22       8.588  15.637   6.619  1.00 14.89           C  
ATOM    307  OG  SER    22       9.205  14.382   6.810  1.00 15.58           O  
ATOM    308  H   SER    22       6.019  14.083   5.843  1.00 15.70           H  
ATOM    309  HA  SER    22       7.387  16.581   5.213  1.00 13.72           H  
ATOM    310 1HB  SER    22       9.349  16.391   6.327  1.00 14.97           H  
ATOM    311 2HB  SER    22       8.157  15.959   7.552  1.00 14.87           H  
ATOM    312  HG  SER    22      10.121  14.547   7.062  1.00 16.65           H  
ATOM    313  N   SER    23       9.076  15.300   3.723  1.00 11.92           N  
ATOM    314  CA  SER    23       9.698  14.565   2.601  1.00 11.52           C  
ATOM    315  C   SER    23      10.362  13.281   3.060  1.00 11.31           C  
ATOM    316  O   SER    23      10.598  12.467   2.126  1.00 12.66           O  
ATOM    317  CB  SER    23      10.701  15.467   1.863  1.00 13.94           C  
ATOM    318  OG  SER    23      11.666  15.805   2.730  1.00 17.79           O  
ATOM    319  H   SER    23       9.441  16.174   3.972  1.00 12.02           H  
ATOM    320  HA  SER    23       8.959  14.343   1.868  1.00 11.51           H  
ATOM    321 1HB  SER    23      11.088  14.904   0.995  1.00 13.99           H  
ATOM    322 2HB  SER    23      10.146  16.325   1.493  1.00 14.04           H  
ATOM    323  HG  SER    23      11.504  16.489   3.369  1.00 18.09           H  
ATOM    324  N   ASN    24      10.633  13.043   4.297  1.00 10.87           N  
ATOM    325  CA  ASN    24      11.282  11.828   4.770  1.00 10.98           C  
ATOM    326  C   ASN    24      10.325  10.661   5.092  1.00  9.77           C  
ATOM    327  O   ASN    24      10.817   9.622   5.543  1.00 10.11           O  
ATOM    328  CB  ASN    24      12.265  12.054   5.914  1.00 15.71           C  
ATOM    329  CG  ASN    24      13.615  12.522   5.311  1.00 21.17           C  
ATOM    330  OD1 ASN    24      14.254  11.743   4.564  1.00 24.28           O  
ATOM    331  ND2 ASN    24      13.996  13.714   5.630  1.00 23.72           N  
ATOM    332  H   ASN    24      10.430  13.743   4.993  1.00 11.90           H  
ATOM    333  HA  ASN    24      11.949  11.451   4.003  1.00 11.30           H  
ATOM    334 1HB  ASN    24      11.900  12.766   6.621  1.00 15.49           H  
ATOM    335 2HB  ASN    24      12.520  11.148   6.442  1.00 15.73           H  
ATOM    336 1HD2 ASN    24      13.449  14.315   6.241  1.00 22.41           H  
ATOM    337 2HD2 ASN    24      14.890  14.093   5.265  1.00 22.49           H  
ATOM    338  N   TYR    25       9.049  10.916   4.969  1.00  8.79           N  
ATOM    339  CA  TYR    25       8.074   9.866   5.338  1.00  9.10           C  
ATOM    340  C   TYR    25       8.390   8.530   4.688  1.00  8.84           C  
ATOM    341  O   TYR    25       8.464   7.487   5.373  1.00  9.02           O  
ATOM    342  CB  TYR    25       6.642  10.371   5.054  1.00  8.97           C  
ATOM    343  CG  TYR    25       5.585   9.343   5.352  1.00  9.11           C  
ATOM    344  CD1 TYR    25       5.017   9.233   6.635  1.00  9.78           C  
ATOM    345  CD2 TYR    25       5.210   8.400   4.383  1.00  9.19           C  
ATOM    346  CE1 TYR    25       4.034   8.247   6.895  1.00  9.73           C  
ATOM    347  CE2 TYR    25       4.252   7.419   4.651  1.00  9.66           C  
ATOM    348  CZ  TYR    25       3.709   7.354   5.896  1.00  8.73           C  
ATOM    349  OH  TYR    25       2.775   6.342   6.125  1.00  9.89           O  
ATOM    350  H   TYR    25       8.697  11.781   4.643  1.00  9.68           H  
ATOM    351  HA  TYR    25       8.116   9.739   6.410  1.00  8.21           H  
ATOM    352 1HB  TYR    25       6.506  11.265   5.681  1.00  9.44           H  
ATOM    353 2HB  TYR    25       6.561  10.687   4.032  1.00  8.75           H  
ATOM    354  HD1 TYR    25       5.284   9.926   7.413  1.00  9.24           H  
ATOM    355  HD2 TYR    25       5.646   8.460   3.391  1.00  9.35           H  
ATOM    356  HE1 TYR    25       3.604   8.183   7.884  1.00  9.33           H  
ATOM    357  HE2 TYR    25       3.965   6.711   3.873  1.00  9.78           H  
ATOM    358  HH  TYR    25       2.921   5.961   6.991  1.00  9.28           H  
ATOM    359  N   CYS    26       8.510   8.541   3.387  1.00  8.12           N  
ATOM    360  CA  CYS    26       8.762   7.286   2.652  1.00  7.94           C  
ATOM    361  C   CYS    26      10.042   6.611   3.101  1.00  8.29           C  
ATOM    362  O   CYS    26      10.126   5.394   3.365  1.00  9.10           O  
ATOM    363  CB  CYS    26       8.708   7.425   1.162  1.00  8.67           C  
ATOM    364  SG  CYS    26       7.020   7.687   0.568  1.00  8.73           S  
ATOM    365  H   CYS    26       8.417   9.367   2.832  1.00  8.64           H  
ATOM    366  HA  CYS    26       7.927   6.632   2.896  1.00  7.15           H  
ATOM    367 1HB  CYS    26       9.294   8.246   0.808  1.00  8.74           H  
ATOM    368 2HB  CYS    26       9.049   6.521   0.699  1.00  8.91           H  
ATOM    369  N   ASN    27      11.109   7.410   3.210  1.00  8.67           N  
ATOM    370  CA  ASN    27      12.406   6.809   3.660  1.00  9.59           C  
ATOM    371  C   ASN    27      12.279   6.070   4.977  1.00 10.23           C  
ATOM    372  O   ASN    27      12.804   4.972   5.154  1.00 11.44           O  
ATOM    373  CB  ASN    27      13.472   7.894   3.743  1.00 11.14           C  
ATOM    374  CG  ASN    27      13.913   8.328   2.338  1.00 12.70           C  
ATOM    375  OD1 ASN    27      13.723   7.687   1.352  1.00 12.16           O  
ATOM    376  ND2 ASN    27      14.460   9.546   2.333  1.00 15.77           N  
ATOM    377  H   ASN    27      11.075   8.374   3.010  1.00  8.97           H  
ATOM    378  HA  ASN    27      12.768   6.185   2.888  1.00  9.70           H  
ATOM    379 1HB  ASN    27      13.088   8.787   4.250  1.00 11.15           H  
ATOM    380 2HB  ASN    27      14.313   7.579   4.318  1.00 11.41           H  
ATOM    381 1HD2 ASN    27      14.592  10.102   3.165  1.00 15.74           H  
ATOM    382 2HD2 ASN    27      14.785   9.951   1.431  1.00 15.34           H  
ATOM    383  N   GLN    28      11.618   6.740   5.933  1.00  9.49           N  
ATOM    384  CA  GLN    28      11.420   6.228   7.280  1.00 11.26           C  
ATOM    385  C   GLN    28      10.530   4.981   7.324  1.00  9.94           C  
ATOM    386  O   GLN    28      10.909   3.949   7.919  1.00 10.24           O  
ATOM    387  CB  GLN    28      10.829   7.321   8.191  1.00 15.90           C  
ATOM    388  CG  GLN    28      11.719   8.526   8.340  1.00 23.97           C  
ATOM    389  CD  GLN    28      11.017   9.771   8.821  1.00 29.69           C  
ATOM    390  OE1 GLN    28       9.772   9.967   8.801  1.00 33.33           O  
ATOM    391  NE2 GLN    28      11.837  10.764   9.296  1.00 32.77           N  
ATOM    392  H   GLN    28      11.228   7.646   5.752  1.00  9.96           H  
ATOM    393  HA  GLN    28      12.360   5.982   7.730  1.00 11.41           H  
ATOM    394 1HB  GLN    28       9.853   7.593   7.831  1.00 15.95           H  
ATOM    395 2HB  GLN    28      10.710   6.844   9.193  1.00 15.93           H  
ATOM    396 1HG  GLN    28      12.547   8.312   9.053  1.00 23.75           H  
ATOM    397 2HG  GLN    28      12.238   8.786   7.427  1.00 23.68           H  
ATOM    398 1HE2 GLN    28      11.447  11.613   9.634  1.00 31.33           H  
ATOM    399 2HE2 GLN    28      12.836  10.585   9.303  1.00 31.47           H  
ATOM    400  N   MET    29       9.414   5.084   6.634  1.00  8.85           N  
ATOM    401  CA  MET    29       8.448   3.983   6.625  1.00  8.74           C  
ATOM    402  C   MET    29       8.840   2.765   5.845  1.00  8.25           C  
ATOM    403  O   MET    29       8.576   1.604   6.243  1.00  9.22           O  
ATOM    404  CB  MET    29       7.096   4.482   6.132  1.00 10.90           C  
ATOM    405  CG  MET    29       6.459   5.413   7.071  1.00 15.07           C  
ATOM    406  SD  MET    29       5.942   4.713   8.632  1.00 18.46           S  
ATOM    407  CE  MET    29       4.644   3.652   8.150  1.00 20.03           C  
ATOM    408  H   MET    29       9.137   5.919   6.172  1.00  9.46           H  
ATOM    409  HA  MET    29       8.276   3.695   7.649  1.00  8.94           H  
ATOM    410 1HB  MET    29       7.226   4.977   5.152  1.00 11.11           H  
ATOM    411 2HB  MET    29       6.445   3.632   5.954  1.00 11.26           H  
ATOM    412 1HG  MET    29       6.937   6.331   7.385  1.00 14.81           H  
ATOM    413 2HG  MET    29       5.496   5.600   6.605  1.00 15.16           H  
ATOM    414 1HE  MET    29       4.020   4.123   7.411  1.00 19.51           H  
ATOM    415 2HE  MET    29       5.138   2.783   7.682  1.00 19.82           H  
ATOM    416 3HE  MET    29       4.124   3.344   9.046  1.00 19.64           H  
ATOM    417  N   MET    30       9.482   2.936   4.724  1.00  8.42           N  
ATOM    418  CA  MET    30       9.921   1.815   3.919  1.00  9.02           C  
ATOM    419  C   MET    30      10.909   0.951   4.728  1.00  9.78           C  
ATOM    420  O   MET    30      10.963  -0.284   4.686  1.00 10.89           O  
ATOM    421  CB  MET    30      10.519   2.317   2.624  1.00  8.96           C  
ATOM    422  CG  MET    30       9.505   2.902   1.654  1.00 10.17           C  
ATOM    423  SD  MET    30       8.277   1.761   1.015  1.00 10.13           S  
ATOM    424  CE  MET    30       9.159   0.751  -0.158  1.00  9.58           C  
ATOM    425  H   MET    30       9.722   3.857   4.382  1.00  8.54           H  
ATOM    426  HA  MET    30       9.073   1.209   3.671  1.00  9.05           H  
ATOM    427 1HB  MET    30      11.248   3.107   2.834  1.00  9.25           H  
ATOM    428 2HB  MET    30      11.028   1.491   2.149  1.00  9.38           H  
ATOM    429 1HG  MET    30       8.814   3.635   2.118  1.00 10.19           H  
ATOM    430 2HG  MET    30       9.976   3.352   0.795  1.00 10.17           H  
ATOM    431 1HE  MET    30      10.039   0.380   0.341  1.00  9.36           H  
ATOM    432 2HE  MET    30       8.501  -0.067  -0.406  1.00  9.58           H  
ATOM    433 3HE  MET    30       9.406   1.381  -0.993  1.00  9.01           H  
ATOM    434  N   LYS    31      11.747   1.682   5.498  1.00  9.63           N  
ATOM    435  CA  LYS    31      12.742   0.989   6.333  1.00 10.35           C  
ATOM    436  C   LYS    31      12.101   0.323   7.513  1.00 10.06           C  
ATOM    437  O   LYS    31      12.365  -0.868   7.806  1.00 10.83           O  
ATOM    438  CB  LYS    31      13.826   1.963   6.802  1.00 14.48           C  
ATOM    439  CG  LYS    31      14.989   1.079   7.311  1.00 20.51           C  
ATOM    440  CD  LYS    31      16.161   1.854   7.809  1.00 25.84           C  
ATOM    441  CE  LYS    31      17.298   0.841   8.162  0.50 27.59           C  
ATOM    442  NZ  LYS    31      17.654   0.994   9.598  0.50 29.31           N  
ATOM    443  H   LYS    31      11.712   2.672   5.535  1.00  9.76           H  
ATOM    444  HA  LYS    31      13.221   0.247   5.726  1.00 11.00           H  
ATOM    445 1HB  LYS    31      14.181   2.571   5.993  1.00 14.69           H  
ATOM    446 2HB  LYS    31      13.421   2.541   7.617  1.00 14.66           H  
ATOM    447 1HG  LYS    31      14.654   0.401   8.116  1.00 20.17           H  
ATOM    448 2HG  LYS    31      15.327   0.417   6.498  1.00 20.21           H  
ATOM    449 1HD  LYS    31      16.559   2.542   7.071  1.00 25.49           H  
ATOM    450 2HD  LYS    31      15.923   2.411   8.716  1.00 25.62           H  
ATOM    451 1HE  LYS    31      17.053  -0.183   7.989  0.50 27.67           H  
ATOM    452 2HE  LYS    31      18.174   1.130   7.580  0.50 27.66           H  
ATOM    453 1HZ  LYS    31      16.903   0.601  10.182  1.00  0.00           H  
ATOM    454 2HZ  LYS    31      18.533   0.493   9.789  1.00  0.00           H  
ATOM    455  N   SER    32      11.254   1.078   8.274  1.00 10.17           N  
ATOM    456  CA  SER    32      10.610   0.563   9.486  1.00 10.78           C  
ATOM    457  C   SER    32       9.707  -0.620   9.253  1.00 10.56           C  
ATOM    458  O   SER    32       9.609  -1.516  10.177  1.00 11.27           O  
ATOM    459  CB  SER    32       9.936   1.642  10.350  0.67 13.41           C  
ATOM    460  OG  SER    32       8.798   2.035   9.680  0.67 15.41           O  
ATOM    461  H   SER    32      11.093   2.025   8.016  1.00 10.30           H  
ATOM    462  HA  SER    32      11.411   0.253  10.154  1.00 11.48           H  
ATOM    463 1HB  SER    32       9.745   1.245  11.352  0.67 12.81           H  
ATOM    464 2HB  SER    32      10.629   2.476  10.444  0.67 13.26           H  
ATOM    465  HG  SER    32       8.434   2.853  10.028  0.67 15.33           H  
ATOM    466  N   ARG    33       9.073  -0.673   8.112  1.00  9.29           N  
ATOM    467  CA  ARG    33       8.205  -1.789   7.754  1.00  9.76           C  
ATOM    468  C   ARG    33       8.900  -2.977   7.136  1.00  9.93           C  
ATOM    469  O   ARG    33       8.283  -3.919   6.671  1.00 10.67           O  
ATOM    470  CB  ARG    33       7.051  -1.242   6.925  1.00  8.96           C  
ATOM    471  CG  ARG    33       6.132  -0.244   7.631  1.00  8.68           C  
ATOM    472  CD  ARG    33       5.384  -0.854   8.761  1.00  9.09           C  
ATOM    473  NE  ARG    33       4.453   0.042   9.408  1.00 10.69           N  
ATOM    474  CZ  ARG    33       3.219   0.321   8.939  1.00 10.73           C  
ATOM    475  NH1 ARG    33       2.710  -0.332   7.938  1.00  9.65           N  
ATOM    476  NH2 ARG    33       2.503   1.236   9.588  1.00 11.01           N  
ATOM    477  H   ARG    33       9.183   0.034   7.418  1.00 10.09           H  
ATOM    478  HA  ARG    33       7.784  -2.156   8.688  1.00  9.82           H  
ATOM    479 1HB  ARG    33       7.496  -0.721   6.077  1.00  8.94           H  
ATOM    480 2HB  ARG    33       6.499  -2.054   6.481  1.00  8.89           H  
ATOM    481 1HG  ARG    33       6.681   0.580   7.978  1.00  8.82           H  
ATOM    482 2HG  ARG    33       5.465   0.191   6.915  1.00  8.32           H  
ATOM    483 1HD  ARG    33       4.939  -1.775   8.492  1.00  8.91           H  
ATOM    484 2HD  ARG    33       6.082  -1.130   9.531  1.00  9.40           H  
ATOM    485 1HH1 ARG    33       3.190  -1.050   7.418  1.00 10.31           H  
ATOM    486 2HH1 ARG    33       1.760  -0.121   7.617  1.00 10.39           H  
ATOM    487 1HH2 ARG    33       2.858   1.752  10.371  1.00 11.91           H  
ATOM    488 2HH2 ARG    33       1.567   1.433   9.254  1.00 10.82           H  
ATOM    489  N   ASN    34      10.244  -2.884   7.086  1.00 10.07           N  
ATOM    490  CA  ASN    34      11.065  -4.023   6.561  1.00 11.18           C  
ATOM    491  C   ASN    34      10.907  -4.285   5.125  1.00 11.88           C  
ATOM    492  O   ASN    34      11.124  -5.408   4.601  1.00 13.71           O  
ATOM    493  CB  ASN    34      10.792  -5.218   7.498  0.33 12.52           C  
ATOM    494  CG  ASN    34      11.950  -6.137   7.744  0.33 13.52           C  
ATOM    495  OD1 ASN    34      13.066  -5.636   7.947  0.33 15.06           O  
ATOM    496  ND2 ASN    34      11.669  -7.433   7.765  0.33 14.14           N  
ATOM    497  H   ASN    34      10.764  -2.156   7.468  1.00  9.93           H  
ATOM    498  HA  ASN    34      12.084  -3.769   6.744  1.00 11.31           H  
ATOM    499 1HB  ASN    34      10.445  -4.843   8.457  0.33 12.36           H  
ATOM    500 2HB  ASN    34       9.953  -5.783   7.081  0.33 12.28           H  
ATOM    501 1HD2 ASN    34      12.388  -8.123   7.949  0.33 14.00           H  
ATOM    502 2HD2 ASN    34      10.733  -7.776   7.603  0.33 13.92           H  
ATOM    503  N   LEU    35      10.594  -3.191   4.393  1.00 11.16           N  
ATOM    504  CA  LEU    35      10.422  -3.284   2.920  1.00 12.28           C  
ATOM    505  C   LEU    35      11.716  -3.226   2.144  1.00 13.26           C  
ATOM    506  O   LEU    35      11.695  -3.655   0.954  1.00 15.70           O  
ATOM    507  CB  LEU    35       9.362  -2.289   2.430  1.00 11.32           C  
ATOM    508  CG  LEU    35       8.026  -2.437   3.230  1.00 13.29           C  
ATOM    509  CD1 LEU    35       7.058  -1.389   2.764  1.00 13.91           C  
ATOM    510  CD2 LEU    35       7.429  -3.799   3.250  1.00 14.73           C  
ATOM    511  H   LEU    35      10.453  -2.305   4.802  1.00 11.74           H  
ATOM    512  HA  LEU    35       9.995  -4.277   2.729  1.00 12.16           H  
ATOM    513 1HB  LEU    35       9.716  -1.277   2.595  1.00 11.36           H  
ATOM    514 2HB  LEU    35       9.132  -2.451   1.391  1.00 11.29           H  
ATOM    515  HG  LEU    35       8.264  -2.195   4.283  1.00 13.06           H  
ATOM    516 1HD1 LEU    35       7.559  -0.424   2.651  1.00 13.45           H  
ATOM    517 2HD1 LEU    35       6.636  -1.694   1.814  1.00 13.25           H  
ATOM    518 3HD1 LEU    35       6.291  -1.281   3.533  1.00 13.45           H  
ATOM    519 1HD2 LEU    35       8.061  -4.560   3.723  1.00 14.35           H  
ATOM    520 2HD2 LEU    35       6.498  -3.782   3.863  1.00 14.25           H  
ATOM    521 3HD2 LEU    35       7.163  -4.147   2.255  1.00 14.52           H  
ATOM    522  N   THR    36      12.739  -2.792   2.797  1.00 11.95           N  
ATOM    523  CA  THR    36      14.075  -2.636   2.147  1.00 12.97           C  
ATOM    524  C   THR    36      15.139  -3.598   2.663  1.00 14.23           C  
ATOM    525  O   THR    36      16.335  -3.322   2.501  1.00 15.20           O  
ATOM    526  CB  THR    36      14.495  -1.173   2.376  1.00 13.44           C  
ATOM    527  OG1 THR    36      14.806  -0.863   3.669  1.00 16.47           O  
ATOM    528  CG2 THR    36      13.442  -0.213   1.797  1.00 15.81           C  
ATOM    529  H   THR    36      12.700  -2.451   3.725  1.00 12.76           H  
ATOM    530  HA  THR    36      13.907  -2.692   1.117  1.00 13.08           H  
ATOM    531  HB  THR    36      15.423  -1.058   1.750  1.00 14.26           H  
ATOM    532  HG1 THR    36      14.328  -1.167   4.431  1.00 16.07           H  
ATOM    533 1HG2 THR    36      13.115  -0.526   0.787  1.00 14.60           H  
ATOM    534 2HG2 THR    36      12.540  -0.263   2.426  1.00 15.30           H  
ATOM    535 3HG2 THR    36      13.811   0.792   1.813  1.00 15.19           H  
ATOM    536  N   LYS    37      14.652  -4.652   3.291  1.00 15.58           N  
ATOM    537  CA  LYS    37      15.588  -5.601   3.962  1.00 16.62           C  
ATOM    538  C   LYS    37      16.451  -6.334   2.967  1.00 14.65           C  
ATOM    539  O   LYS    37      17.720  -6.383   3.245  1.00 16.23           O  
ATOM    540  CB  LYS    37      14.763  -6.515   4.883  1.00 20.85           C  
ATOM    541  CG  LYS    37      15.624  -7.555   5.606  1.00 26.92           C  
ATOM    542  CD  LYS    37      14.809  -8.455   6.515  1.00 31.31           C  
ATOM    543  CE  LYS    37      13.937  -9.460   5.795  1.00 34.60           C  
ATOM    544  NZ  LYS    37      14.714 -10.737   5.588  1.00 36.89           N  
ATOM    545  H   LYS    37      13.684  -4.819   3.438  1.00 15.07           H  
ATOM    546  HA  LYS    37      16.215  -5.012   4.613  1.00 16.24           H  
ATOM    547 1HB  LYS    37      14.249  -5.919   5.632  1.00 21.00           H  
ATOM    548 2HB  LYS    37      14.043  -7.027   4.243  1.00 20.80           H  
ATOM    549 1HG  LYS    37      16.181  -8.187   4.899  1.00 26.76           H  
ATOM    550 2HG  LYS    37      16.372  -7.046   6.227  1.00 26.73           H  
ATOM    551 1HD  LYS    37      15.537  -9.063   7.108  1.00 31.25           H  
ATOM    552 2HD  LYS    37      14.309  -7.826   7.225  1.00 31.14           H  
ATOM    553 1HE  LYS    37      13.050  -9.719   6.366  1.00 34.63           H  
ATOM    554 2HE  LYS    37      13.642  -9.150   4.800  1.00 34.52           H  
ATOM    555 1HZ  LYS    37      14.213 -11.519   6.032  1.00  0.00           H  
ATOM    556 2HZ  LYS    37      14.813 -10.920   4.579  1.00  0.00           H  
ATOM    557  N   ASP    38      15.880  -6.894   1.977  1.00 13.33           N  
ATOM    558  CA  ASP    38      16.592  -7.700   0.968  1.00 13.99           C  
ATOM    559  C   ASP    38      17.025  -6.940  -0.259  1.00 12.24           C  
ATOM    560  O   ASP    38      18.048  -7.296  -0.912  1.00 11.74           O  
ATOM    561  CB  ASP    38      15.849  -8.901   0.524  1.00 18.91           C  
ATOM    562  CG  ASP    38      15.597  -9.881   1.647  1.00 25.71           C  
ATOM    563  OD1 ASP    38      14.415 -10.277   1.765  1.00 30.14           O  
ATOM    564  OD2 ASP    38      16.611 -10.195   2.437  1.00 27.80           O  
ATOM    565  H   ASP    38      14.885  -6.851   1.838  1.00 15.64           H  
ATOM    566  HA  ASP    38      17.493  -8.074   1.467  1.00 14.40           H  
ATOM    567 1HB  ASP    38      14.859  -8.615   0.147  1.00 19.37           H  
ATOM    568 2HB  ASP    38      16.294  -9.416  -0.317  1.00 19.34           H  
ATOM    569  N   ARG    39      16.242  -5.909  -0.586  1.00 10.95           N  
ATOM    570  CA  ARG    39      16.453  -5.064  -1.767  1.00  9.98           C  
ATOM    571  C   ARG    39      15.573  -3.810  -1.631  1.00  9.58           C  
ATOM    572  O   ARG    39      14.684  -3.790  -0.816  1.00 10.36           O  
ATOM    573  CB  ARG    39      15.991  -5.853  -3.028  1.00 13.90           C  
ATOM    574  CG  ARG    39      14.488  -6.170  -3.003  1.00 19.47           C  
ATOM    575  CD  ARG    39      14.011  -7.285  -3.829  1.00 26.71           C  
ATOM    576  NE  ARG    39      12.529  -7.232  -3.963  1.00 32.87           N  
ATOM    577  CZ  ARG    39      11.834  -6.457  -4.800  1.00 34.99           C  
ATOM    578  NH1 ARG    39      12.377  -5.518  -5.575  1.00 35.76           N  
ATOM    579  NH2 ARG    39      10.511  -6.665  -4.902  1.00 36.36           N  
ATOM    580  H   ARG    39      15.443  -5.762  -0.013  1.00 11.70           H  
ATOM    581  HA  ARG    39      17.456  -4.811  -1.907  1.00 10.24           H  
ATOM    582 1HB  ARG    39      16.246  -5.286  -3.885  1.00 14.10           H  
ATOM    583 2HB  ARG    39      16.588  -6.748  -3.079  1.00 14.46           H  
ATOM    584 1HG  ARG    39      14.138  -6.304  -2.017  1.00 19.66           H  
ATOM    585 2HG  ARG    39      13.963  -5.267  -3.308  1.00 19.74           H  
ATOM    586 1HD  ARG    39      14.468  -7.370  -4.762  1.00 26.73           H  
ATOM    587 2HD  ARG    39      14.228  -8.205  -3.307  1.00 26.58           H  
ATOM    588 1HH1 ARG    39      11.795  -4.982  -6.203  1.00 35.45           H  
ATOM    589 2HH1 ARG    39      13.372  -5.326  -5.553  1.00 35.74           H  
ATOM    590 1HH2 ARG    39      10.038  -7.355  -4.344  1.00 35.79           H  
ATOM    591 2HH2 ARG    39       9.971  -6.094  -5.538  1.00 35.59           H  
ATOM    592  N   CYS    40      15.865  -2.813  -2.416  1.00  9.39           N  
ATOM    593  CA  CYS    40      14.971  -1.625  -2.370  1.00 10.48           C  
ATOM    594  C   CYS    40      13.732  -1.985  -3.209  1.00 11.90           C  
ATOM    595  O   CYS    40      13.850  -2.174  -4.422  1.00 14.02           O  
ATOM    596  CB  CYS    40      15.658  -0.411  -2.939  1.00 10.88           C  
ATOM    597  SG  CYS    40      17.281  -0.013  -2.243  1.00 10.58           S  
ATOM    598  H   CYS    40      16.574  -2.828  -3.114  1.00  9.45           H  
ATOM    599  HA  CYS    40      14.682  -1.411  -1.379  1.00 10.36           H  
ATOM    600 1HB  CYS    40      15.827  -0.568  -3.999  1.00 10.87           H  
ATOM    601 2HB  CYS    40      15.051   0.460  -2.795  1.00 10.73           H  
ATOM    602  N   LYS    41      12.621  -2.017  -2.557  1.00 13.71           N  
ATOM    603  CA  LYS    41      11.307  -2.202  -3.303  1.00 14.33           C  
ATOM    604  C   LYS    41      11.111  -0.928  -4.088  1.00 13.87           C  
ATOM    605  O   LYS    41      11.208   0.175  -3.460  1.00 14.03           O  
ATOM    606  CB  LYS    41      10.244  -2.359  -2.225  1.00 18.30           C  
ATOM    607  CG  LYS    41       8.877  -2.604  -2.747  1.00 22.51           C  
ATOM    608  CD  LYS    41       7.720  -2.806  -1.773  1.00 26.35           C  
ATOM    609  CE  LYS    41       6.533  -3.461  -2.492  1.00 29.84           C  
ATOM    610  NZ  LYS    41       5.208  -2.854  -2.237  1.00 30.76           N  
ATOM    611  H   LYS    41      12.526  -1.804  -1.585  1.00 13.13           H  
ATOM    612  HA  LYS    41      11.377  -3.076  -3.875  1.00 14.75           H  
ATOM    613 1HB  LYS    41      10.575  -3.230  -1.625  1.00 18.38           H  
ATOM    614 2HB  LYS    41      10.254  -1.477  -1.576  1.00 18.38           H  
ATOM    615 1HG  LYS    41       8.559  -1.793  -3.423  1.00 22.51           H  
ATOM    616 2HG  LYS    41       8.923  -3.507  -3.424  1.00 22.76           H  
ATOM    617 1HD  LYS    41       8.045  -3.473  -0.972  1.00 26.43           H  
ATOM    618 2HD  LYS    41       7.395  -1.871  -1.349  1.00 26.35           H  
ATOM    619 1HE  LYS    41       6.641  -3.426  -3.585  1.00 29.68           H  
ATOM    620 2HE  LYS    41       6.473  -4.517  -2.202  1.00 29.56           H  
ATOM    621 1HZ  LYS    41       4.478  -3.421  -2.690  1.00  0.00           H  
ATOM    622 2HZ  LYS    41       5.036  -2.819  -1.223  1.00  0.00           H  
ATOM    623  N   PRO    42      10.887  -0.874  -5.347  1.00 13.40           N  
ATOM    624  CA  PRO    42      10.813   0.276  -6.148  1.00 13.31           C  
ATOM    625  C   PRO    42       9.672   1.277  -5.965  1.00 13.33           C  
ATOM    626  O   PRO    42       9.860   2.502  -6.148  1.00 13.62           O  
ATOM    627  CB  PRO    42      10.904  -0.161  -7.601  1.00 14.92           C  
ATOM    628  CG  PRO    42      10.381  -1.565  -7.563  1.00 15.89           C  
ATOM    629  CD  PRO    42      10.845  -2.119  -6.245  1.00 14.59           C  
ATOM    630  HA  PRO    42      11.629   0.940  -5.910  1.00 13.94           H  
ATOM    631 1HB  PRO    42      10.434   0.397  -8.266  1.00 14.77           H  
ATOM    632 2HB  PRO    42      11.914  -0.118  -7.937  1.00 15.15           H  
ATOM    633 1HG  PRO    42       9.452  -1.571  -7.595  1.00 15.92           H  
ATOM    634 2HG  PRO    42      10.757  -2.168  -8.378  1.00 15.86           H  
ATOM    635 1HD  PRO    42      10.200  -2.862  -5.864  1.00 14.44           H  
ATOM    636 2HD  PRO    42      11.850  -2.515  -6.235  1.00 14.67           H  
ATOM    637  N   VAL    43       8.533   0.726  -5.611  1.00 11.25           N  
ATOM    638  CA  VAL    43       7.322   1.519  -5.425  1.00 11.41           C  
ATOM    639  C   VAL    43       6.498   0.917  -4.284  1.00 10.32           C  
ATOM    640  O   VAL    43       6.481  -0.305  -4.148  1.00 11.83           O  
ATOM    641  CB  VAL    43       6.382   1.505  -6.662  0.40 12.20           C  
ATOM    642  CG1 VAL    43       5.274   2.582  -6.592  0.40 12.62           C  
ATOM    643  CG2 VAL    43       7.034   1.482  -8.017  0.40 12.59           C  
ATOM    644  H   VAL    43       8.446  -0.268  -5.509  1.00 11.98           H  
ATOM    645  HA  VAL    43       7.587   2.516  -5.226  1.00 11.62           H  
ATOM    646  HB  VAL    43       5.859   0.526  -6.576  0.40 12.03           H  
ATOM    647 1HG1 VAL    43       5.638   3.561  -6.413  0.40 12.41           H  
ATOM    648 2HG1 VAL    43       4.680   2.575  -7.476  0.40 12.56           H  
ATOM    649 3HG1 VAL    43       4.581   2.345  -5.799  0.40 12.38           H  
ATOM    650 1HG2 VAL    43       7.668   0.605  -8.127  0.40 12.60           H  
ATOM    651 2HG2 VAL    43       6.321   1.427  -8.825  0.40 12.55           H  
ATOM    652 3HG2 VAL    43       7.662   2.338  -8.186  0.40 12.59           H  
ATOM    653  N   ASN    44       5.817   1.817  -3.584  1.00  8.69           N  
ATOM    654  CA  ASN    44       4.916   1.336  -2.548  1.00  8.24           C  
ATOM    655  C   ASN    44       3.875   2.457  -2.255  1.00  8.94           C  
ATOM    656  O   ASN    44       4.188   3.621  -2.449  1.00 11.02           O  
ATOM    657  CB  ASN    44       5.669   0.865  -1.337  1.00  8.72           C  
ATOM    658  CG  ASN    44       4.792   0.103  -0.355  1.00 10.65           C  
ATOM    659  OD1 ASN    44       4.124  -0.848  -0.785  1.00 13.81           O  
ATOM    660  ND2 ASN    44       4.684   0.549   0.899  1.00 10.34           N  
ATOM    661  H   ASN    44       5.888   2.781  -3.809  1.00  9.81           H  
ATOM    662  HA  ASN    44       4.397   0.476  -2.905  1.00  8.12           H  
ATOM    663 1HB  ASN    44       6.533   0.261  -1.575  1.00  8.67           H  
ATOM    664 2HB  ASN    44       6.054   1.747  -0.816  1.00  8.57           H  
ATOM    665 1HD2 ASN    44       4.096   0.065   1.565  1.00 10.59           H  
ATOM    666 2HD2 ASN    44       5.255   1.327   1.155  1.00 10.86           H  
ATOM    667  N   THR    45       2.730   2.057  -1.792  1.00  8.39           N  
ATOM    668  CA  THR    45       1.651   2.971  -1.384  1.00  8.68           C  
ATOM    669  C   THR    45       1.248   2.655   0.068  1.00  8.56           C  
ATOM    670  O   THR    45       1.047   1.486   0.402  1.00  9.16           O  
ATOM    671  CB  THR    45       0.382   2.836  -2.260  1.00 10.26           C  
ATOM    672  OG1 THR    45       0.804   3.034  -3.606  1.00 11.67           O  
ATOM    673  CG2 THR    45      -0.701   3.864  -1.939  1.00 11.34           C  
ATOM    674  H   THR    45       2.515   1.081  -1.631  1.00  9.05           H  
ATOM    675  HA  THR    45       2.004   3.948  -1.460  1.00  8.64           H  
ATOM    676  HB  THR    45      -0.012   1.814  -2.156  1.00 10.11           H  
ATOM    677  HG1 THR    45       1.145   3.937  -3.762  1.00 11.11           H  
ATOM    678 1HG2 THR    45      -0.317   4.838  -1.749  1.00 10.91           H  
ATOM    679 2HG2 THR    45      -1.433   3.947  -2.774  1.00 11.18           H  
ATOM    680 3HG2 THR    45      -1.255   3.505  -1.067  1.00 10.43           H  
ATOM    681  N   PHE    46       1.166   3.680   0.859  1.00  7.62           N  
ATOM    682  CA  PHE    46       0.633   3.624   2.240  1.00  7.79           C  
ATOM    683  C   PHE    46      -0.793   4.300   2.245  1.00  8.04           C  
ATOM    684  O   PHE    46      -1.014   5.305   1.587  1.00  8.49           O  
ATOM    685  CB  PHE    46       1.526   4.316   3.261  1.00  8.02           C  
ATOM    686  CG  PHE    46       2.818   3.554   3.461  1.00  8.47           C  
ATOM    687  CD1 PHE    46       2.852   2.482   4.330  1.00 10.22           C  
ATOM    688  CD2 PHE    46       3.949   3.954   2.798  1.00  8.79           C  
ATOM    689  CE1 PHE    46       4.057   1.750   4.484  1.00 10.60           C  
ATOM    690  CE2 PHE    46       5.166   3.317   2.984  1.00 10.90           C  
ATOM    691  CZ  PHE    46       5.193   2.188   3.831  1.00 10.62           C  
ATOM    692  H   PHE    46       1.287   4.631   0.521  1.00  8.35           H  
ATOM    693  HA  PHE    46       0.552   2.651   2.564  1.00  7.46           H  
ATOM    694 1HB  PHE    46       1.719   5.329   2.921  1.00  7.69           H  
ATOM    695 2HB  PHE    46       0.995   4.435   4.182  1.00  7.89           H  
ATOM    696  HD1 PHE    46       1.998   2.163   4.805  1.00  9.43           H  
ATOM    697  HD2 PHE    46       3.912   4.826   2.149  1.00  9.35           H  
ATOM    698  HE1 PHE    46       4.092   0.937   5.141  1.00 10.10           H  
ATOM    699  HE2 PHE    46       6.022   3.590   2.497  1.00 10.23           H  
ATOM    700  HZ  PHE    46       6.108   1.672   3.966  1.00 11.00           H  
ATOM    701  N   VAL    47      -1.686   3.678   3.008  1.00  8.13           N  
ATOM    702  CA  VAL    47      -3.069   4.169   3.128  1.00  7.95           C  
ATOM    703  C   VAL    47      -3.290   4.717   4.555  1.00  8.02           C  
ATOM    704  O   VAL    47      -3.111   3.993   5.539  1.00  9.28           O  
ATOM    705  CB  VAL    47      -4.040   3.089   2.728  1.00  8.87           C  
ATOM    706  CG1 VAL    47      -5.512   3.550   2.796  1.00 10.17           C  
ATOM    707  CG2 VAL    47      -3.702   2.405   1.398  1.00  9.87           C  
ATOM    708  H   VAL    47      -1.451   2.858   3.522  1.00  8.51           H  
ATOM    709  HA  VAL    47      -3.209   4.976   2.437  1.00  8.31           H  
ATOM    710  HB  VAL    47      -3.895   2.320   3.504  1.00  8.95           H  
ATOM    711 1HG1 VAL    47      -5.792   3.926   3.744  1.00 10.00           H  
ATOM    712 2HG1 VAL    47      -5.713   4.266   2.035  1.00  9.70           H  
ATOM    713 3HG1 VAL    47      -6.184   2.733   2.572  1.00  9.49           H  
ATOM    714 1HG2 VAL    47      -2.892   2.872   0.872  1.00  9.83           H  
ATOM    715 2HG2 VAL    47      -3.453   1.363   1.531  1.00 10.02           H  
ATOM    716 3HG2 VAL    47      -4.538   2.395   0.713  1.00 10.18           H  
ATOM    717  N   HIS    48      -3.735   5.950   4.627  1.00  7.63           N  
ATOM    718  CA  HIS    48      -3.964   6.622   5.897  1.00  9.13           C  
ATOM    719  C   HIS    48      -5.412   6.638   6.351  1.00 10.00           C  
ATOM    720  O   HIS    48      -6.069   7.722   6.389  1.00 11.67           O  
ATOM    721  CB  HIS    48      -3.422   8.052   5.790  1.00  8.79           C  
ATOM    722  CG  HIS    48      -1.984   8.164   5.401  1.00  8.32           C  
ATOM    723  ND1 HIS    48      -1.010   7.966   6.296  1.00 12.34           N  
ATOM    724  CD2 HIS    48      -1.447   8.504   4.191  1.00  9.89           C  
ATOM    725  CE1 HIS    48       0.137   8.151   5.615  1.00 12.46           C  
ATOM    726  NE2 HIS    48      -0.094   8.442   4.348  1.00 10.99           N  
ATOM    727  H   HIS    48      -3.860   6.494   3.787  1.00  8.10           H  
ATOM    728  HA  HIS    48      -3.423   6.119   6.643  1.00  9.27           H  
ATOM    729 1HB  HIS    48      -4.004   8.538   4.964  1.00  8.79           H  
ATOM    730 2HB  HIS    48      -3.668   8.596   6.668  1.00  8.60           H  
ATOM    731  HD1 HIS    48      -1.056   7.710   7.296  1.00 11.12           H  
ATOM    732  HD2 HIS    48      -1.911   8.725   3.243  1.00 10.38           H  
ATOM    733  HE1 HIS    48       1.082   8.070   6.099  1.00 11.61           H  
ATOM    734  N   GLU    49      -5.941   5.486   6.544  1.00  9.98           N  
ATOM    735  CA  GLU    49      -7.318   5.225   6.883  1.00  9.75           C  
ATOM    736  C   GLU    49      -7.378   4.127   7.894  1.00  9.85           C  
ATOM    737  O   GLU    49      -6.426   3.380   8.159  1.00 10.72           O  
ATOM    738  CB  GLU    49      -8.115   4.794   5.619  1.00 11.40           C  
ATOM    739  CG  GLU    49      -8.162   5.806   4.472  1.00 12.91           C  
ATOM    740  CD  GLU    49      -8.849   7.141   4.677  1.00 14.48           C  
ATOM    741  OE1 GLU    49      -8.658   8.106   3.955  1.00 13.19           O  
ATOM    742  OE2 GLU    49      -9.636   7.189   5.723  1.00 16.33           O  
ATOM    743  H   GLU    49      -5.378   4.632   6.426  1.00 10.58           H  
ATOM    744  HA  GLU    49      -7.777   6.119   7.269  1.00 10.34           H  
ATOM    745 1HB  GLU    49      -7.678   3.872   5.277  1.00 11.58           H  
ATOM    746 2HB  GLU    49      -9.154   4.588   5.895  1.00 11.13           H  
ATOM    747 1HG  GLU    49      -7.145   6.033   4.170  1.00 12.84           H  
ATOM    748 2HG  GLU    49      -8.624   5.382   3.586  1.00 13.18           H  
ATOM    749  N   SER    50      -8.544   3.997   8.509  1.00 10.01           N  
ATOM    750  CA  SER    50      -8.751   2.888   9.470  1.00 10.30           C  
ATOM    751  C   SER    50      -8.645   1.554   8.658  1.00  9.73           C  
ATOM    752  O   SER    50      -9.021   1.451   7.497  1.00 10.48           O  
ATOM    753  CB  SER    50     -10.085   2.981  10.226  0.67 10.73           C  
ATOM    754  OG  SER    50     -11.101   2.706   9.359  0.67 10.90           O  
ATOM    755  H   SER    50      -9.343   4.561   8.294  1.00 10.44           H  
ATOM    756  HA  SER    50      -8.043   2.896  10.228  1.00 10.35           H  
ATOM    757 1HB  SER    50     -10.035   2.328  11.106  0.67 10.54           H  
ATOM    758 2HB  SER    50     -10.178   4.005  10.607  0.67 10.55           H  
ATOM    759  HG  SER    50     -11.856   2.302   9.788  0.67 12.15           H  
ATOM    760  N   LEU    51      -8.154   0.531   9.362  1.00 10.39           N  
ATOM    761  CA  LEU    51      -8.085  -0.811   8.772  1.00 10.90           C  
ATOM    762  C   LEU    51      -9.471  -1.282   8.408  1.00 10.49           C  
ATOM    763  O   LEU    51      -9.671  -1.885   7.351  1.00 10.70           O  
ATOM    764  CB  LEU    51      -7.354  -1.805   9.691  1.00 12.87           C  
ATOM    765  CG  LEU    51      -7.227  -3.221   9.191  1.00 14.39           C  
ATOM    766  CD1 LEU    51      -6.410  -3.335   7.949  1.00 14.44           C  
ATOM    767  CD2 LEU    51      -6.662  -4.119  10.280  1.00 15.89           C  
ATOM    768  H   LEU    51      -7.887   0.628  10.316  1.00 10.71           H  
ATOM    769  HA  LEU    51      -7.500  -0.713   7.863  1.00 10.88           H  
ATOM    770 1HB  LEU    51      -6.336  -1.428   9.870  1.00 12.79           H  
ATOM    771 2HB  LEU    51      -7.877  -1.810  10.661  1.00 12.83           H  
ATOM    772  HG  LEU    51      -8.243  -3.595   9.022  1.00 14.30           H  
ATOM    773 1HD1 LEU    51      -6.759  -2.712   7.135  1.00 13.84           H  
ATOM    774 2HD1 LEU    51      -5.375  -3.116   8.183  1.00 14.53           H  
ATOM    775 3HD1 LEU    51      -6.459  -4.383   7.597  1.00 14.27           H  
ATOM    776 1HD2 LEU    51      -5.722  -3.729  10.657  1.00 15.90           H  
ATOM    777 2HD2 LEU    51      -7.373  -4.137  11.130  1.00 15.67           H  
ATOM    778 3HD2 LEU    51      -6.525  -5.133   9.905  1.00 15.70           H  
ATOM    779  N   ALA    52     -10.482  -1.055   9.248  1.00 10.66           N  
ATOM    780  CA  ALA    52     -11.850  -1.441   8.905  1.00 11.00           C  
ATOM    781  C   ALA    52     -12.350  -0.825   7.626  1.00 10.92           C  
ATOM    782  O   ALA    52     -13.013  -1.531   6.818  1.00 12.20           O  
ATOM    783  CB  ALA    52     -12.775  -1.123  10.078  1.00 12.01           C  
ATOM    784  H   ALA    52     -10.299  -0.556  10.102  1.00 11.67           H  
ATOM    785  HA  ALA    52     -11.861  -2.517   8.804  1.00 11.49           H  
ATOM    786 1HB  ALA    52     -12.396  -1.603  10.966  1.00 11.57           H  
ATOM    787 2HB  ALA    52     -12.808  -0.052  10.241  1.00 11.24           H  
ATOM    788 3HB  ALA    52     -13.765  -1.469   9.851  1.00 11.55           H  
ATOM    789  N   ASP    53     -12.072   0.454   7.350  1.00 10.24           N  
ATOM    790  CA  ASP    53     -12.463   1.102   6.148  1.00 10.07           C  
ATOM    791  C   ASP    53     -11.799   0.453   4.892  1.00  9.15           C  
ATOM    792  O   ASP    53     -12.425   0.435   3.858  1.00 10.51           O  
ATOM    793  CB  ASP    53     -12.280   2.608   6.113  1.00 11.70           C  
ATOM    794  CG  ASP    53     -13.247   3.353   7.000  1.00 15.29           C  
ATOM    795  OD1 ASP    53     -14.257   2.838   7.443  1.00 18.56           O  
ATOM    796  OD2 ASP    53     -12.977   4.567   7.218  1.00 18.77           O  
ATOM    797  H   ASP    53     -11.524   0.968   8.024  1.00 10.68           H  
ATOM    798  HA  ASP    53     -13.529   0.969   5.978  1.00 10.03           H  
ATOM    799 1HB  ASP    53     -11.290   2.874   6.460  1.00 11.90           H  
ATOM    800 2HB  ASP    53     -12.340   3.006   5.113  1.00 11.66           H  
ATOM    801  N   VAL    54     -10.553   0.053   5.063  1.00  8.38           N  
ATOM    802  CA  VAL    54      -9.855  -0.587   3.937  1.00  8.93           C  
ATOM    803  C   VAL    54     -10.371  -2.037   3.735  1.00  8.92           C  
ATOM    804  O   VAL    54     -10.545  -2.435   2.587  1.00  9.70           O  
ATOM    805  CB  VAL    54      -8.332  -0.487   4.065  1.00  9.20           C  
ATOM    806  CG1 VAL    54      -7.571  -1.311   3.027  1.00 10.26           C  
ATOM    807  CG2 VAL    54      -7.889   0.970   4.070  1.00 10.41           C  
ATOM    808  H   VAL    54     -10.083   0.107   5.937  1.00  8.51           H  
ATOM    809  HA  VAL    54     -10.082  -0.027   3.057  1.00  8.33           H  
ATOM    810  HB  VAL    54      -8.107  -0.906   5.045  1.00  9.49           H  
ATOM    811 1HG1 VAL    54      -7.788  -1.031   2.025  1.00  9.79           H  
ATOM    812 2HG1 VAL    54      -6.502  -1.159   3.125  1.00  9.88           H  
ATOM    813 3HG1 VAL    54      -7.737  -2.362   3.118  1.00  9.96           H  
ATOM    814 1HG2 VAL    54      -8.380   1.550   4.833  1.00  9.37           H  
ATOM    815 2HG2 VAL    54      -6.831   1.072   4.223  1.00  9.64           H  
ATOM    816 3HG2 VAL    54      -8.114   1.422   3.118  1.00  9.55           H  
ATOM    817  N   GLN    55     -10.566  -2.726   4.826  1.00  9.25           N  
ATOM    818  CA  GLN    55     -11.135  -4.103   4.713  1.00 10.12           C  
ATOM    819  C   GLN    55     -12.482  -4.090   4.040  1.00 10.65           C  
ATOM    820  O   GLN    55     -12.855  -4.983   3.251  1.00 11.35           O  
ATOM    821  CB  GLN    55     -11.252  -4.771   6.066  1.00 11.98           C  
ATOM    822  CG  GLN    55      -9.909  -5.136   6.668  1.00 12.39           C  
ATOM    823  CD  GLN    55     -10.059  -5.543   8.137  1.00 13.70           C  
ATOM    824  OE1 GLN    55     -10.958  -5.155   8.883  1.00 16.51           O  
ATOM    825  NE2 GLN    55      -9.112  -6.289   8.542  1.00 14.45           N  
ATOM    826  H   GLN    55     -10.434  -2.353   5.733  1.00  9.49           H  
ATOM    827  HA  GLN    55     -10.431  -4.668   4.156  1.00 10.60           H  
ATOM    828 1HB  GLN    55     -11.767  -4.028   6.729  1.00 11.67           H  
ATOM    829 2HB  GLN    55     -11.932  -5.610   6.020  1.00 11.67           H  
ATOM    830 1HG  GLN    55      -9.503  -6.021   6.188  1.00 12.70           H  
ATOM    831 2HG  GLN    55      -9.149  -4.377   6.618  1.00 12.40           H  
ATOM    832 1HE2 GLN    55      -9.016  -6.666   9.469  1.00 14.88           H  
ATOM    833 2HE2 GLN    55      -8.342  -6.641   7.970  1.00 15.89           H  
ATOM    834  N   ALA    56     -13.285  -3.038   4.303  1.00 10.96           N  
ATOM    835  CA  ALA    56     -14.592  -2.872   3.701  1.00 11.65           C  
ATOM    836  C   ALA    56     -14.622  -2.772   2.184  1.00 11.33           C  
ATOM    837  O   ALA    56     -15.620  -3.020   1.526  1.00 11.29           O  
ATOM    838  CB  ALA    56     -15.326  -1.645   4.273  1.00 12.49           C  
ATOM    839  H   ALA    56     -12.958  -2.320   4.934  1.00 11.39           H  
ATOM    840  HA  ALA    56     -15.206  -3.711   3.998  1.00 12.08           H  
ATOM    841 1HB  ALA    56     -15.095  -1.547   5.324  1.00 12.48           H  
ATOM    842 2HB  ALA    56     -15.025  -0.758   3.752  1.00 12.13           H  
ATOM    843 3HB  ALA    56     -16.392  -1.795   4.186  1.00 12.33           H  
ATOM    844  N   VAL    57     -13.473  -2.379   1.572  1.00 10.27           N  
ATOM    845  CA  VAL    57     -13.369  -2.292   0.116  1.00  9.75           C  
ATOM    846  C   VAL    57     -13.665  -3.658  -0.556  1.00  9.70           C  
ATOM    847  O   VAL    57     -14.132  -3.663  -1.689  1.00 10.25           O  
ATOM    848  CB  VAL    57     -12.041  -1.634  -0.283  1.00  9.76           C  
ATOM    849  CG1 VAL    57     -11.835  -1.596  -1.792  1.00 10.77           C  
ATOM    850  CG2 VAL    57     -11.908  -0.273   0.280  1.00 10.34           C  
ATOM    851  H   VAL    57     -12.666  -2.196   2.125  1.00 10.27           H  
ATOM    852  HA  VAL    57     -14.083  -1.620  -0.294  1.00  9.67           H  
ATOM    853  HB  VAL    57     -11.287  -2.303   0.142  1.00 10.14           H  
ATOM    854 1HG1 VAL    57     -12.686  -1.253  -2.321  1.00 10.29           H  
ATOM    855 2HG1 VAL    57     -10.974  -1.050  -2.100  1.00 11.01           H  
ATOM    856 3HG1 VAL    57     -11.697  -2.611  -2.150  1.00 10.58           H  
ATOM    857 1HG2 VAL    57     -12.071  -0.207   1.341  1.00 10.13           H  
ATOM    858 2HG2 VAL    57     -10.919   0.154   0.122  1.00 10.24           H  
ATOM    859 3HG2 VAL    57     -12.569   0.462  -0.175  1.00 10.06           H  
ATOM    860  N   CYS    58     -13.404  -4.715   0.166  1.00  9.87           N  
ATOM    861  CA  CYS    58     -13.675  -6.073  -0.346  1.00 11.50           C  
ATOM    862  C   CYS    58     -15.148  -6.342  -0.630  1.00 12.58           C  
ATOM    863  O   CYS    58     -15.471  -7.362  -1.272  1.00 14.45           O  
ATOM    864  CB  CYS    58     -13.062  -7.156   0.537  1.00 11.76           C  
ATOM    865  SG  CYS    58     -11.290  -7.126   0.585  1.00 11.86           S  
ATOM    866  H   CYS    58     -13.042  -4.665   1.094  1.00  9.88           H  
ATOM    867  HA  CYS    58     -13.162  -6.139  -1.299  1.00 11.17           H  
ATOM    868 1HB  CYS    58     -13.433  -7.113   1.541  1.00 11.75           H  
ATOM    869 2HB  CYS    58     -13.341  -8.116   0.131  1.00 11.49           H  
ATOM    870  N   SER    59     -15.979  -5.492  -0.174  1.00 13.49           N  
ATOM    871  CA  SER    59     -17.454  -5.569  -0.405  1.00 15.14           C  
ATOM    872  C   SER    59     -17.896  -4.378  -1.256  1.00 14.81           C  
ATOM    873  O   SER    59     -19.151  -4.105  -1.307  1.00 15.90           O  
ATOM    874  CB  SER    59     -18.171  -5.637   0.950  1.00 18.12           C  
ATOM    875  OG  SER    59     -18.063  -4.374   1.579  1.00 22.57           O  
ATOM    876  H   SER    59     -15.766  -4.674   0.359  1.00 13.21           H  
ATOM    877  HA  SER    59     -17.657  -6.483  -0.901  1.00 15.46           H  
ATOM    878 1HB  SER    59     -19.228  -5.880   0.783  1.00 18.42           H  
ATOM    879 2HB  SER    59     -17.740  -6.387   1.584  1.00 17.93           H  
ATOM    880  HG  SER    59     -17.901  -4.532   2.524  1.00 21.80           H  
ATOM    881  N   GLN    60     -17.052  -3.717  -1.966  1.00 13.71           N  
ATOM    882  CA  GLN    60     -17.342  -2.530  -2.775  1.00 13.13           C  
ATOM    883  C   GLN    60     -17.374  -2.837  -4.274  1.00 13.75           C  
ATOM    884  O   GLN    60     -17.916  -3.954  -4.605  1.00 15.06           O  
ATOM    885  CB  GLN    60     -16.564  -1.333  -2.303  1.00 11.26           C  
ATOM    886  CG  GLN    60     -16.962  -0.999  -0.880  1.00 12.55           C  
ATOM    887  CD  GLN    60     -16.216   0.175  -0.307  1.00 12.52           C  
ATOM    888  OE1 GLN    60     -15.570   0.970  -0.953  1.00 14.11           O  
ATOM    889  NE2 GLN    60     -16.276   0.236   0.997  1.00 14.29           N  
ATOM    890  H   GLN    60     -16.059  -3.957  -1.902  1.00 13.71           H  
ATOM    891  HA  GLN    60     -18.380  -2.259  -2.506  1.00 12.92           H  
ATOM    892 1HB  GLN    60     -15.493  -1.471  -2.372  1.00 11.78           H  
ATOM    893 2HB  GLN    60     -16.806  -0.450  -2.911  1.00 11.68           H  
ATOM    894 1HG  GLN    60     -18.016  -0.796  -0.813  1.00 11.85           H  
ATOM    895 2HG  GLN    60     -16.812  -1.845  -0.215  1.00 12.01           H  
ATOM    896 1HE2 GLN    60     -15.821   0.961   1.536  1.00 14.25           H  
ATOM    897 2HE2 GLN    60     -16.794  -0.403   1.584  1.00 15.18           H  
ATOM    898  N   LYS    61     -16.901  -2.052  -5.139  1.00 14.10           N  
ATOM    899  CA  LYS    61     -17.032  -2.276  -6.606  1.00 15.68           C  
ATOM    900  C   LYS    61     -16.099  -3.319  -7.145  1.00 15.12           C  
ATOM    901  O   LYS    61     -14.912  -3.050  -7.181  1.00 15.69           O  
ATOM    902  CB  LYS    61     -16.827  -0.941  -7.310  0.50 18.32           C  
ATOM    903  CG  LYS    61     -17.179  -0.891  -8.797  0.50 21.14           C  
ATOM    904  CD  LYS    61     -17.156   0.513  -9.383  0.50 23.46           C  
ATOM    905  CE  LYS    61     -18.094   0.678 -10.570  0.50 25.72           C  
ATOM    906  NZ  LYS    61     -18.400   2.084 -10.934  0.50 27.05           N  
ATOM    907  H   LYS    61     -16.509  -1.157  -4.892  1.00 14.09           H  
ATOM    908  HA  LYS    61     -18.055  -2.596  -6.763  1.00 15.93           H  
ATOM    909 1HB  LYS    61     -17.426  -0.178  -6.797  0.50 18.43           H  
ATOM    910 2HB  LYS    61     -15.771  -0.665  -7.185  0.50 18.35           H  
ATOM    911 1HG  LYS    61     -16.480  -1.532  -9.368  0.50 21.13           H  
ATOM    912 2HG  LYS    61     -18.162  -1.341  -8.951  0.50 21.14           H  
ATOM    913 1HD  LYS    61     -17.432   1.245  -8.631  0.50 23.45           H  
ATOM    914 2HD  LYS    61     -16.136   0.741  -9.719  0.50 23.41           H  
ATOM    915 1HE  LYS    61     -17.697   0.182 -11.450  0.50 25.61           H  
ATOM    916 2HE  LYS    61     -19.069   0.230 -10.320  0.50 25.57           H  
ATOM    917 1HZ  LYS    61     -18.990   2.507 -10.203  1.00  0.00           H  
ATOM    918 2HZ  LYS    61     -17.522   2.616 -11.017  1.00  0.00           H  
ATOM    919  N   ASN    62     -16.621  -4.408  -7.618  1.00 14.50           N  
ATOM    920  CA  ASN    62     -15.796  -5.454  -8.232  1.00 15.69           C  
ATOM    921  C   ASN    62     -15.292  -4.944  -9.546  1.00 15.84           C  
ATOM    922  O   ASN    62     -16.029  -4.367 -10.401  1.00 16.77           O  
ATOM    923  CB  ASN    62     -16.532  -6.804  -8.273  1.00 18.58           C  
ATOM    924  CG  ASN    62     -15.715  -7.914  -8.943  1.00 21.04           C  
ATOM    925  OD1 ASN    62     -16.086  -8.449 -10.014  1.00 24.34           O  
ATOM    926  ND2 ASN    62     -14.551  -8.276  -8.393  1.00 18.96           N  
ATOM    927  H   ASN    62     -17.617  -4.582  -7.608  1.00 15.72           H  
ATOM    928  HA  ASN    62     -14.996  -5.670  -7.558  1.00 15.77           H  
ATOM    929 1HB  ASN    62     -16.853  -7.123  -7.296  1.00 18.40           H  
ATOM    930 2HB  ASN    62     -17.447  -6.663  -8.851  1.00 18.67           H  
ATOM    931 1HD2 ASN    62     -14.261  -7.814  -7.534  1.00 19.10           H  
ATOM    932 2HD2 ASN    62     -13.965  -8.989  -8.801  1.00 19.34           H  
ATOM    933  N   VAL    63     -13.961  -5.110  -9.819  1.00 14.02           N  
ATOM    934  CA  VAL    63     -13.247  -4.685 -10.987  1.00 14.10           C  
ATOM    935  C   VAL    63     -12.093  -5.682 -11.271  1.00 13.70           C  
ATOM    936  O   VAL    63     -11.627  -6.425 -10.413  1.00 14.92           O  
ATOM    937  CB  VAL    63     -12.758  -3.236 -10.919  1.00 15.26           C  
ATOM    938  CG1 VAL    63     -13.881  -2.250 -10.645  1.00 18.30           C  
ATOM    939  CG2 VAL    63     -11.667  -2.970  -9.914  1.00 15.37           C  
ATOM    940  H   VAL    63     -13.429  -5.564  -9.081  1.00 14.37           H  
ATOM    941  HA  VAL    63     -13.874  -4.711 -11.862  1.00 14.58           H  
ATOM    942  HB  VAL    63     -12.345  -3.018 -11.917  1.00 15.45           H  
ATOM    943 1HG1 VAL    63     -14.703  -2.408 -11.317  1.00 17.69           H  
ATOM    944 2HG1 VAL    63     -14.274  -2.341  -9.640  1.00 18.02           H  
ATOM    945 3HG1 VAL    63     -13.558  -1.239 -10.757  1.00 18.22           H  
ATOM    946 1HG2 VAL    63     -11.904  -3.300  -8.929  1.00 15.21           H  
ATOM    947 2HG2 VAL    63     -10.737  -3.443 -10.212  1.00 15.17           H  
ATOM    948 3HG2 VAL    63     -11.434  -1.911  -9.851  1.00 15.19           H  
ATOM    949  N   ALA    64     -11.729  -5.732 -12.536  1.00 12.88           N  
ATOM    950  CA  ALA    64     -10.595  -6.602 -12.926  1.00 11.64           C  
ATOM    951  C   ALA    64      -9.330  -5.915 -12.399  1.00 11.24           C  
ATOM    952  O   ALA    64      -9.139  -4.664 -12.441  1.00 12.55           O  
ATOM    953  CB  ALA    64     -10.562  -6.927 -14.401  1.00 12.25           C  
ATOM    954  H   ALA    64     -12.097  -5.118 -13.233  1.00 13.78           H  
ATOM    955  HA  ALA    64     -10.708  -7.533 -12.396  1.00 11.92           H  
ATOM    956 1HB  ALA    64     -10.471  -6.053 -15.012  1.00 11.89           H  
ATOM    957 2HB  ALA    64      -9.695  -7.555 -14.636  1.00 12.00           H  
ATOM    958 3HB  ALA    64     -11.437  -7.494 -14.693  1.00 11.65           H  
ATOM    959  N   CYS    65      -8.424  -6.697 -11.837  1.00  9.61           N  
ATOM    960  CA  CYS    65      -7.088  -6.244 -11.420  1.00  9.87           C  
ATOM    961  C   CYS    65      -6.293  -5.941 -12.729  1.00 10.32           C  
ATOM    962  O   CYS    65      -6.661  -6.401 -13.822  1.00 10.77           O  
ATOM    963  CB  CYS    65      -6.338  -7.364 -10.692  1.00  9.94           C  
ATOM    964  SG  CYS    65      -7.259  -8.077  -9.293  1.00 11.68           S  
ATOM    965  H   CYS    65      -8.550  -7.703 -11.854  1.00 10.96           H  
ATOM    966  HA  CYS    65      -7.131  -5.392 -10.829  1.00  9.73           H  
ATOM    967 1HB  CYS    65      -6.162  -8.185 -11.368  1.00 10.36           H  
ATOM    968 2HB  CYS    65      -5.407  -7.022 -10.315  1.00 10.05           H  
ATOM    969  N   LYS    66      -5.175  -5.240 -12.577  1.00 10.38           N  
ATOM    970  CA  LYS    66      -4.333  -4.889 -13.703  1.00 13.03           C  
ATOM    971  C   LYS    66      -3.897  -6.142 -14.486  1.00 13.19           C  
ATOM    972  O   LYS    66      -3.791  -6.034 -15.701  1.00 14.17           O  
ATOM    973  CB  LYS    66      -3.026  -4.195 -13.378  1.00 18.59           C  
ATOM    974  CG  LYS    66      -3.217  -2.786 -12.863  1.00 24.50           C  
ATOM    975  CD  LYS    66      -1.793  -2.260 -12.396  1.00 28.16           C  
ATOM    976  CE  LYS    66      -1.844  -0.793 -12.092  1.00 30.89           C  
ATOM    977  NZ  LYS    66      -0.888  -0.421 -10.958  1.00 32.86           N  
ATOM    978  H   LYS    66      -4.912  -4.897 -11.691  1.00 10.72           H  
ATOM    979  HA  LYS    66      -4.872  -4.238 -14.368  1.00 13.01           H  
ATOM    980 1HB  LYS    66      -2.443  -4.769 -12.657  1.00 18.45           H  
ATOM    981 2HB  LYS    66      -2.436  -4.140 -14.320  1.00 18.47           H  
ATOM    982 1HG  LYS    66      -3.585  -2.117 -13.638  1.00 24.32           H  
ATOM    983 2HG  LYS    66      -3.852  -2.729 -12.000  1.00 24.19           H  
ATOM    984 1HD  LYS    66      -1.486  -2.841 -11.549  1.00 27.90           H  
ATOM    985 2HD  LYS    66      -1.110  -2.436 -13.237  1.00 28.04           H  
ATOM    986 1HE  LYS    66      -1.513  -0.203 -12.953  1.00 30.96           H  
ATOM    987 2HE  LYS    66      -2.816  -0.445 -11.785  1.00 31.03           H  
ATOM    988 1HZ  LYS    66      -1.275  -0.753 -10.062  1.00  0.00           H  
ATOM    989 2HZ  LYS    66       0.028  -0.861 -11.121  1.00  0.00           H  
ATOM    990  N   ASN    67      -3.665  -7.265 -13.771  1.00 12.34           N  
ATOM    991  CA  ASN    67      -3.221  -8.515 -14.408  1.00 13.31           C  
ATOM    992  C   ASN    67      -4.352  -9.386 -14.957  1.00 13.07           C  
ATOM    993  O   ASN    67      -4.115 -10.521 -15.488  1.00 14.97           O  
ATOM    994  CB  ASN    67      -2.362  -9.323 -13.431  0.67 14.22           C  
ATOM    995  CG  ASN    67      -3.172 -10.096 -12.381  0.67 14.84           C  
ATOM    996  OD1 ASN    67      -4.349  -9.841 -12.027  0.67 13.19           O  
ATOM    997  ND2 ASN    67      -2.554 -11.199 -11.893  0.67 15.90           N  
ATOM    998  H   ASN    67      -3.774  -7.252 -12.788  1.00 13.45           H  
ATOM    999  HA  ASN    67      -2.608  -8.259 -15.256  1.00 13.76           H  
ATOM   1000 1HB  ASN    67      -1.769 -10.051 -13.997  0.67 14.26           H  
ATOM   1001 2HB  ASN    67      -1.647  -8.697 -12.929  0.67 14.17           H  
ATOM   1002 1HD2 ASN    67      -1.602 -11.418 -12.209  0.67 16.83           H  
ATOM   1003 2HD2 ASN    67      -2.949 -11.805 -11.226  0.67 15.87           H  
ATOM   1004  N   GLY    68      -5.587  -8.903 -14.830  1.00 11.96           N  
ATOM   1005  CA  GLY    68      -6.723  -9.632 -15.350  1.00 11.56           C  
ATOM   1006  C   GLY    68      -7.513 -10.417 -14.355  1.00 11.36           C  
ATOM   1007  O   GLY    68      -8.656 -10.817 -14.657  1.00 12.73           O  
ATOM   1008  H   GLY    68      -5.759  -8.027 -14.414  1.00 12.43           H  
ATOM   1009 1HA  GLY    68      -7.385  -8.935 -15.851  1.00 11.72           H  
ATOM   1010 2HA  GLY    68      -6.353 -10.294 -16.164  1.00 11.54           H  
ATOM   1011  N   GLN    69      -6.966 -10.639 -13.173  1.00 11.34           N  
ATOM   1012  CA  GLN    69      -7.763 -11.401 -12.153  1.00 11.96           C  
ATOM   1013  C   GLN    69      -9.061 -10.630 -11.807  1.00 11.52           C  
ATOM   1014  O   GLN    69      -9.074  -9.403 -11.907  1.00 12.62           O  
ATOM   1015  CB  GLN    69      -6.973 -11.545 -10.855  1.00 13.20           C  
ATOM   1016  CG  GLN    69      -5.852 -12.494 -10.941  1.00 15.25           C  
ATOM   1017  CD  GLN    69      -5.011 -12.520  -9.671  1.00 18.36           C  
ATOM   1018  OE1 GLN    69      -3.823 -12.841  -9.858  1.00 20.07           O  
ATOM   1019  NE2 GLN    69      -5.464 -12.095  -8.454  1.00 17.86           N  
ATOM   1020  H   GLN    69      -6.073 -10.306 -12.926  1.00 11.56           H  
ATOM   1021  HA  GLN    69      -7.970 -12.364 -12.516  1.00 12.34           H  
ATOM   1022 1HB  GLN    69      -6.693 -10.547 -10.518  1.00 12.99           H  
ATOM   1023 2HB  GLN    69      -7.722 -11.930 -10.116  1.00 13.58           H  
ATOM   1024 1HG  GLN    69      -6.179 -13.521 -11.180  1.00 15.55           H  
ATOM   1025 2HG  GLN    69      -5.150 -12.261 -11.743  1.00 15.04           H  
ATOM   1026 1HE2 GLN    69      -6.419 -11.833  -8.422  1.00 17.73           H  
ATOM   1027 2HE2 GLN    69      -4.864 -12.112  -7.697  1.00 17.98           H  
ATOM   1028  N   THR    70     -10.067 -11.332 -11.376  1.00 10.48           N  
ATOM   1029  CA  THR    70     -11.336 -10.719 -11.086  1.00 11.30           C  
ATOM   1030  C   THR    70     -11.703 -10.600  -9.642  1.00 12.40           C  
ATOM   1031  O   THR    70     -12.873 -10.329  -9.313  1.00 14.53           O  
ATOM   1032  CB  THR    70     -12.376 -11.413 -11.980  1.00 12.92           C  
ATOM   1033  OG1 THR    70     -12.345 -12.775 -11.690  1.00 13.81           O  
ATOM   1034  CG2 THR    70     -12.219 -11.201 -13.466  1.00 14.02           C  
ATOM   1035  H   THR    70     -10.021 -12.342 -11.332  1.00 11.26           H  
ATOM   1036  HA  THR    70     -11.309  -9.701 -11.491  1.00 11.80           H  
ATOM   1037  HB  THR    70     -13.370 -10.990 -11.683  1.00 13.29           H  
ATOM   1038  HG1 THR    70     -12.714 -12.975 -10.793  1.00 12.55           H  
ATOM   1039 1HG2 THR    70     -11.256 -10.770 -13.735  1.00 13.65           H  
ATOM   1040 2HG2 THR    70     -12.360 -12.136 -13.989  1.00 13.56           H  
ATOM   1041 3HG2 THR    70     -12.978 -10.455 -13.819  1.00 13.64           H  
ATOM   1042  N   ASN    71     -10.729 -10.732  -8.751  1.00 11.58           N  
ATOM   1043  CA  ASN    71     -10.915 -10.564  -7.308  1.00 10.88           C  
ATOM   1044  C   ASN    71     -10.468  -9.180  -6.791  1.00 10.90           C  
ATOM   1045  O   ASN    71     -10.145  -9.101  -5.625  1.00 11.01           O  
ATOM   1046  CB  ASN    71     -10.330 -11.738  -6.517  1.00 12.00           C  
ATOM   1047  CG  ASN    71      -8.837 -11.820  -6.613  1.00 13.21           C  
ATOM   1048  OD1 ASN    71      -8.256 -11.578  -7.679  1.00 13.61           O  
ATOM   1049  ND2 ASN    71      -8.191 -12.126  -5.485  1.00 13.76           N  
ATOM   1050  H   ASN    71      -9.787 -10.914  -9.060  1.00 12.24           H  
ATOM   1051  HA  ASN    71     -11.958 -10.663  -7.076  1.00 10.90           H  
ATOM   1052 1HB  ASN    71     -10.643 -11.618  -5.475  1.00 11.93           H  
ATOM   1053 2HB  ASN    71     -10.759 -12.676  -6.851  1.00 11.84           H  
ATOM   1054 1HD2 ASN    71      -8.697 -12.313  -4.630  1.00 13.01           H  
ATOM   1055 2HD2 ASN    71      -7.169 -12.198  -5.492  1.00 14.38           H  
ATOM   1056  N   CYS    72     -10.490  -8.186  -7.682  1.00 10.23           N  
ATOM   1057  CA  CYS    72     -10.135  -6.848  -7.274  1.00  9.57           C  
ATOM   1058  C   CYS    72     -11.466  -6.043  -7.006  1.00 10.33           C  
ATOM   1059  O   CYS    72     -12.532  -6.395  -7.547  1.00 10.32           O  
ATOM   1060  CB  CYS    72      -9.284  -6.166  -8.242  1.00 10.08           C  
ATOM   1061  SG  CYS    72      -7.464  -6.535  -8.084  1.00 11.82           S  
ATOM   1062  H   CYS    72     -10.780  -8.330  -8.620  1.00 10.78           H  
ATOM   1063  HA  CYS    72      -9.662  -6.942  -6.347  1.00  9.63           H  
ATOM   1064 1HB  CYS    72      -9.492  -6.456  -9.259  1.00  9.83           H  
ATOM   1065 2HB  CYS    72      -9.312  -5.095  -8.207  1.00 10.12           H  
ATOM   1066  N   TYR    73     -11.253  -5.061  -6.156  1.00  9.78           N  
ATOM   1067  CA  TYR    73     -12.329  -4.173  -5.697  1.00 10.52           C  
ATOM   1068  C   TYR    73     -11.832  -2.749  -5.608  1.00 10.60           C  
ATOM   1069  O   TYR    73     -10.744  -2.519  -5.079  1.00 10.04           O  
ATOM   1070  CB  TYR    73     -12.925  -4.584  -4.337  1.00 11.63           C  
ATOM   1071  CG  TYR    73     -13.551  -5.952  -4.427  1.00 12.38           C  
ATOM   1072  CD1 TYR    73     -14.922  -6.080  -4.682  1.00 13.67           C  
ATOM   1073  CD2 TYR    73     -12.773  -7.091  -4.287  1.00 12.78           C  
ATOM   1074  CE1 TYR    73     -15.461  -7.373  -4.820  1.00 14.92           C  
ATOM   1075  CE2 TYR    73     -13.285  -8.369  -4.470  1.00 15.63           C  
ATOM   1076  CZ  TYR    73     -14.640  -8.474  -4.751  1.00 16.41           C  
ATOM   1077  OH  TYR    73     -15.125  -9.785  -4.853  1.00 20.81           O  
ATOM   1078  H   TYR    73     -10.362  -4.865  -5.765  1.00 10.21           H  
ATOM   1079  HA  TYR    73     -13.104  -4.301  -6.392  1.00 10.61           H  
ATOM   1080 1HB  TYR    73     -12.190  -4.521  -3.554  1.00 11.40           H  
ATOM   1081 2HB  TYR    73     -13.692  -3.835  -4.128  1.00 11.87           H  
ATOM   1082  HD1 TYR    73     -15.557  -5.226  -4.792  1.00 13.58           H  
ATOM   1083  HD2 TYR    73     -11.691  -6.993  -4.130  1.00 13.48           H  
ATOM   1084  HE1 TYR    73     -16.517  -7.496  -5.029  1.00 15.48           H  
ATOM   1085  HE2 TYR    73     -12.662  -9.251  -4.376  1.00 15.19           H  
ATOM   1086  HH  TYR    73     -14.611 -10.285  -5.531  1.00 19.75           H  
ATOM   1087  N   GLN    74     -12.646  -1.825  -6.078  1.00 10.93           N  
ATOM   1088  CA  GLN    74     -12.373  -0.356  -6.045  1.00 12.00           C  
ATOM   1089  C   GLN    74     -13.220   0.315  -4.972  1.00 11.86           C  
ATOM   1090  O   GLN    74     -14.444   0.077  -4.878  1.00 12.34           O  
ATOM   1091  CB  GLN    74     -12.647   0.308  -7.370  1.00 14.42           C  
ATOM   1092  CG  GLN    74     -12.323   1.783  -7.410  1.00 17.24           C  
ATOM   1093  CD  GLN    74     -12.707   2.340  -8.758  1.00 22.30           C  
ATOM   1094  OE1 GLN    74     -12.290   1.845  -9.771  1.00 25.92           O  
ATOM   1095  NE2 GLN    74     -13.576   3.301  -8.720  1.00 25.98           N  
ATOM   1096  H   GLN    74     -13.540  -2.058  -6.467  1.00 11.41           H  
ATOM   1097  HA  GLN    74     -11.344  -0.260  -5.846  1.00 12.36           H  
ATOM   1098 1HB  GLN    74     -12.059  -0.190  -8.169  1.00 14.35           H  
ATOM   1099 2HB  GLN    74     -13.701   0.140  -7.626  1.00 14.53           H  
ATOM   1100 1HG  GLN    74     -12.890   2.338  -6.691  1.00 17.41           H  
ATOM   1101 2HG  GLN    74     -11.268   1.988  -7.244  1.00 17.52           H  
ATOM   1102 1HE2 GLN    74     -13.953   3.685  -7.847  1.00 24.38           H  
ATOM   1103 2HE2 GLN    74     -13.931   3.734  -9.568  1.00 24.59           H  
ATOM   1104  N   SER    75     -12.549   1.162  -4.198  1.00 11.01           N  
ATOM   1105  CA  SER    75     -13.280   1.824  -3.122  1.00 11.51           C  
ATOM   1106  C   SER    75     -14.288   2.835  -3.752  1.00 12.02           C  
ATOM   1107  O   SER    75     -14.010   3.509  -4.738  1.00 12.38           O  
ATOM   1108  CB  SER    75     -12.315   2.554  -2.192  1.00 10.73           C  
ATOM   1109  OG  SER    75     -11.555   3.526  -2.907  1.00 11.01           O  
ATOM   1110  H   SER    75     -11.584   1.335  -4.298  1.00 11.33           H  
ATOM   1111  HA  SER    75     -13.761   1.088  -2.561  1.00 11.25           H  
ATOM   1112 1HB  SER    75     -12.818   2.961  -1.337  1.00 10.70           H  
ATOM   1113 2HB  SER    75     -11.598   1.807  -1.830  1.00 10.75           H  
ATOM   1114  HG  SER    75     -11.194   4.136  -2.258  1.00 10.98           H  
ATOM   1115  N   TYR    76     -15.418   2.918  -3.055  1.00 13.63           N  
ATOM   1116  CA  TYR    76     -16.440   3.916  -3.473  1.00 15.12           C  
ATOM   1117  C   TYR    76     -15.939   5.335  -3.216  1.00 16.36           C  
ATOM   1118  O   TYR    76     -16.285   6.208  -4.034  1.00 18.50           O  
ATOM   1119  CB  TYR    76     -17.773   3.630  -2.778  1.00 17.35           C  
ATOM   1120  CG  TYR    76     -18.496   2.367  -3.170  1.00 19.30           C  
ATOM   1121  CD1 TYR    76     -18.655   2.022  -4.504  1.00 21.62           C  
ATOM   1122  CD2 TYR    76     -19.105   1.558  -2.223  1.00 21.91           C  
ATOM   1123  CE1 TYR    76     -19.306   0.866  -4.895  1.00 22.93           C  
ATOM   1124  CE2 TYR    76     -19.795   0.397  -2.587  1.00 22.30           C  
ATOM   1125  CZ  TYR    76     -19.871   0.065  -3.925  1.00 23.41           C  
ATOM   1126  OH  TYR    76     -20.545  -1.087  -4.315  1.00 25.85           O  
ATOM   1127  H   TYR    76     -15.606   2.406  -2.231  1.00 13.28           H  
ATOM   1128  HA  TYR    76     -16.563   3.840  -4.530  1.00 14.99           H  
ATOM   1129 1HB  TYR    76     -17.593   3.629  -1.690  1.00 17.06           H  
ATOM   1130 2HB  TYR    76     -18.420   4.483  -2.984  1.00 17.57           H  
ATOM   1131  HD1 TYR    76     -18.202   2.644  -5.295  1.00 21.52           H  
ATOM   1132  HD2 TYR    76     -19.023   1.806  -1.160  1.00 21.39           H  
ATOM   1133  HE1 TYR    76     -19.388   0.618  -5.958  1.00 22.72           H  
ATOM   1134  HE2 TYR    76     -20.235  -0.259  -1.851  1.00 22.33           H  
ATOM   1135  HH  TYR    76     -21.330  -1.268  -3.756  1.00 25.66           H  
ATOM   1136  N   SER    77     -15.231   5.554  -2.168  1.00 16.15           N  
ATOM   1137  CA  SER    77     -14.688   6.847  -1.773  1.00 16.98           C  
ATOM   1138  C   SER    77     -13.172   6.944  -2.075  1.00 15.96           C  
ATOM   1139  O   SER    77     -12.507   5.956  -2.133  1.00 14.48           O  
ATOM   1140  CB  SER    77     -15.017   7.110  -0.314  0.67 20.72           C  
ATOM   1141  OG  SER    77     -16.456   7.347  -0.265  0.67 24.78           O  
ATOM   1142  H   SER    77     -14.997   4.791  -1.528  1.00 16.92           H  
ATOM   1143  HA  SER    77     -15.160   7.650  -2.274  1.00 17.56           H  
ATOM   1144 1HB  SER    77     -14.761   6.282   0.328  0.67 20.83           H  
ATOM   1145 2HB  SER    77     -14.539   8.013   0.027  0.67 20.78           H  
ATOM   1146  HG  SER    77     -16.575   8.269  -0.510  0.67 26.84           H  
ATOM   1147  N   THR    78     -12.742   8.176  -2.135  1.00 15.29           N  
ATOM   1148  CA  THR    78     -11.275   8.477  -2.261  1.00 15.36           C  
ATOM   1149  C   THR    78     -10.776   8.307  -0.803  1.00 14.46           C  
ATOM   1150  O   THR    78     -11.444   8.434   0.230  1.00 14.60           O  
ATOM   1151  CB  THR    78     -10.901   9.838  -2.819  1.00 17.44           C  
ATOM   1152  OG1 THR    78     -11.440  10.783  -1.934  1.00 18.98           O  
ATOM   1153  CG2 THR    78     -11.393  10.134  -4.217  1.00 18.49           C  
ATOM   1154  H   THR    78     -13.315   8.989  -2.001  1.00 16.03           H  
ATOM   1155  HA  THR    78     -10.831   7.721  -2.871  1.00 15.39           H  
ATOM   1156  HB  THR    78      -9.784   9.928  -2.795  1.00 17.62           H  
ATOM   1157  HG1 THR    78     -11.091  11.695  -2.126  1.00 18.96           H  
ATOM   1158 1HG2 THR    78     -12.456  10.067  -4.293  1.00 18.21           H  
ATOM   1159 2HG2 THR    78     -11.080  11.184  -4.482  1.00 18.38           H  
ATOM   1160 3HG2 THR    78     -10.916   9.496  -4.955  1.00 18.16           H  
ATOM   1161  N   MET    79      -9.468   7.971  -0.736  1.00 12.75           N  
ATOM   1162  CA  MET    79      -8.763   7.766   0.518  1.00 11.56           C  
ATOM   1163  C   MET    79      -7.477   8.548   0.546  1.00 10.26           C  
ATOM   1164  O   MET    79      -6.962   8.802  -0.543  1.00 11.76           O  
ATOM   1165  CB  MET    79      -8.444   6.236   0.695  1.00 10.37           C  
ATOM   1166  CG  MET    79      -9.761   5.495   0.910  1.00 12.31           C  
ATOM   1167  SD  MET    79      -9.421   3.785   1.379  1.00 15.22           S  
ATOM   1168  CE  MET    79     -11.035   3.288   1.868  1.00 17.50           C  
ATOM   1169  H   MET    79      -8.930   7.853  -1.581  1.00 13.29           H  
ATOM   1170  HA  MET    79      -9.368   7.985   1.358  1.00 11.44           H  
ATOM   1171 1HB  MET    79      -8.016   5.882  -0.235  1.00 10.46           H  
ATOM   1172 2HB  MET    79      -7.784   6.105   1.502  1.00 10.44           H  
ATOM   1173 1HG  MET    79     -10.333   5.805   1.783  1.00 11.85           H  
ATOM   1174 2HG  MET    79     -10.346   5.442   0.018  1.00 12.23           H  
ATOM   1175 1HE  MET    79     -11.729   3.751   1.160  1.00 16.99           H  
ATOM   1176 2HE  MET    79     -11.082   2.212   1.763  1.00 17.35           H  
ATOM   1177 3HE  MET    79     -11.200   3.650   2.872  1.00 16.91           H  
ATOM   1178  N   SER    80      -7.001   8.903   1.710  1.00  9.94           N  
ATOM   1179  CA  SER    80      -5.715   9.597   1.853  1.00  9.06           C  
ATOM   1180  C   SER    80      -4.588   8.522   1.693  1.00  9.47           C  
ATOM   1181  O   SER    80      -4.583   7.531   2.442  1.00  9.65           O  
ATOM   1182  CB  SER    80      -5.649  10.198   3.245  1.00  9.74           C  
ATOM   1183  OG  SER    80      -4.365  10.767   3.335  1.00 12.09           O  
ATOM   1184  H   SER    80      -7.455   8.631   2.564  1.00 10.58           H  
ATOM   1185  HA  SER    80      -5.584  10.343   1.132  1.00  9.21           H  
ATOM   1186 1HB  SER    80      -6.446  10.941   3.356  1.00  9.78           H  
ATOM   1187 2HB  SER    80      -5.768   9.457   4.012  1.00  9.38           H  
ATOM   1188  HG  SER    80      -4.100  10.676   4.259  1.00 11.29           H  
ATOM   1189  N   ILE    81      -3.776   8.765   0.714  1.00  9.09           N  
ATOM   1190  CA  ILE    81      -2.660   7.812   0.425  1.00  9.30           C  
ATOM   1191  C   ILE    81      -1.347   8.587   0.271  1.00  9.49           C  
ATOM   1192  O   ILE    81      -1.260   9.793  -0.025  1.00 10.12           O  
ATOM   1193  CB  ILE    81      -2.903   6.967  -0.806  1.00 10.30           C  
ATOM   1194  CG1 ILE    81      -2.974   7.779  -2.078  1.00 11.60           C  
ATOM   1195  CG2 ILE    81      -4.150   6.049  -0.635  1.00 10.84           C  
ATOM   1196  CD1 ILE    81      -2.929   6.914  -3.349  1.00 14.25           C  
ATOM   1197  H   ILE    81      -3.824   9.556   0.123  1.00  9.54           H  
ATOM   1198  HA  ILE    81      -2.606   7.159   1.266  1.00  9.17           H  
ATOM   1199  HB  ILE    81      -2.058   6.266  -0.868  1.00 10.51           H  
ATOM   1200 1HG1 ILE    81      -3.936   8.301  -2.095  1.00 11.78           H  
ATOM   1201 2HG1 ILE    81      -2.176   8.478  -2.185  1.00 11.48           H  
ATOM   1202 1HG2 ILE    81      -4.105   5.590   0.342  1.00 10.69           H  
ATOM   1203 2HG2 ILE    81      -5.027   6.689  -0.729  1.00 10.79           H  
ATOM   1204 3HG2 ILE    81      -4.141   5.292  -1.404  1.00 10.44           H  
ATOM   1205 1HD1 ILE    81      -2.150   6.145  -3.243  1.00 14.07           H  
ATOM   1206 2HD1 ILE    81      -3.919   6.461  -3.468  1.00 13.69           H  
ATOM   1207 3HD1 ILE    81      -2.716   7.585  -4.178  1.00 14.10           H  
ATOM   1208  N   THR    82      -0.251   7.823   0.457  1.00  8.05           N  
ATOM   1209  CA  THR    82       1.123   8.328   0.231  1.00  8.47           C  
ATOM   1210  C   THR    82       1.773   7.372  -0.823  1.00  9.42           C  
ATOM   1211  O   THR    82       1.798   6.158  -0.552  1.00 10.11           O  
ATOM   1212  CB  THR    82       1.994   8.426   1.444  1.00  8.38           C  
ATOM   1213  OG1 THR    82       1.389   9.360   2.380  1.00 10.61           O  
ATOM   1214  CG2 THR    82       3.373   8.935   1.155  1.00  9.78           C  
ATOM   1215  H   THR    82      -0.333   6.841   0.642  1.00  8.86           H  
ATOM   1216  HA  THR    82       1.059   9.301  -0.209  1.00  8.66           H  
ATOM   1217  HB  THR    82       2.047   7.451   1.967  1.00  8.47           H  
ATOM   1218  HG1 THR    82       1.329  10.257   2.028  1.00 10.70           H  
ATOM   1219 1HG2 THR    82       3.332   9.853   0.560  1.00  9.67           H  
ATOM   1220 2HG2 THR    82       3.912   9.207   2.070  1.00  9.39           H  
ATOM   1221 3HG2 THR    82       3.939   8.192   0.609  1.00  9.23           H  
ATOM   1222  N   ASP    83       2.269   7.930  -1.867  1.00  9.16           N  
ATOM   1223  CA  ASP    83       2.993   7.174  -2.870  1.00 10.63           C  
ATOM   1224  C   ASP    83       4.491   7.345  -2.616  1.00  9.53           C  
ATOM   1225  O   ASP    83       4.942   8.471  -2.389  1.00  9.99           O  
ATOM   1226  CB  ASP    83       2.657   7.690  -4.280  0.50 13.74           C  
ATOM   1227  CG  ASP    83       1.838   6.655  -5.003  0.50 16.63           C  
ATOM   1228  OD1 ASP    83       0.851   6.110  -4.592  0.50 17.78           O  
ATOM   1229  OD2 ASP    83       2.396   6.360  -6.148  0.50 18.60           O  
ATOM   1230  H   ASP    83       2.253   8.933  -1.988  1.00 10.11           H  
ATOM   1231  HA  ASP    83       2.740   6.149  -2.863  1.00 10.78           H  
ATOM   1232 1HB  ASP    83       2.083   8.591  -4.200  0.50 13.65           H  
ATOM   1233 2HB  ASP    83       3.530   7.943  -4.852  0.50 13.71           H  
ATOM   1234  N   CYS    84       5.230   6.232  -2.609  1.00  9.13           N  
ATOM   1235  CA  CYS    84       6.633   6.183  -2.421  1.00  9.64           C  
ATOM   1236  C   CYS    84       7.286   5.652  -3.692  1.00 10.91           C  
ATOM   1237  O   CYS    84       6.915   4.545  -4.070  1.00 11.86           O  
ATOM   1238  CB  CYS    84       6.984   5.262  -1.241  1.00  9.17           C  
ATOM   1239  SG  CYS    84       6.312   5.829   0.347  1.00  9.17           S  
ATOM   1240  H   CYS    84       4.777   5.343  -2.802  1.00  9.94           H  
ATOM   1241  HA  CYS    84       7.064   7.113  -2.177  1.00  9.79           H  
ATOM   1242 1HB  CYS    84       6.517   4.303  -1.414  1.00  8.89           H  
ATOM   1243 2HB  CYS    84       8.024   5.168  -1.146  1.00  9.14           H  
ATOM   1244  N   ARG    85       8.195   6.413  -4.222  1.00 11.26           N  
ATOM   1245  CA  ARG    85       8.896   5.984  -5.485  1.00 12.74           C  
ATOM   1246  C   ARG    85      10.401   6.171  -5.305  1.00 11.60           C  
ATOM   1247  O   ARG    85      10.846   7.194  -4.826  1.00 10.89           O  
ATOM   1248  CB  ARG    85       8.516   6.829  -6.673  0.50 16.60           C  
ATOM   1249  CG  ARG    85       8.519   6.146  -8.029  0.50 20.26           C  
ATOM   1250  CD  ARG    85       7.151   5.639  -8.372  0.50 23.67           C  
ATOM   1251  NE  ARG    85       6.189   6.731  -8.397  0.50 26.55           N  
ATOM   1252  CZ  ARG    85       4.872   6.574  -8.469  0.50 27.79           C  
ATOM   1253  NH1 ARG    85       4.277   5.389  -8.567  0.50 27.49           N  
ATOM   1254  NH2 ARG    85       4.115   7.683  -8.433  0.50 28.91           N  
ATOM   1255  H   ARG    85       8.449   7.303  -3.856  1.00 11.09           H  
ATOM   1256  HA  ARG    85       8.674   4.964  -5.645  1.00 13.46           H  
ATOM   1257 1HB  ARG    85       7.569   7.289  -6.419  0.50 16.75           H  
ATOM   1258 2HB  ARG    85       9.165   7.717  -6.703  0.50 16.69           H  
ATOM   1259 1HG  ARG    85       8.857   6.763  -8.800  0.50 20.24           H  
ATOM   1260 2HG  ARG    85       9.187   5.304  -8.034  0.50 20.18           H  
ATOM   1261 1HD  ARG    85       7.149   5.077  -9.260  0.50 23.55           H  
ATOM   1262 2HD  ARG    85       6.880   4.946  -7.607  0.50 23.53           H  
ATOM   1263 1HH1 ARG    85       4.796   4.533  -8.608  0.50 27.53           H  
ATOM   1264 2HH1 ARG    85       3.261   5.347  -8.616  0.50 27.49           H  
ATOM   1265 1HH2 ARG    85       4.502   8.603  -8.365  0.50 29.02           H  
ATOM   1266 2HH2 ARG    85       3.105   7.578  -8.486  0.50 28.48           H  
ATOM   1267  N   GLU    86      11.134   5.108  -5.676  1.00 10.69           N  
ATOM   1268  CA  GLU    86      12.579   5.142  -5.554  1.00 10.91           C  
ATOM   1269  C   GLU    86      13.180   6.274  -6.418  1.00 11.78           C  
ATOM   1270  O   GLU    86      12.731   6.545  -7.532  1.00 12.98           O  
ATOM   1271  CB  GLU    86      13.083   3.771  -5.973  1.00 13.94           C  
ATOM   1272  CG  GLU    86      14.405   3.452  -5.521  1.00 15.98           C  
ATOM   1273  CD  GLU    86      15.034   2.082  -5.904  1.00 13.92           C  
ATOM   1274  OE1 GLU    86      15.996   1.812  -5.316  1.00 13.74           O  
ATOM   1275  OE2 GLU    86      14.395   1.473  -6.782  1.00 16.08           O  
ATOM   1276  H   GLU    86      10.707   4.294  -6.064  1.00 11.28           H  
ATOM   1277  HA  GLU    86      12.820   5.321  -4.545  1.00 11.25           H  
ATOM   1278 1HB  GLU    86      12.381   3.015  -5.598  1.00 13.58           H  
ATOM   1279 2HB  GLU    86      12.995   3.713  -7.075  1.00 13.90           H  
ATOM   1280 1HG  GLU    86      15.103   4.162  -6.009  1.00 15.34           H  
ATOM   1281 2HG  GLU    86      14.607   3.556  -4.461  1.00 15.45           H  
ATOM   1282  N   THR    87      14.119   6.968  -5.816  1.00 11.34           N  
ATOM   1283  CA  THR    87      14.824   8.116  -6.447  1.00 11.89           C  
ATOM   1284  C   THR    87      16.059   7.579  -7.170  1.00 11.66           C  
ATOM   1285  O   THR    87      16.546   6.477  -6.917  1.00 11.50           O  
ATOM   1286  CB  THR    87      15.223   9.217  -5.472  1.00 13.80           C  
ATOM   1287  OG1 THR    87      16.337   8.773  -4.743  1.00 15.31           O  
ATOM   1288  CG2 THR    87      14.092   9.637  -4.586  1.00 15.47           C  
ATOM   1289  H   THR    87      14.457   6.733  -4.895  1.00 11.48           H  
ATOM   1290  HA  THR    87      14.156   8.553  -7.158  1.00 12.17           H  
ATOM   1291  HB  THR    87      15.537  10.112  -6.079  1.00 14.28           H  
ATOM   1292  HG1 THR    87      16.626   9.366  -4.018  1.00 16.46           H  
ATOM   1293 1HG2 THR    87      13.453   8.839  -4.252  1.00 15.47           H  
ATOM   1294 2HG2 THR    87      14.462  10.186  -3.685  1.00 15.10           H  
ATOM   1295 3HG2 THR    87      13.463  10.350  -5.154  1.00 15.32           H  
ATOM   1296  N   GLY    88      16.597   8.497  -8.020  1.00 11.38           N  
ATOM   1297  CA  GLY    88      17.785   8.166  -8.802  1.00 11.93           C  
ATOM   1298  C   GLY    88      18.965   7.904  -7.948  1.00 12.66           C  
ATOM   1299  O   GLY    88      19.898   7.200  -8.423  1.00 14.60           O  
ATOM   1300  H   GLY    88      16.187   9.374  -8.182  1.00 11.69           H  
ATOM   1301 1HA  GLY    88      17.573   7.325  -9.426  1.00 11.84           H  
ATOM   1302 2HA  GLY    88      17.965   9.026  -9.482  1.00 11.82           H  
ATOM   1303  N   SER    89      19.044   8.428  -6.724  1.00 11.20           N  
ATOM   1304  CA  SER    89      20.115   8.247  -5.810  1.00 13.18           C  
ATOM   1305  C   SER    89      20.018   7.007  -4.880  1.00 13.30           C  
ATOM   1306  O   SER    89      20.923   6.728  -4.105  1.00 15.20           O  
ATOM   1307  CB  SER    89      20.333   9.460  -4.953  1.00 13.86           C  
ATOM   1308  OG  SER    89      20.849  10.451  -5.832  1.00 17.59           O  
ATOM   1309  H   SER    89      18.258   8.991  -6.402  1.00 12.27           H  
ATOM   1310  HA  SER    89      21.040   8.147  -6.373  1.00 13.29           H  
ATOM   1311 1HB  SER    89      19.410   9.811  -4.481  1.00 14.06           H  
ATOM   1312 2HB  SER    89      21.068   9.258  -4.193  1.00 14.58           H  
ATOM   1313  HG  SER    89      21.777  10.258  -6.003  1.00 17.03           H  
ATOM   1314  N   SER    90      18.949   6.307  -5.000  1.00 13.03           N  
ATOM   1315  CA  SER    90      18.696   5.086  -4.164  1.00 12.61           C  
ATOM   1316  C   SER    90      19.769   4.019  -4.507  1.00 13.43           C  
ATOM   1317  O   SER    90      19.904   3.744  -5.697  1.00 15.28           O  
ATOM   1318  CB  SER    90      17.348   4.632  -4.488  1.00 10.62           C  
ATOM   1319  OG  SER    90      16.989   3.575  -3.581  1.00 11.81           O  
ATOM   1320  H   SER    90      18.195   6.523  -5.635  1.00 13.39           H  
ATOM   1321  HA  SER    90      18.767   5.330  -3.144  1.00 12.25           H  
ATOM   1322 1HB  SER    90      16.573   5.398  -4.424  1.00 10.63           H  
ATOM   1323 2HB  SER    90      17.312   4.196  -5.488  1.00 10.82           H  
ATOM   1324  HG  SER    90      17.344   2.763  -3.906  1.00 11.86           H  
ATOM   1325  N   LYS    91      20.356   3.506  -3.471  1.00 13.82           N  
ATOM   1326  CA  LYS    91      21.370   2.457  -3.659  1.00 15.19           C  
ATOM   1327  C   LYS    91      21.441   1.567  -2.406  1.00 12.54           C  
ATOM   1328  O   LYS    91      21.820   2.007  -1.366  1.00 13.58           O  
ATOM   1329  CB  LYS    91      22.713   3.104  -3.923  0.50 18.45           C  
ATOM   1330  CG  LYS    91      23.904   2.133  -4.024  0.50 21.26           C  
ATOM   1331  CD  LYS    91      25.035   2.707  -4.855  0.50 23.60           C  
ATOM   1332  CE  LYS    91      26.178   1.727  -5.039  0.50 25.10           C  
ATOM   1333  NZ  LYS    91      26.958   1.586  -3.794  0.50 26.21           N  
ATOM   1334  H   LYS    91      20.170   3.774  -2.527  1.00 13.78           H  
ATOM   1335  HA  LYS    91      21.058   1.851  -4.487  1.00 15.35           H  
ATOM   1336 1HB  LYS    91      22.646   3.693  -4.841  0.50 18.56           H  
ATOM   1337 2HB  LYS    91      22.908   3.793  -3.084  0.50 18.36           H  
ATOM   1338 1HG  LYS    91      24.260   1.868  -3.024  0.50 21.14           H  
ATOM   1339 2HG  LYS    91      23.587   1.197  -4.487  0.50 21.39           H  
ATOM   1340 1HD  LYS    91      24.654   2.939  -5.860  0.50 23.60           H  
ATOM   1341 2HD  LYS    91      25.396   3.634  -4.413  0.50 23.54           H  
ATOM   1342 1HE  LYS    91      25.806   0.743  -5.316  0.50 25.23           H  
ATOM   1343 2HE  LYS    91      26.866   2.082  -5.810  0.50 25.15           H  
ATOM   1344 1HZ  LYS    91      26.381   1.125  -3.076  1.00  0.00           H  
ATOM   1345 2HZ  LYS    91      27.797   1.017  -3.976  1.00  0.00           H  
ATOM   1346  N   TYR    92      21.006   0.361  -2.627  1.00 12.58           N  
ATOM   1347  CA  TYR    92      20.945  -0.630  -1.511  1.00 13.10           C  
ATOM   1348  C   TYR    92      22.293  -0.700  -0.761  1.00 13.53           C  
ATOM   1349  O   TYR    92      23.275  -0.796  -1.477  1.00 14.78           O  
ATOM   1350  CB  TYR    92      20.625  -2.012  -2.105  1.00 12.08           C  
ATOM   1351  CG  TYR    92      20.425  -3.016  -1.024  1.00 11.51           C  
ATOM   1352  CD1 TYR    92      19.184  -3.166  -0.362  1.00 11.11           C  
ATOM   1353  CD2 TYR    92      21.504  -3.816  -0.596  1.00 12.74           C  
ATOM   1354  CE1 TYR    92      19.039  -4.088   0.646  1.00 11.59           C  
ATOM   1355  CE2 TYR    92      21.348  -4.720   0.411  1.00 12.10           C  
ATOM   1356  CZ  TYR    92      20.154  -4.872   1.031  1.00 11.89           C  
ATOM   1357  OH  TYR    92      20.031  -5.812   2.005  1.00 13.79           O  
ATOM   1358  H   TYR    92      20.657   0.017  -3.504  1.00 13.58           H  
ATOM   1359  HA  TYR    92      20.146  -0.377  -0.885  1.00 12.99           H  
ATOM   1360 1HB  TYR    92      19.746  -1.933  -2.747  1.00 12.07           H  
ATOM   1361 2HB  TYR    92      21.465  -2.272  -2.740  1.00 11.87           H  
ATOM   1362  HD1 TYR    92      18.348  -2.559  -0.671  1.00 11.29           H  
ATOM   1363  HD2 TYR    92      22.471  -3.690  -1.092  1.00 12.33           H  
ATOM   1364  HE1 TYR    92      18.087  -4.218   1.126  1.00 12.00           H  
ATOM   1365  HE2 TYR    92      22.199  -5.355   0.719  1.00 12.36           H  
ATOM   1366  HH  TYR    92      20.820  -5.924   2.572  1.00 13.43           H  
ATOM   1367  N   PRO    93      22.259  -0.708   0.561  1.00 14.62           N  
ATOM   1368  CA  PRO    93      21.164  -0.719   1.467  1.00 14.65           C  
ATOM   1369  C   PRO    93      20.549   0.629   1.781  1.00 14.62           C  
ATOM   1370  O   PRO    93      19.596   0.659   2.558  1.00 15.63           O  
ATOM   1371  CB  PRO    93      21.703  -1.480   2.723  1.00 14.57           C  
ATOM   1372  CG  PRO    93      23.116  -1.018   2.793  1.00 15.14           C  
ATOM   1373  CD  PRO    93      23.569  -0.867   1.355  1.00 14.76           C  
ATOM   1374  HA  PRO    93      20.257  -1.124   1.166  1.00 14.63           H  
ATOM   1375 1HB  PRO    93      21.195  -1.321   3.551  1.00 14.35           H  
ATOM   1376 2HB  PRO    93      21.585  -2.519   2.575  1.00 14.41           H  
ATOM   1377 1HG  PRO    93      23.183  -0.211   3.235  1.00 14.86           H  
ATOM   1378 2HG  PRO    93      23.778  -1.720   3.292  1.00 14.96           H  
ATOM   1379 1HD  PRO    93      24.165  -0.005   1.203  1.00 14.90           H  
ATOM   1380 2HD  PRO    93      24.101  -1.710   0.957  1.00 14.76           H  
ATOM   1381  N   ASN    94      21.091   1.643   1.139  1.00 15.58           N  
ATOM   1382  CA  ASN    94      20.567   3.031   1.392  1.00 17.10           C  
ATOM   1383  C   ASN    94      19.492   3.417   0.365  1.00 14.72           C  
ATOM   1384  O   ASN    94      19.701   4.200  -0.533  1.00 14.76           O  
ATOM   1385  CB  ASN    94      21.760   4.000   1.454  1.00 22.03           C  
ATOM   1386  CG  ASN    94      22.743   3.674   2.600  1.00 25.63           C  
ATOM   1387  OD1 ASN    94      22.364   3.510   3.776  1.00 28.37           O  
ATOM   1388  ND2 ASN    94      24.003   3.506   2.229  1.00 27.65           N  
ATOM   1389  H   ASN    94      21.851   1.594   0.521  1.00 14.86           H  
ATOM   1390  HA  ASN    94      20.182   3.061   2.376  1.00 17.26           H  
ATOM   1391 1HB  ASN    94      22.270   4.001   0.503  1.00 21.85           H  
ATOM   1392 2HB  ASN    94      21.397   5.010   1.625  1.00 22.14           H  
ATOM   1393 1HD2 ASN    94      24.295   3.621   1.265  1.00 26.86           H  
ATOM   1394 2HD2 ASN    94      24.717   3.272   2.934  1.00 26.95           H  
ATOM   1395  N   CYS    95      18.346   2.765   0.578  1.00 13.50           N  
ATOM   1396  CA  CYS    95      17.202   2.955  -0.329  1.00 12.35           C  
ATOM   1397  C   CYS    95      16.622   4.340  -0.064  1.00 12.71           C  
ATOM   1398  O   CYS    95      16.480   4.709   1.113  1.00 14.20           O  
ATOM   1399  CB  CYS    95      16.174   1.865  -0.090  1.00 10.83           C  
ATOM   1400  SG  CYS    95      16.823   0.184  -0.292  1.00 10.89           S  
ATOM   1401  H   CYS    95      18.201   2.112   1.313  1.00 14.07           H  
ATOM   1402  HA  CYS    95      17.547   2.842  -1.316  1.00 12.22           H  
ATOM   1403 1HB  CYS    95      15.794   1.928   0.910  1.00 10.75           H  
ATOM   1404 2HB  CYS    95      15.373   1.948  -0.798  1.00 10.38           H  
ATOM   1405  N   ALA    96      16.317   5.014  -1.140  1.00 11.08           N  
ATOM   1406  CA  ALA    96      15.818   6.398  -1.051  1.00 11.55           C  
ATOM   1407  C   ALA    96      14.590   6.528  -1.942  1.00 10.02           C  
ATOM   1408  O   ALA    96      14.521   6.022  -3.015  1.00  9.94           O  
ATOM   1409  CB  ALA    96      16.908   7.382  -1.496  1.00 12.72           C  
ATOM   1410  H   ALA    96      16.440   4.672  -2.029  1.00 11.72           H  
ATOM   1411  HA  ALA    96      15.569   6.629  -0.047  1.00 11.31           H  
ATOM   1412 1HB  ALA    96      17.564   6.931  -2.211  1.00 12.13           H  
ATOM   1413 2HB  ALA    96      16.452   8.267  -1.918  1.00 12.13           H  
ATOM   1414 3HB  ALA    96      17.484   7.682  -0.630  1.00 12.35           H  
ATOM   1415  N   TYR    97      13.626   7.262  -1.385  1.00  9.85           N  
ATOM   1416  CA  TYR    97      12.342   7.478  -1.984  1.00  9.55           C  
ATOM   1417  C   TYR    97      11.813   8.928  -2.014  1.00  8.98           C  
ATOM   1418  O   TYR    97      12.139   9.692  -1.065  1.00 10.31           O  
ATOM   1419  CB  TYR    97      11.212   6.691  -1.096  1.00  8.88           C  
ATOM   1420  CG  TYR    97      11.529   5.175  -1.140  1.00  8.11           C  
ATOM   1421  CD1 TYR    97      11.028   4.417  -2.159  1.00  8.27           C  
ATOM   1422  CD2 TYR    97      12.378   4.613  -0.225  1.00  9.08           C  
ATOM   1423  CE1 TYR    97      11.366   3.076  -2.299  1.00  9.55           C  
ATOM   1424  CE2 TYR    97      12.710   3.251  -0.322  1.00  8.97           C  
ATOM   1425  CZ  TYR    97      12.227   2.515  -1.369  1.00  9.14           C  
ATOM   1426  OH  TYR    97      12.560   1.190  -1.435  1.00 11.47           O  
ATOM   1427  H   TYR    97      13.731   7.646  -0.445  1.00 10.49           H  
ATOM   1428  HA  TYR    97      12.275   7.014  -2.915  1.00  9.39           H  
ATOM   1429 1HB  TYR    97      11.280   7.108  -0.119  1.00  8.16           H  
ATOM   1430 2HB  TYR    97      10.293   6.914  -1.554  1.00  8.17           H  
ATOM   1431  HD1 TYR    97      10.370   4.854  -2.930  1.00  8.40           H  
ATOM   1432  HD2 TYR    97      12.803   5.205   0.587  1.00  8.72           H  
ATOM   1433  HE1 TYR    97      10.938   2.510  -3.122  1.00  8.90           H  
ATOM   1434  HE2 TYR    97      13.388   2.803   0.408  1.00  9.17           H  
ATOM   1435  HH  TYR    97      12.787   0.915  -2.339  1.00 11.20           H  
ATOM   1436  N   LYS    98      11.046   9.167  -3.016  1.00  9.50           N  
ATOM   1437  CA  LYS    98      10.294  10.435  -3.093  1.00 11.02           C  
ATOM   1438  C   LYS    98       8.859  10.116  -2.492  1.00 10.66           C  
ATOM   1439  O   LYS    98       8.273   9.119  -2.889  1.00 10.94           O  
ATOM   1440  CB  LYS    98      10.105  10.813  -4.542  0.50 13.92           C  
ATOM   1441  CG  LYS    98       9.287  12.078  -4.841  0.50 16.49           C  
ATOM   1442  CD  LYS    98      10.095  13.302  -4.522  0.50 18.93           C  
ATOM   1443  CE  LYS    98       9.359  14.632  -4.467  0.50 21.31           C  
ATOM   1444  NZ  LYS    98      10.312  15.634  -3.866  0.50 21.98           N  
ATOM   1445  H   LYS    98      10.757   8.499  -3.698  1.00  9.26           H  
ATOM   1446  HA  LYS    98      10.728  11.176  -2.512  1.00 11.28           H  
ATOM   1447 1HB  LYS    98      11.072  10.899  -5.045  0.50 13.93           H  
ATOM   1448 2HB  LYS    98       9.568   9.962  -5.020  0.50 13.88           H  
ATOM   1449 1HG  LYS    98       9.005  12.076  -5.915  0.50 16.58           H  
ATOM   1450 2HG  LYS    98       8.353  12.080  -4.296  0.50 16.46           H  
ATOM   1451 1HD  LYS    98      10.600  13.212  -3.553  0.50 19.09           H  
ATOM   1452 2HD  LYS    98      10.904  13.404  -5.270  0.50 19.08           H  
ATOM   1453 1HE  LYS    98       9.087  14.990  -5.443  0.50 21.22           H  
ATOM   1454 2HE  LYS    98       8.506  14.585  -3.807  0.50 21.13           H  
ATOM   1455 1HZ  LYS    98      10.216  15.629  -2.840  1.00  0.00           H  
ATOM   1456 2HZ  LYS    98      11.280  15.384  -4.120  1.00  0.00           H  
ATOM   1457  N   THR    99       8.431  11.029  -1.642  1.00  9.52           N  
ATOM   1458  CA  THR    99       7.131  10.962  -0.999  1.00 10.44           C  
ATOM   1459  C   THR    99       6.126  11.904  -1.741  1.00 11.59           C  
ATOM   1460  O   THR    99       6.430  13.128  -1.869  1.00 12.28           O  
ATOM   1461  CB  THR    99       7.313  11.478   0.456  1.00 10.12           C  
ATOM   1462  OG1 THR    99       8.159  10.596   1.141  1.00  9.69           O  
ATOM   1463  CG2 THR    99       5.998  11.534   1.224  1.00 10.59           C  
ATOM   1464  H   THR    99       8.982  11.822  -1.387  1.00 10.84           H  
ATOM   1465  HA  THR    99       6.752   9.991  -0.969  1.00 10.30           H  
ATOM   1466  HB  THR    99       7.761  12.489   0.414  1.00  9.85           H  
ATOM   1467  HG1 THR    99       9.103  10.716   0.897  1.00 10.15           H  
ATOM   1468 1HG2 THR    99       5.237  12.096   0.760  1.00 10.18           H  
ATOM   1469 2HG2 THR    99       5.672  10.490   1.402  1.00 10.45           H  
ATOM   1470 3HG2 THR    99       6.216  11.957   2.219  1.00  9.90           H  
ATOM   1471  N   THR   100       4.993  11.402  -2.104  1.00 11.22           N  
ATOM   1472  CA  THR   100       3.935  12.159  -2.771  1.00 12.73           C  
ATOM   1473  C   THR   100       2.624  11.879  -2.053  1.00 13.32           C  
ATOM   1474  O   THR   100       2.238  10.713  -1.904  1.00 13.57           O  
ATOM   1475  CB  THR   100       3.772  11.747  -4.266  1.00 14.54           C  
ATOM   1476  OG1 THR   100       4.973  11.944  -4.909  1.00 16.74           O  
ATOM   1477  CG2 THR   100       2.714  12.690  -4.933  1.00 16.23           C  
ATOM   1478  H   THR   100       4.798  10.415  -1.948  1.00 11.42           H  
ATOM   1479  HA  THR   100       4.161  13.192  -2.758  1.00 12.71           H  
ATOM   1480  HB  THR   100       3.440  10.713  -4.316  1.00 14.25           H  
ATOM   1481  HG1 THR   100       5.439  12.792  -4.749  1.00 16.22           H  
ATOM   1482 1HG2 THR   100       2.914  13.704  -4.699  1.00 15.63           H  
ATOM   1483 2HG2 THR   100       2.755  12.514  -6.029  1.00 16.06           H  
ATOM   1484 3HG2 THR   100       1.732  12.377  -4.599  1.00 15.82           H  
ATOM   1485  N   GLN   101       1.941  12.923  -1.673  1.00 14.66           N  
ATOM   1486  CA  GLN   101       0.602  12.798  -1.013  1.00 16.33           C  
ATOM   1487  C   GLN   101      -0.484  12.823  -2.078  1.00 15.58           C  
ATOM   1488  O   GLN   101      -0.380  13.615  -3.076  1.00 16.36           O  
ATOM   1489  CB  GLN   101       0.272  14.002  -0.135  1.00 22.15           C  
ATOM   1490  CG  GLN   101       1.166  14.507   0.930  1.00 27.06           C  
ATOM   1491  CD  GLN   101       0.633  15.808   1.574  1.00 31.07           C  
ATOM   1492  OE1 GLN   101      -0.053  16.691   0.985  1.00 33.12           O  
ATOM   1493  NE2 GLN   101       0.961  15.915   2.857  1.00 32.14           N  
ATOM   1494  H   GLN   101       2.257  13.862  -1.830  1.00 14.58           H  
ATOM   1495  HA  GLN   101       0.566  11.920  -0.433  1.00 16.82           H  
ATOM   1496 1HB  GLN   101       0.072  14.847  -0.849  1.00 21.95           H  
ATOM   1497 2HB  GLN   101      -0.736  13.794   0.320  1.00 21.97           H  
ATOM   1498 1HG  GLN   101       1.378  13.823   1.745  1.00 27.07           H  
ATOM   1499 2HG  GLN   101       2.121  14.806   0.505  1.00 27.23           H  
ATOM   1500 1HE2 GLN   101       1.509  15.218   3.326  1.00 29.73           H  
ATOM   1501 2HE2 GLN   101       0.683  16.714   3.395  1.00 31.66           H  
ATOM   1502  N   ALA   102      -1.541  12.048  -1.886  1.00 14.28           N  
ATOM   1503  CA  ALA   102      -2.652  12.066  -2.857  1.00 14.37           C  
ATOM   1504  C   ALA   102      -3.906  11.641  -2.128  1.00 13.40           C  
ATOM   1505  O   ALA   102      -3.913  11.119  -1.039  1.00 12.68           O  
ATOM   1506  CB  ALA   102      -2.284  11.161  -4.017  1.00 14.33           C  
ATOM   1507  H   ALA   102      -1.564  11.465  -1.083  1.00 14.34           H  
ATOM   1508  HA  ALA   102      -2.796  13.049  -3.247  1.00 14.16           H  
ATOM   1509 1HB  ALA   102      -1.310  11.375  -4.427  1.00 14.22           H  
ATOM   1510 2HB  ALA   102      -2.278  10.123  -3.707  1.00 14.40           H  
ATOM   1511 3HB  ALA   102      -3.019  11.253  -4.815  1.00 14.07           H  
ATOM   1512  N   ASN   103      -5.011  11.864  -2.825  1.00 14.90           N  
ATOM   1513  CA  ASN   103      -6.374  11.447  -2.326  1.00 16.06           C  
ATOM   1514  C   ASN   103      -7.003  10.736  -3.527  1.00 14.92           C  
ATOM   1515  O   ASN   103      -7.384  11.445  -4.457  1.00 15.99           O  
ATOM   1516  CB  ASN   103      -7.250  12.562  -1.784  1.00 21.59           C  
ATOM   1517  CG  ASN   103      -6.912  13.061  -0.397  1.00 26.08           C  
ATOM   1518  OD1 ASN   103      -6.020  13.918  -0.185  1.00 30.72           O  
ATOM   1519  ND2 ASN   103      -7.622  12.557   0.615  1.00 28.40           N  
ATOM   1520  H   ASN   103      -5.022  12.257  -3.746  1.00 14.56           H  
ATOM   1521  HA  ASN   103      -6.210  10.826  -1.517  1.00 16.71           H  
ATOM   1522 1HB  ASN   103      -7.227  13.390  -2.497  1.00 21.55           H  
ATOM   1523 2HB  ASN   103      -8.299  12.226  -1.768  1.00 21.14           H  
ATOM   1524 1HD2 ASN   103      -7.452  12.859   1.578  1.00 27.73           H  
ATOM   1525 2HD2 ASN   103      -8.348  11.869   0.430  1.00 27.83           H  
ATOM   1526  N   LYS   104      -7.102   9.453  -3.499  1.00 13.91           N  
ATOM   1527  CA  LYS   104      -7.552   8.680  -4.644  1.00 13.32           C  
ATOM   1528  C   LYS   104      -8.344   7.485  -4.205  1.00 11.70           C  
ATOM   1529  O   LYS   104      -8.294   7.104  -3.054  1.00 11.73           O  
ATOM   1530  CB  LYS   104      -6.302   8.215  -5.410  0.50 14.85           C  
ATOM   1531  CG  LYS   104      -5.661   9.355  -6.183  0.50 16.49           C  
ATOM   1532  CD  LYS   104      -4.570   8.873  -7.117  0.50 19.42           C  
ATOM   1533  CE  LYS   104      -4.062   9.983  -8.024  0.50 21.73           C  
ATOM   1534  NZ  LYS   104      -5.088  10.516  -8.962  0.50 22.86           N  
ATOM   1535  H   LYS   104      -6.709   8.930  -2.712  1.00 14.43           H  
ATOM   1536  HA  LYS   104      -8.093   9.279  -5.332  1.00 13.60           H  
ATOM   1537 1HB  LYS   104      -5.606   7.839  -4.652  0.50 14.84           H  
ATOM   1538 2HB  LYS   104      -6.568   7.406  -6.080  0.50 14.92           H  
ATOM   1539 1HG  LYS   104      -6.417   9.873  -6.776  0.50 16.71           H  
ATOM   1540 2HG  LYS   104      -5.240  10.108  -5.495  0.50 16.59           H  
ATOM   1541 1HD  LYS   104      -3.735   8.471  -6.535  0.50 19.31           H  
ATOM   1542 2HD  LYS   104      -4.948   8.071  -7.748  0.50 19.44           H  
ATOM   1543 1HE  LYS   104      -3.736  10.828  -7.412  0.50 21.79           H  
ATOM   1544 2HE  LYS   104      -3.223   9.641  -8.629  0.50 21.78           H  
ATOM   1545 1HZ  LYS   104      -5.883   9.863  -9.008  1.00  0.00           H  
ATOM   1546 2HZ  LYS   104      -5.416  11.433  -8.626  1.00  0.00           H  
ATOM   1547  N   HIS   105      -9.007   6.862  -5.181  1.00 11.64           N  
ATOM   1548  CA  HIS   105      -9.729   5.641  -4.894  1.00 11.82           C  
ATOM   1549  C   HIS   105      -8.613   4.572  -4.921  1.00 11.80           C  
ATOM   1550  O   HIS   105      -7.657   4.717  -5.657  1.00 12.42           O  
ATOM   1551  CB  HIS   105     -10.743   5.233  -5.961  1.00 13.50           C  
ATOM   1552  CG  HIS   105     -11.821   6.291  -6.150  1.00 15.81           C  
ATOM   1553  ND1 HIS   105     -12.986   6.224  -5.494  1.00 17.68           N  
ATOM   1554  CD2 HIS   105     -11.771   7.420  -6.948  1.00 16.69           C  
ATOM   1555  CE1 HIS   105     -13.681   7.313  -5.873  1.00 18.00           C  
ATOM   1556  NE2 HIS   105     -12.966   8.047  -6.694  1.00 18.54           N  
ATOM   1557  H   HIS   105      -8.976   7.171  -6.128  1.00 12.47           H  
ATOM   1558  HA  HIS   105     -10.193   5.657  -3.948  1.00 11.61           H  
ATOM   1559 1HB  HIS   105     -10.212   5.063  -6.902  1.00 13.29           H  
ATOM   1560 2HB  HIS   105     -11.208   4.319  -5.618  1.00 13.03           H  
ATOM   1561  HD1 HIS   105     -13.345   5.518  -4.838  1.00 16.72           H  
ATOM   1562  HD2 HIS   105     -11.012   7.806  -7.574  1.00 16.79           H  
ATOM   1563  HE1 HIS   105     -14.660   7.539  -5.514  1.00 17.40           H  
ATOM   1564  N   ILE   106      -8.822   3.520  -4.163  1.00 10.61           N  
ATOM   1565  CA  ILE   106      -7.851   2.427  -4.132  1.00 10.59           C  
ATOM   1566  C   ILE   106      -8.518   1.158  -4.757  1.00 10.46           C  
ATOM   1567  O   ILE   106      -9.712   0.985  -4.666  1.00 10.81           O  
ATOM   1568  CB  ILE   106      -7.300   2.160  -2.744  1.00 10.39           C  
ATOM   1569  CG1 ILE   106      -8.332   1.700  -1.772  1.00 12.99           C  
ATOM   1570  CG2 ILE   106      -6.604   3.418  -2.260  1.00 11.78           C  
ATOM   1571  CD1 ILE   106      -7.764   1.060  -0.472  1.00 13.16           C  
ATOM   1572  H   ILE   106      -9.628   3.423  -3.583  1.00 11.19           H  
ATOM   1573  HA  ILE   106      -7.020   2.669  -4.762  1.00 10.77           H  
ATOM   1574  HB  ILE   106      -6.531   1.396  -2.876  1.00 11.04           H  
ATOM   1575 1HG1 ILE   106      -8.974   2.534  -1.450  1.00 12.83           H  
ATOM   1576 2HG1 ILE   106      -8.986   0.945  -2.185  1.00 12.70           H  
ATOM   1577 1HG2 ILE   106      -5.879   3.767  -3.006  1.00 11.46           H  
ATOM   1578 2HG2 ILE   106      -7.354   4.211  -2.125  1.00 11.46           H  
ATOM   1579 3HG2 ILE   106      -6.097   3.225  -1.327  1.00 11.60           H  
ATOM   1580 1HD1 ILE   106      -7.145   0.204  -0.751  1.00 13.03           H  
ATOM   1581 2HD1 ILE   106      -7.254   1.860   0.056  1.00 13.03           H  
ATOM   1582 3HD1 ILE   106      -8.634   0.763   0.116  1.00 13.29           H  
ATOM   1583  N   ILE   107      -7.648   0.339  -5.312  1.00  9.22           N  
ATOM   1584  CA  ILE   107      -8.038  -0.937  -5.866  1.00  9.33           C  
ATOM   1585  C   ILE   107      -7.167  -2.015  -5.132  1.00  9.65           C  
ATOM   1586  O   ILE   107      -5.927  -1.957  -5.245  1.00  9.51           O  
ATOM   1587  CB  ILE   107      -7.874  -1.061  -7.372  1.00  8.85           C  
ATOM   1588  CG1 ILE   107      -8.694   0.062  -8.085  1.00 10.23           C  
ATOM   1589  CG2 ILE   107      -8.350  -2.405  -7.877  1.00  9.25           C  
ATOM   1590  CD1 ILE   107      -8.601   0.181  -9.558  1.00 13.42           C  
ATOM   1591  H   ILE   107      -6.653   0.537  -5.323  1.00  9.93           H  
ATOM   1592  HA  ILE   107      -9.043  -1.074  -5.637  1.00  9.36           H  
ATOM   1593  HB  ILE   107      -6.840  -0.889  -7.606  1.00  8.99           H  
ATOM   1594 1HG1 ILE   107      -9.735  -0.051  -7.818  1.00  9.89           H  
ATOM   1595 2HG1 ILE   107      -8.292   0.998  -7.675  1.00 10.11           H  
ATOM   1596 1HG2 ILE   107      -7.997  -3.252  -7.302  1.00  8.95           H  
ATOM   1597 2HG2 ILE   107      -9.448  -2.437  -7.898  1.00  8.63           H  
ATOM   1598 3HG2 ILE   107      -7.996  -2.552  -8.912  1.00  8.66           H  
ATOM   1599 1HD1 ILE   107      -7.559   0.377  -9.835  1.00 13.06           H  
ATOM   1600 2HD1 ILE   107      -8.968  -0.730 -10.078  1.00 12.90           H  
ATOM   1601 3HD1 ILE   107      -9.227   1.040  -9.881  1.00 13.06           H  
ATOM   1602  N   VAL   108      -7.845  -2.943  -4.515  1.00  9.39           N  
ATOM   1603  CA  VAL   108      -7.184  -4.082  -3.817  1.00  9.10           C  
ATOM   1604  C   VAL   108      -7.725  -5.431  -4.310  1.00  9.17           C  
ATOM   1605  O   VAL   108      -8.853  -5.491  -4.726  1.00 10.27           O  
ATOM   1606  CB  VAL   108      -7.398  -3.955  -2.278  1.00 10.07           C  
ATOM   1607  CG1 VAL   108      -6.776  -2.651  -1.737  1.00 11.96           C  
ATOM   1608  CG2 VAL   108      -8.827  -4.079  -1.849  1.00 11.18           C  
ATOM   1609  H   VAL   108      -8.844  -2.943  -4.500  1.00  9.69           H  
ATOM   1610  HA  VAL   108      -6.138  -4.004  -3.963  1.00  9.18           H  
ATOM   1611  HB  VAL   108      -6.858  -4.811  -1.861  1.00 10.32           H  
ATOM   1612 1HG1 VAL   108      -5.954  -2.312  -2.315  1.00 11.29           H  
ATOM   1613 2HG1 VAL   108      -7.484  -1.829  -1.819  1.00 11.97           H  
ATOM   1614 3HG1 VAL   108      -6.497  -2.687  -0.704  1.00 11.56           H  
ATOM   1615 1HG2 VAL   108      -9.286  -5.007  -2.154  1.00 10.78           H  
ATOM   1616 2HG2 VAL   108      -8.895  -4.055  -0.760  1.00 11.15           H  
ATOM   1617 3HG2 VAL   108      -9.477  -3.293  -2.192  1.00 10.86           H  
ATOM   1618  N   ALA   109      -6.882  -6.448  -4.151  1.00  8.63           N  
ATOM   1619  CA  ALA   109      -7.275  -7.840  -4.426  1.00  9.18           C  
ATOM   1620  C   ALA   109      -7.614  -8.430  -3.019  1.00  9.27           C  
ATOM   1621  O   ALA   109      -6.911  -8.176  -2.074  1.00  9.09           O  
ATOM   1622  CB  ALA   109      -6.278  -8.703  -5.090  1.00  9.33           C  
ATOM   1623  H   ALA   109      -5.977  -6.332  -3.732  1.00  9.19           H  
ATOM   1624  HA  ALA   109      -8.165  -7.818  -4.988  1.00  9.71           H  
ATOM   1625 1HB  ALA   109      -5.755  -8.249  -5.906  1.00  8.96           H  
ATOM   1626 2HB  ALA   109      -5.542  -9.082  -4.385  1.00  8.76           H  
ATOM   1627 3HB  ALA   109      -6.781  -9.612  -5.471  1.00  9.12           H  
ATOM   1628  N   CYS   110      -8.722  -9.203  -2.977  1.00  8.96           N  
ATOM   1629  CA  CYS   110      -9.197  -9.793  -1.751  1.00  9.99           C  
ATOM   1630  C   CYS   110      -9.217 -11.324  -1.822  1.00 11.40           C  
ATOM   1631  O   CYS   110      -9.485 -11.880  -2.889  1.00 11.32           O  
ATOM   1632  CB  CYS   110     -10.616  -9.281  -1.420  1.00 11.04           C  
ATOM   1633  SG  CYS   110     -10.599  -7.460  -1.239  1.00 11.44           S  
ATOM   1634  H   CYS   110      -9.258  -9.351  -3.807  1.00  9.40           H  
ATOM   1635  HA  CYS   110      -8.577  -9.478  -0.970  1.00 10.19           H  
ATOM   1636 1HB  CYS   110     -11.271  -9.492  -2.227  1.00 10.61           H  
ATOM   1637 2HB  CYS   110     -10.942  -9.701  -0.508  1.00 10.22           H  
ATOM   1638  N   GLU   111      -8.984 -11.923  -0.662  1.00 11.81           N  
ATOM   1639  CA  GLU   111      -9.049 -13.414  -0.594  1.00 13.71           C  
ATOM   1640  C   GLU   111      -9.398 -13.834   0.842  1.00 14.64           C  
ATOM   1641  O   GLU   111      -9.174 -13.078   1.733  1.00 14.31           O  
ATOM   1642  CB  GLU   111      -7.699 -14.063  -0.841  1.00 16.43           C  
ATOM   1643  CG  GLU   111      -7.045 -13.820  -2.166  1.00 21.69           C  
ATOM   1644  CD  GLU   111      -5.692 -14.450  -2.427  1.00 25.07           C  
ATOM   1645  OE1 GLU   111      -5.020 -14.067  -3.390  1.00 27.22           O  
ATOM   1646  OE2 GLU   111      -5.351 -15.350  -1.548  1.00 28.69           O  
ATOM   1647  H   GLU   111      -8.792 -11.386   0.154  1.00 11.05           H  
ATOM   1648  HA  GLU   111      -9.773 -13.778  -1.271  1.00 14.09           H  
ATOM   1649 1HB  GLU   111      -6.982 -13.745  -0.059  1.00 16.58           H  
ATOM   1650 2HB  GLU   111      -7.808 -15.132  -0.681  1.00 17.03           H  
ATOM   1651 1HG  GLU   111      -7.671 -14.131  -3.005  1.00 21.51           H  
ATOM   1652 2HG  GLU   111      -6.902 -12.748  -2.310  1.00 21.25           H  
ATOM   1653  N   GLY   112      -9.825 -15.100   0.926  1.00 15.14           N  
ATOM   1654  CA  GLY   112     -10.038 -15.805   2.157  1.00 16.15           C  
ATOM   1655  C   GLY   112     -11.346 -15.631   2.853  1.00 17.08           C  
ATOM   1656  O   GLY   112     -12.277 -15.079   2.352  1.00 16.83           O  
ATOM   1657  H   GLY   112      -9.891 -15.685   0.094  1.00 14.82           H  
ATOM   1658 1HA  GLY   112      -9.879 -16.876   1.939  1.00 16.24           H  
ATOM   1659 2HA  GLY   112      -9.187 -15.576   2.834  1.00 16.15           H  
ATOM   1660  N   ASN   113     -11.371 -16.225   4.075  1.00 19.28           N  
ATOM   1661  CA  ASN   113     -12.557 -16.105   4.932  1.00 22.60           C  
ATOM   1662  C   ASN   113     -12.019 -15.801   6.349  1.00 22.26           C  
ATOM   1663  O   ASN   113     -11.344 -16.735   6.897  1.00 23.79           O  
ATOM   1664  CB  ASN   113     -13.536 -17.279   4.778  1.00 28.63           C  
ATOM   1665  CG  ASN   113     -14.674 -17.167   5.773  1.00 33.57           C  
ATOM   1666  OD1 ASN   113     -14.707 -17.977   6.749  1.00 37.86           O  
ATOM   1667  ND2 ASN   113     -15.569 -16.176   5.593  1.00 35.48           N  
ATOM   1668  H   ASN   113     -10.575 -16.677   4.470  1.00 18.76           H  
ATOM   1669  HA  ASN   113     -13.198 -15.338   4.567  1.00 22.94           H  
ATOM   1670 1HB  ASN   113     -13.945 -17.291   3.773  1.00 28.49           H  
ATOM   1671 2HB  ASN   113     -13.008 -18.207   4.936  1.00 28.46           H  
ATOM   1672 1HD2 ASN   113     -15.476 -15.577   4.798  1.00 34.57           H  
ATOM   1673 2HD2 ASN   113     -16.336 -16.103   6.265  1.00 34.66           H  
ATOM   1674  N   PRO   114     -12.210 -14.608   6.913  1.00 19.70           N  
ATOM   1675  CA  PRO   114     -12.900 -13.474   6.376  1.00 18.02           C  
ATOM   1676  C   PRO   114     -12.161 -12.995   5.057  1.00 15.76           C  
ATOM   1677  O   PRO   114     -10.935 -13.197   4.881  1.00 14.87           O  
ATOM   1678  CB  PRO   114     -12.756 -12.365   7.469  1.00 18.58           C  
ATOM   1679  CG  PRO   114     -11.955 -12.897   8.594  1.00 19.66           C  
ATOM   1680  CD  PRO   114     -11.679 -14.349   8.269  1.00 20.01           C  
ATOM   1681  HA  PRO   114     -13.937 -13.664   6.213  1.00 18.24           H  
ATOM   1682 1HB  PRO   114     -12.374 -11.518   7.116  1.00 18.60           H  
ATOM   1683 2HB  PRO   114     -13.738 -12.072   7.745  1.00 18.49           H  
ATOM   1684 1HG  PRO   114     -11.138 -12.441   8.680  1.00 19.44           H  
ATOM   1685 2HG  PRO   114     -12.432 -12.859   9.566  1.00 19.60           H  
ATOM   1686 1HD  PRO   114     -10.623 -14.546   8.332  1.00 20.00           H  
ATOM   1687 2HD  PRO   114     -12.140 -15.051   8.960  1.00 20.28           H  
ATOM   1688  N   TYR   115     -12.951 -12.396   4.240  1.00 14.43           N  
ATOM   1689  CA  TYR   115     -12.503 -11.894   2.911  1.00 13.79           C  
ATOM   1690  C   TYR   115     -11.859 -10.557   3.068  1.00 13.17           C  
ATOM   1691  O   TYR   115     -12.606  -9.595   3.259  1.00 13.05           O  
ATOM   1692  CB  TYR   115     -13.734 -11.884   2.034  1.00 15.13           C  
ATOM   1693  CG  TYR   115     -13.531 -11.779   0.557  1.00 13.79           C  
ATOM   1694  CD1 TYR   115     -12.780 -12.763  -0.133  1.00 13.97           C  
ATOM   1695  CD2 TYR   115     -14.137 -10.766  -0.178  1.00 13.27           C  
ATOM   1696  CE1 TYR   115     -12.600 -12.738  -1.502  1.00 14.29           C  
ATOM   1697  CE2 TYR   115     -13.979 -10.725  -1.542  1.00 13.41           C  
ATOM   1698  CZ  TYR   115     -13.212 -11.705  -2.232  1.00 14.57           C  
ATOM   1699  OH  TYR   115     -13.101 -11.658  -3.555  1.00 15.51           O  
ATOM   1700  H   TYR   115     -13.934 -12.255   4.399  1.00 15.41           H  
ATOM   1701  HA  TYR   115     -11.795 -12.588   2.509  1.00 13.79           H  
ATOM   1702 1HB  TYR   115     -14.262 -12.855   2.239  1.00 14.93           H  
ATOM   1703 2HB  TYR   115     -14.426 -11.129   2.397  1.00 14.71           H  
ATOM   1704  HD1 TYR   115     -12.317 -13.562   0.428  1.00 14.23           H  
ATOM   1705  HD2 TYR   115     -14.730 -10.019   0.312  1.00 13.42           H  
ATOM   1706  HE1 TYR   115     -12.027 -13.496  -2.004  1.00 14.19           H  
ATOM   1707  HE2 TYR   115     -14.436  -9.947  -2.150  1.00 13.95           H  
ATOM   1708  HH  TYR   115     -12.183 -11.731  -3.888  1.00 15.27           H  
ATOM   1709  N   VAL   116     -10.546 -10.540   2.951  1.00 11.28           N  
ATOM   1710  CA  VAL   116      -9.789  -9.315   3.228  1.00 10.50           C  
ATOM   1711  C   VAL   116      -8.790  -9.009   2.132  1.00  9.39           C  
ATOM   1712  O   VAL   116      -8.431  -9.845   1.336  1.00  8.75           O  
ATOM   1713  CB  VAL   116      -9.096  -9.578   4.619  1.00 11.29           C  
ATOM   1714  CG1 VAL   116     -10.145  -9.683   5.746  1.00 12.40           C  
ATOM   1715  CG2 VAL   116      -8.171 -10.792   4.617  1.00 12.30           C  
ATOM   1716  H   VAL   116     -10.004 -11.365   2.835  1.00 12.32           H  
ATOM   1717  HA  VAL   116     -10.464  -8.539   3.446  1.00 10.47           H  
ATOM   1718  HB  VAL   116      -8.485  -8.721   4.815  1.00 11.51           H  
ATOM   1719 1HG1 VAL   116     -10.817  -8.873   5.720  1.00 11.72           H  
ATOM   1720 2HG1 VAL   116     -10.707 -10.584   5.641  1.00 12.19           H  
ATOM   1721 3HG1 VAL   116      -9.685  -9.730   6.717  1.00 12.07           H  
ATOM   1722 1HG2 VAL   116      -7.608 -10.908   3.715  1.00 11.90           H  
ATOM   1723 2HG2 VAL   116      -7.481 -10.753   5.440  1.00 11.79           H  
ATOM   1724 3HG2 VAL   116      -8.726 -11.717   4.738  1.00 11.59           H  
ATOM   1725  N   PRO   117      -8.291  -7.767   2.162  1.00  9.42           N  
ATOM   1726  CA  PRO   117      -7.286  -7.356   1.181  1.00  8.90           C  
ATOM   1727  C   PRO   117      -5.945  -8.069   1.439  1.00  8.98           C  
ATOM   1728  O   PRO   117      -5.440  -8.148   2.536  1.00  9.71           O  
ATOM   1729  CB  PRO   117      -7.067  -5.850   1.410  1.00 10.12           C  
ATOM   1730  CG  PRO   117      -8.234  -5.396   2.241  1.00 11.10           C  
ATOM   1731  CD  PRO   117      -8.713  -6.607   3.037  1.00 10.64           C  
ATOM   1732  HA  PRO   117      -7.689  -7.537   0.208  1.00  9.27           H  
ATOM   1733 1HB  PRO   117      -6.209  -5.636   1.872  1.00  9.92           H  
ATOM   1734 2HB  PRO   117      -6.970  -5.351   0.499  1.00 10.38           H  
ATOM   1735 1HG  PRO   117      -7.924  -4.791   2.896  1.00 11.03           H  
ATOM   1736 2HG  PRO   117      -9.031  -4.968   1.693  1.00 10.40           H  
ATOM   1737 1HD  PRO   117      -8.299  -6.630   4.004  1.00 10.78           H  
ATOM   1738 2HD  PRO   117      -9.780  -6.539   3.152  1.00 10.50           H  
ATOM   1739  N   VAL   118      -5.414  -8.537   0.311  1.00  8.42           N  
ATOM   1740  CA  VAL   118      -4.124  -9.214   0.284  1.00  9.93           C  
ATOM   1741  C   VAL   118      -3.110  -8.585  -0.648  1.00 11.18           C  
ATOM   1742  O   VAL   118      -1.924  -8.929  -0.622  1.00 12.25           O  
ATOM   1743  CB  VAL   118      -4.254 -10.748   0.087  1.00 10.97           C  
ATOM   1744  CG1 VAL   118      -5.069 -11.362   1.215  1.00 11.29           C  
ATOM   1745  CG2 VAL   118      -4.783 -11.019  -1.294  1.00 11.79           C  
ATOM   1746  H   VAL   118      -5.895  -8.468  -0.561  1.00  8.74           H  
ATOM   1747  HA  VAL   118      -3.703  -9.173   1.287  1.00 10.29           H  
ATOM   1748  HB  VAL   118      -3.218 -11.089   0.134  1.00 11.26           H  
ATOM   1749 1HG1 VAL   118      -4.676 -11.104   2.177  1.00 10.62           H  
ATOM   1750 2HG1 VAL   118      -6.100 -11.034   1.218  1.00 10.96           H  
ATOM   1751 3HG1 VAL   118      -5.093 -12.427   1.180  1.00 10.95           H  
ATOM   1752 1HG2 VAL   118      -5.714 -10.519  -1.516  1.00 11.39           H  
ATOM   1753 2HG2 VAL   118      -4.086 -10.728  -2.071  1.00 11.54           H  
ATOM   1754 3HG2 VAL   118      -4.941 -12.063  -1.394  1.00 11.49           H  
ATOM   1755  N   HIS   119      -3.540  -7.681  -1.547  1.00 10.33           N  
ATOM   1756  CA  HIS   119      -2.650  -7.036  -2.503  1.00 10.50           C  
ATOM   1757  C   HIS   119      -3.210  -5.655  -2.800  1.00 10.16           C  
ATOM   1758  O   HIS   119      -4.392  -5.484  -2.949  1.00 10.37           O  
ATOM   1759  CB  HIS   119      -2.593  -7.927  -3.746  1.00 14.62           C  
ATOM   1760  CG  HIS   119      -2.176  -7.334  -5.017  1.00 18.83           C  
ATOM   1761  ND1 HIS   119      -0.862  -7.166  -5.349  1.00 21.52           N  
ATOM   1762  CD2 HIS   119      -2.971  -6.874  -6.052  1.00 21.08           C  
ATOM   1763  CE1 HIS   119      -0.839  -6.603  -6.556  1.00 21.22           C  
ATOM   1764  NE2 HIS   119      -2.115  -6.418  -7.010  1.00 22.00           N  
ATOM   1765  H   HIS   119      -4.494  -7.428  -1.576  1.00 10.58           H  
ATOM   1766  HA  HIS   119      -1.670  -6.964  -2.102  1.00 10.69           H  
ATOM   1767 1HB  HIS   119      -1.908  -8.798  -3.521  1.00 14.46           H  
ATOM   1768 2HB  HIS   119      -3.575  -8.382  -3.832  1.00 14.59           H  
ATOM   1769  HD1 HIS   119      -0.011  -7.389  -4.819  1.00 20.63           H  
ATOM   1770  HD2 HIS   119      -4.026  -6.836  -6.180  1.00 20.61           H  
ATOM   1771  HE1 HIS   119       0.031  -6.349  -7.097  1.00 20.93           H  
ATOM   1772  N   PHE   120      -2.319  -4.743  -2.985  1.00 10.00           N  
ATOM   1773  CA  PHE   120      -2.649  -3.318  -3.404  1.00  9.79           C  
ATOM   1774  C   PHE   120      -2.402  -3.311  -4.883  1.00 10.02           C  
ATOM   1775  O   PHE   120      -1.235  -3.447  -5.349  1.00 11.23           O  
ATOM   1776  CB  PHE   120      -1.758  -2.333  -2.656  1.00 10.89           C  
ATOM   1777  CG  PHE   120      -2.241  -0.908  -2.873  1.00 11.78           C  
ATOM   1778  CD1 PHE   120      -1.833  -0.215  -3.984  1.00 12.75           C  
ATOM   1779  CD2 PHE   120      -3.098  -0.364  -1.936  1.00 13.70           C  
ATOM   1780  CE1 PHE   120      -2.314   1.085  -4.210  1.00 13.60           C  
ATOM   1781  CE2 PHE   120      -3.564   0.966  -2.145  1.00 15.76           C  
ATOM   1782  CZ  PHE   120      -3.184   1.611  -3.290  1.00 14.14           C  
ATOM   1783  H   PHE   120      -1.319  -4.887  -2.920  1.00 10.66           H  
ATOM   1784  HA  PHE   120      -3.616  -3.080  -3.136  1.00  9.95           H  
ATOM   1785 1HB  PHE   120      -1.792  -2.607  -1.613  1.00 10.77           H  
ATOM   1786 2HB  PHE   120      -0.759  -2.440  -3.021  1.00 10.84           H  
ATOM   1787  HD1 PHE   120      -1.203  -0.659  -4.668  1.00 12.98           H  
ATOM   1788  HD2 PHE   120      -3.405  -0.918  -1.083  1.00 13.54           H  
ATOM   1789  HE1 PHE   120      -2.007   1.620  -5.063  1.00 13.56           H  
ATOM   1790  HE2 PHE   120      -4.215   1.396  -1.479  1.00 14.60           H  
ATOM   1791  HZ  PHE   120      -3.524   2.628  -3.449  1.00 14.59           H  
ATOM   1792  N   ASP   121      -3.454  -3.080  -5.676  1.00 10.06           N  
ATOM   1793  CA  ASP   121      -3.338  -3.120  -7.128  1.00 10.32           C  
ATOM   1794  C   ASP   121      -3.034  -1.806  -7.722  1.00 10.77           C  
ATOM   1795  O   ASP   121      -2.155  -1.702  -8.572  1.00 12.92           O  
ATOM   1796  CB  ASP   121      -4.601  -3.736  -7.714  1.00 11.31           C  
ATOM   1797  CG  ASP   121      -4.337  -4.317  -9.087  1.00 13.70           C  
ATOM   1798  OD1 ASP   121      -3.422  -5.125  -9.263  1.00 16.84           O  
ATOM   1799  OD2 ASP   121      -4.994  -3.970 -10.092  1.00 12.97           O  
ATOM   1800  H   ASP   121      -4.374  -3.001  -5.286  1.00 10.06           H  
ATOM   1801  HA  ASP   121      -2.574  -3.839  -7.396  1.00 10.34           H  
ATOM   1802 1HB  ASP   121      -4.961  -4.510  -7.062  1.00 11.21           H  
ATOM   1803 2HB  ASP   121      -5.408  -3.021  -7.762  1.00 11.47           H  
ATOM   1804  N   ALA   122      -3.711  -0.738  -7.351  1.00 11.04           N  
ATOM   1805  CA  ALA   122      -3.563   0.584  -7.909  1.00 11.38           C  
ATOM   1806  C   ALA   122      -4.361   1.641  -7.149  1.00 11.84           C  
ATOM   1807  O   ALA   122      -5.236   1.346  -6.339  1.00 11.77           O  
ATOM   1808  CB  ALA   122      -4.194   0.529  -9.325  1.00 11.75           C  
ATOM   1809  H   ALA   122      -4.460  -0.889  -6.655  1.00 11.41           H  
ATOM   1810  HA  ALA   122      -2.558   0.869  -8.034  1.00 11.15           H  
ATOM   1811 1HB  ALA   122      -3.910  -0.354  -9.848  1.00 11.47           H  
ATOM   1812 2HB  ALA   122      -5.274   0.496  -9.247  1.00 11.08           H  
ATOM   1813 3HB  ALA   122      -3.896   1.402  -9.871  1.00 11.13           H  
ATOM   1814  N   SER   123      -4.044   2.883  -7.474  1.00 14.08           N  
ATOM   1815  CA  SER   123      -4.804   4.058  -6.967  1.00 15.31           C  
ATOM   1816  C   SER   123      -5.293   4.760  -8.275  1.00 16.63           C  
ATOM   1817  O   SER   123      -4.494   4.870  -9.257  1.00 17.90           O  
ATOM   1818  CB  SER   123      -4.072   4.947  -6.030  1.00 16.86           C  
ATOM   1819  OG  SER   123      -3.065   5.618  -6.738  1.00 20.43           O  
ATOM   1820  H   SER   123      -3.350   3.082  -8.169  1.00 14.26           H  
ATOM   1821  HA  SER   123      -5.656   3.660  -6.476  1.00 15.58           H  
ATOM   1822 1HB  SER   123      -4.725   5.639  -5.486  1.00 17.19           H  
ATOM   1823 2HB  SER   123      -3.575   4.348  -5.242  1.00 16.92           H  
ATOM   1824  HG  SER   123      -2.280   5.724  -6.197  1.00 21.19           H  
ATOM   1825  N   VAL   124      -6.531   5.130  -8.304  1.00 16.09           N  
ATOM   1826  CA  VAL   124      -7.139   5.789  -9.496  1.00 17.88           C  
ATOM   1827  C   VAL   124      -7.871   7.036  -9.029  1.00 20.04           C  
ATOM   1828  O   VAL   124      -7.979   7.950  -9.942  1.00 22.34           O  
ATOM   1829  CB  VAL   124      -8.014   4.793 -10.294  1.00 17.92           C  
ATOM   1830  CG1 VAL   124      -7.187   3.665 -10.914  1.00 17.93           C  
ATOM   1831  CG2 VAL   124      -9.148   4.220  -9.530  1.00 18.00           C  
ATOM   1832  OXT VAL   124      -8.344   7.215  -7.922  1.00 18.14           O  
ATOM   1833  H   VAL   124      -7.138   5.005  -7.513  1.00 16.17           H  
ATOM   1834  HA  VAL   124      -6.316   6.102 -10.139  1.00 17.83           H  
ATOM   1835  HB  VAL   124      -8.424   5.408 -11.112  1.00 18.20           H  
ATOM   1836 1HG1 VAL   124      -6.432   4.072 -11.569  1.00 17.76           H  
ATOM   1837 2HG1 VAL   124      -6.692   3.063 -10.193  1.00 17.76           H  
ATOM   1838 3HG1 VAL   124      -7.806   3.055 -11.530  1.00 17.86           H  
ATOM   1839 1HG2 VAL   124      -8.852   3.676  -8.637  1.00 17.99           H  
ATOM   1840 2HG2 VAL   124      -9.842   4.982  -9.209  1.00 18.18           H  
ATOM   1841 3HG2 VAL   124      -9.716   3.505 -10.112  1.00 18.08           H  
TER
'''
  R1_data='''T3      1.6036  0.0268
A4      1.6679  0.0376
A5      1.6587  0.0239
A6      1.6952  0.0246
K7      1.6472  0.0283
F8      1.6343  0.0222
E9      1.7065  0.0285
R10     1.6681  0.0223
Q11     1.6507  0.0277
H12     1.6428  0.0209
M13     1.6599  0.0246
D14     1.6042  0.0299
S15     1.6414  0.0438
A19     1.6953  0.0294
S22     1.4819  0.0339
Y25     1.7074  0.0274
C26     1.7731  0.0247
N27     1.5932  0.0447
Q28     1.6937  0.0242
M30     1.6917  0.0259
K31     1.6872  0.0325
S32     1.7173  0.0201
R33     1.6528  0.0346
N34     1.6724  0.0292
L35     1.6854  0.0246
T36     1.7154  0.0239
K37     1.4974  0.0244
R39     1.5523  0.0354
C40     1.5708  0.0256
K41     1.5666  0.0172
V43     1.6074  0.0220
N44     1.5927  0.0266
T45     1.4483  0.0198
F46     1.5740  0.0372
E49     1.2389  0.0208
S50     1.5934  0.0315
L51     1.7089  0.0210
A52     1.6587  0.0462
D53     1.5934  0.0242
V54     1.6861  0.0177
Q55     1.7237  0.0257
A56     1.6712  0.0397
V57     1.6821  0.0308
C58     1.7439  0.0306
S59     1.6602  0.0185
Q60     1.5460  0.0259
K61     1.5399  0.0208
N62     1.5094  0.0297
A64     1.5725  0.0202
C65     1.5611  0.0181
K66     1.5687  0.0490
N67     1.6542  0.0207
G68     1.6550  0.0279
Q69     1.5450  0.0238
T70     1.5479  0.0279
N71     1.5864  0.0279
C72     1.6513  0.0224
Y73     1.5394  0.0212
Q74     1.5327  0.0286
S75     1.5873  0.0242
Y76     1.7176  0.0284
S77     1.6548  0.0262
T78     1.6215  0.0231
M79     1.6173  0.0286
S80     1.5440  0.0236
C84     1.5134  0.0232
E86     1.6027  0.0239
T87     1.6918  0.0204
S90     1.4948  0.0218
K91     1.6247  0.0341
Y92     1.5430  0.0360
N94     1.6124  0.0290
C95     1.5180  0.0260
A96     1.6486  0.0315
Y97     1.6035  0.0244
K98     1.6981  0.0353
T99     1.5507  0.0274
T100    1.6437  0.0212
N103    1.7391  0.0267
K104    1.6339  0.0314
H105    1.6389  0.0268
I106    1.6028  0.0278
I107    1.5387  0.0284
V108    1.5925  0.0392
A109    1.5769  0.0168
C110    1.6450  0.0186
G112    1.5292  0.0360
N113    1.5116  0.0209
Y115    1.4501  0.0232
V116    1.6529  0.0202
V118    1.6642  0.0200
H119    1.6092  0.0243
F120    1.5400  0.0962
D121    1.6856  0.0271
A122    1.5128  0.0248
S123    1.5368  0.0232
V124    1.6397  0.0214'''
  R2_data = '''T3      7.6807  0.0853
A4      8.5730  0.1563
A5      8.7649  0.1449
A6      8.7658  0.0926
K7      9.2935  0.0935
F8      9.0655  0.1200
E9      9.3026  0.1331
R10     9.0294  0.1384
Q11     9.0287  0.2020
H12     8.6942  0.1878
M13     9.1109  0.1448
D14     8.7334  0.1805
S15     10.1096 0.2220
A19     7.9108  0.2241
S22     10.9907 0.2142
Y25     9.1959  0.1778
C26     8.2273  0.1315
N27     5.0245  0.3825
Q28     8.3822  0.1646
M30     9.1502  0.0923
K31     8.7302  0.1356
S32     8.7266  0.1856
R33     10.7654 0.2071
N34     8.7972  0.1333
L35     8.7799  0.2212
T36     8.2411  0.1958
K37     8.8855  0.1840
R39     7.7780  0.1740
C40     7.8185  0.1218
K41     7.9485  0.1318
V43     9.0336  0.2820
N44     8.4606  0.2314
T45     8.0665  0.1870
F46     13.1421 0.7062
E49     6.1417  0.1034
S50     8.6976  0.1462
L51     8.6091  0.1042
A52     8.9684  0.0939
D53     8.8732  0.1264
V54     8.4255  0.2192
Q55     8.5390  0.1614
A56     8.9550  0.1570
V57     9.2490  0.1512
C58     8.0234  0.1594
S59     8.3997  0.2247
Q60     8.5740  0.1966
K61     8.7896  0.2102
N62     8.9998  0.1771
A64     8.2065  0.1478
C65     9.3741  0.2396
K66     13.4070 0.2587
N67     7.5886  0.0856
G68     8.2427  0.1062
Q69     11.6064 0.2033
T70     8.1167  0.1705
N71     13.3971 0.2833
C72     8.1326  0.2035
Y73     9.0677  0.1930
Q74     9.4643  0.2180
S75     9.6637  0.1649
Y76     14.5151 0.2161
S77     7.9713  0.2228
T78     8.4446  0.1530
M79     8.5017  0.2932
S80     9.0612  0.2049
C84     9.2354  0.1496
E86     8.8045  0.2439
T87     8.3801  0.0908
S90     7.8056  0.1380
K91     7.2685  0.1790
Y92     7.7864  0.1822
N94     7.7774  0.1767
C95     7.3904  0.1709
A96     7.4362  0.1396
Y97     7.9283  0.2485
K98     8.3391  0.1302
T99     8.4935  0.2018
T100    10.2375 0.2571
N103    8.4114  0.2594
K104    8.3460  0.2390
H105    11.5004 0.2349
I106    10.4325 0.1722
I107    9.2474  0.1695
V108    9.5033  0.2771
A109    9.9507  0.2909
C110    8.7970  0.1227
G112    7.6350  0.0948
N113    7.1127  0.1446
Y115    7.7852  0.1593
V116    8.2933  0.1978
V118    8.5690  0.1837
H119    17.1012 0.5608
F120    9.6585  0.2925
D121    10.2843 0.3441
A122    7.9691  0.3775
S123    8.0457  0.1952
V124    7.9262  0.2257'''
  NOE_data='''T3      0.74293 0.03061
A4      0.76979 0.03049
A5      0.79554 0.02532
A6      0.76155 0.02819
K7      0.78500 0.02802
F8      0.78183 0.02669
E9      0.77815 0.02882
R10     0.79100 0.02878
Q11     0.81749 0.03321
H12     0.82258 0.04815
M13     0.79694 0.03549
D14     0.78813 0.04269
S15     0.76379 0.08936
A19     0.80617 0.03943
S22     0.58757 0.05082
Y25     0.74502 0.03070
C26     0.84987 0.05856
N27     0.77208 0.10619
Q28     0.82816 0.03789
M30     0.81370 0.03784
K31     0.85110 0.03909
S32     0.75003 0.02979
R33     0.81309 0.04434
N34     0.77587 0.03241
L35     0.73126 0.03075
T36     0.75281 0.04861
K37     0.77635 0.02808
R39     0.73876 0.02618
C40     0.78531 0.02882
K41     0.75151 0.04118
V43     0.68842 0.02579
N44     0.72765 0.02685
T45     0.77144 0.04647
F46     0.73303 0.13376
E49     0.62605 0.03146
S50     0.75916 0.03566
L51     0.78904 0.02666
A52     0.82984 0.02300
D53     0.80910 0.03037
V54     0.82896 0.03027
Q55     0.80537 0.02723
A56     0.77406 0.02600
V57     0.81539 0.03034
C58     0.79311 0.02217
S59     0.76752 0.03812
Q60     0.83618 0.04065
K61     0.98706 0.04350
N62     0.81559 0.02949
A64     0.77194 0.02666
C65     0.76699 0.03819
K66     0.72214 0.13463
N67     0.74737 0.03825
G68     0.78177 0.03132
Q69     0.79301 0.03549
T70     0.75957 0.03819
N71     0.85807 0.06602
C72     0.85665 0.03522
Y73     0.78771 0.04442
Q74     0.79308 0.04096
S75     0.82716 0.03416
Y76     0.73950 0.03847
S77     0.76391 0.03339
T78     0.76935 0.03148
M79     0.79374 0.04262
S80     0.84064 0.03919
C84     0.77833 0.04110
E86     0.76637 0.03571
T87     0.79326 0.03037
S90     0.74529 0.02331
K91     0.78475 0.03334
Y92     0.70716 0.03862
N94     0.76976 0.02963
C95     0.82883 0.02860
A96     0.74433 0.04459
Y97     0.78309 0.03812
K98     0.79086 0.03806
T99     0.77756 0.04103
T100    0.77199 0.04637
N103    0.79115 0.03438
K104    0.78866 0.07589
H105    0.79063 0.06255
I106    0.80100 0.04442
I107    0.81185 0.03795
V108    0.79078 0.04254
A109    0.84915 0.05045
C110    0.78212 0.04103
G112    0.80522 0.03535
N113    0.69475 0.02579
Y115    0.75521 0.02979
V116    0.87720 0.04222
V118    0.84782 0.05879
H119    0.78389 0.05064
F120    0.87942 0.08130
D121    0.78224 0.05630
A122    0.74532 0.05371
S123    0.74291 0.02987
V124    0.79257 0.03325'''

  job_name='RNaseA'
  cleaned_R1_data=''
  cleaned_R2_data=''
  cleaned_NOE_data=''
  Magnetic_Field='600'

  R1_lines = R1_data.split('\n')
  R2_lines = R2_data.split('\n')
  NOE_lines = NOE_data.split('\n')
  
  # R1
  for line in R1_lines:
    sp_list = line.split()
    resnum = sp_list[0] #re.sub("[^0-9]","", sp_list[0].split('-')[0])
    R1, R1err = sp_list[1], sp_list[2].strip()
    if input_format == 'single field':  
      cleaned_R1_data += '%s\t%s\n' % (resnum, R1)
    else:
      cleaned_R1_data += '%s\t%s\t%s\n' % (resnum, R1, R1err)
  # R2
  for line in R2_lines:
    sp_list = line.split()
    resnum = sp_list[0] #re.sub("[^0-9]","", sp_list[0].split('-')[0])
    R2, R2err = sp_list[1], sp_list[2].strip()
    if input_format == 'single field':
      cleaned_R2_data += '%s\t%s\n' % (resnum, R2)
    else:
      cleaned_R2_data += '%s\t%s\t%s\n' % (resnum, R2, R2err)

  # NOE
  for line in NOE_lines:
    sp_list = line.split()
    resnum = sp_list[0] #re.sub("[^0-9]","", sp_list[0].split('-')[0])
    NOE, NOEerr = sp_list[1], sp_list[2].strip()
    if input_format == 'single field':
      cleaned_NOE_data += '%s\t%s\n' % (resnum, NOE)
    else:
      cleaned_NOE_data += '%s\t%s\t%s\n' % (resnum, NOE, NOEerr)

else:
  # Clean Data
  T1_lines = T1_data.split('\n')
  T2_lines = T2_data.split('\n')
  NOE_lines = NOE_data.split('\n')
  
  R1_dict, R2_dict = {}, {}
  
  # R1
  for line in T1_lines:
    sp_list = line.split()
    resnum = re.sub("[^0-9]","", sp_list[0].split('-')[0])
    try:
      R1 = 1000 / float(sp_list[1]) # ms to s, T to 1/T
      R1err = 1000 / float(sp_list[2]) / (len(sp_list) - 3)
    except:
      continue
    R1_dict[int(resnum)] = [R1, R1err, sp_list[0][0]]

  # R2
  for line in T2_lines:
    sp_list = line.split()
    resnum = re.sub("[^0-9]","", sp_list[0].split('-')[0])
    try:
      R2 = 1000 / float(sp_list[1]) # ms to s, T to 1/T
      R2err = 1000 / float(sp_list[2]) / (len(sp_list) - 3)
    except:
      continue
    R2_dict[int(resnum)] = [R2, R2err, sp_list[0][0]]

  # NOE
  NOE_list = []
  NOEdiff = 0
  for line in NOE_lines:
    sp_list = line.split()
    resnum = re.sub("[^0-9]","", sp_list[0])
    NOE = sp_list[1]
    NOEerr = sp_list[2]
    NOE_list.append([int(resnum), NOE, NOEerr])
    NOEdiff += NOE - 1

  # flip if satNOE and unsatNOE if opposite.  
  if NOEdiff > 0:
    for i in range(len(NOE_list)):
      NOE_list[i][1] = 1 / NOE_list[i][1]
  NOE_dict = dict(NOE_list)
  
  cleaned_R1_data, cleaned_R2_data, cleaned_NOE_data = '', '', ''
  for i in range(1, 1000):
    try:
      R1, R1err, a = R1_dict[i]
      R2, R2err, a = R2_dict[i]
      NOE, NOEerr = NOE_dict[i]
    except:
      continue
    if input_format == 'single field':  
      cleaned_R1_data += '%s%d\t%.3f\n' % (a, i, R1)
      cleaned_R2_data += '%s%d\t%.3f\n' % (a, i, R2)
      cleaned_NOE_data += '%s%d\t%s\n' % (a, i, NOE)
    else:
      cleaned_R1_data += '%s%d\t%.3f\t%.3f\n' % (a, i, R1, R1err)
      cleaned_R2_data += '%s%d\t%.3f\t%.3f\n' % (a, i, R2, R2err)
      cleaned_NOE_data += '%s%d\t%s\t%s\n' % (a, i, NOE, NOEerr)
    

In [ ]:
#@title **Writing Preprocessed Inputs**
# Check job name
if job_name == '':
  job_name = time.strftime("%y%m%d_%H%M%S_") + \
        "%06d" % random.randrange(0, 1000000)
job_name = job_name.replace(' ', '_')

# Replace config
FMFcfg = FMFcfg.replace('$TENSOR', diffusion_tensor)
FMFcfg = FMFcfg.replace('$JOBNAME', job_name)
FMFcfg = FMFcfg.replace('$MAGNETICFIELD', Magnetic_Field)

# Create a working directory
if not os.path.exists('/content/MFAnalysis'):
  os.mkdir('/content/MFAnalysis')
job_dir = '/content/MFAnalysis/' + job_name
if not os.path.exists(job_dir):
  os.mkdir(job_dir)
else:
  os.system('rm -rf {job_dir}/*')
# File names
fmfPDB = job_dir + '/mfpdb'
fmfR1 = job_dir + '/fmfR1.txt'
fmfR2 = job_dir + '/fmfR2.txt'
fmfNOE = job_dir + '/fmfNOE.txt'
fmfCfg = job_dir + '/FMF.config'

# Write PDB
if PDB_data != '':
  f = open(fmfPDB, 'w')
  f.write(PDB_data)
  f.close()

# Write R1
f = open(fmfR1, 'w')
f.write(cleaned_R1_data[:-1]) # remove the last null character
f.close()

# Write R2
f = open(fmfR2, 'w')
f.write(cleaned_R2_data[:-1]) # remove the last null character
f.close()

# Write NOE
f = open(fmfNOE, 'w')
f.write(cleaned_NOE_data[:-1]) # remove the last null character
f.close()

# Write Config
f = open(fmfCfg, 'w')
f.write(FMFcfg)
f.close()

In [ ]:
#@title ### **Download and install ModelFree**
!rm -rf modelfree_linux64.tar.gz
!rm -rf /usr/local/src/modelfree
!rm -rf /usr/local/bin/modelfree4
!wget -nv --no-check-certificate https://comdnmr.nysbc.org/comd-nmr-dissem/comd-nmr-software/software/modelfree/modelfree_linux64.tar.gz
!apt install libgfortran3
!tar xfz modelfree_linux64.tar.gz
!mv linux64 /usr/local/src/modelfree
!ln -s /usr/local/src/modelfree/modelfree4 /usr/local/bin/modelfree4

In [ ]:
#@title ### **Download the FASTModelFree 1.3wl**
!rm -rf fastMF13wl
!wget -nv --no-check-certificate https://poky.clas.ucdenver.edu/poky_releases/misc/fastMF13wl
!chmod +x fastMF13wl
!mv fastMF13wl /usr/local/bin/
fastMF13wl = '/usr/local/bin/fastMF13wl'

In [ ]:
#@title ### **Download and install MFoutParser**
!pip install git+https://github.com/pokynmr/mfoutparser.git

In [ ]:
#@title ### **Run FASTModelFree**
!cd {job_dir}; fastMF13wl | tee mf.log

In [ ]:
#@title ### **Parse ModelFree Results**
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import mfoutparser as mf
output_directory = os.path.join(job_dir, 'mfoutparse')
if not os.path.exists(output_directory):
  os.mkdir(output_directory)

# Parse the ModelFree output file
mfoutfilename = os.path.join(job_dir, 'mfout.final')
tags, loops = mf.parse_mfout(mfoutfilename)

# Write everything to a file
output_filename = os.path.join(output_directory, 'mfout_singlefield')
mf.write_all_to_file(tags, loops, output_filename)

In [ ]:
#@title ### **Plot ModelFree Results**
model = "model_1" #@param ["model_1", "model_2", "model_3", "model_4", "model_5"]

import matplotlib
matplotlib.rcParams[u'font.family'] = [u'sans-serif']
matplotlib.rcParams[u'mathtext.default'] = u'regular'

import matplotlib.pyplot as plt
%matplotlib inline

selector_dict = {'model_free_name':'S2'}
s2_data = mf.get_data_selection(loops[model], selector_dict)

fig = plt.gcf()
fig.set_size_inches(5,3)

ax = plt.axes()
ax.errorbar(s2_data.residue.values, s2_data.fit_value.values, 
  yerr=s2_data.sim_error.values, color='blue', marker='o', ls='', 
  capthick=1.0)

ax.set_ylim(0,1)
ax.set_xlim(0, s2_data.residue.max()+1)
ax.set_xlabel('Residue')
ax.set_ylabel('$S^2$')

# Set the fontsize for the label and tick labels
fontsize = 12.0
ax.xaxis.label.set_fontsize(fontsize)
ax.yaxis.label.set_fontsize(fontsize)
for tick in ax.xaxis.get_ticklabels():
    tick.set_fontsize(fontsize)
for tick in ax.yaxis.get_ticklabels():
    tick.set_fontsize(fontsize)

fig.tight_layout()

output_filename = os.sep.join([output_directory, 'S2_plot.pdf'])
fig.savefig(output_filename)
output_filename = os.sep.join([output_directory, 'S2_plot.png'])
fig.savefig(output_filename, dpi=300)

import pandas as pd
pd.set_option('max_rows', 1000)
pd.set_option('max_colwidth', None)

print(s2_data[['model_free_name', 'residue', 'fit_value', 'fit_error']])

In [ ]:
#@title **py3Dmol visualization**
!pip install py3Dmol

import py3Dmol

selector_dict = {'model_free_name':'S2'}
s2_data = mf.get_data_selection(loops[model], selector_dict) 
colors = {}

for i in range(min(s2_data.residue.values), max(s2_data.residue.values)+1):
  colors[i] = ('#%02x%02x%02x' % (125, 125, 125)).upper()
  
for i in range(len(s2_data.residue.values)):
  s2 = s2_data.fit_value.values[i]
  maxs2 = 1 - max((s2 - 0.7) / 0.3, 0)
  mins2 = max((s2 - 0.7) / 0.3, 0)
  
  r = max(0, min(255, int(maxs2 * 256)))
  g = max(0, min(255, int(mins2 * 256)))
  colors[s2_data.residue.values[i]] = ('#%02x%02x00' % (r, g)).upper()

pdb_name = job_dir + '/mfpdb'
view = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js',)
view.addModel(open(pdb_name, 'r').read(), 'pdb')
view.setStyle({'cartoon': {'colorscheme': {'prop': 'resi', 'map': colors}}})
view.zoomTo()

# create PyMOL coloring script
pymol = ''
for i in colors.keys():
  pymol += 'color %s, resi %d\n' % (colors[i], i)
pymol_name = job_dir + '/pymol_color'
f = open(pymol_name, 'w')
f.write(pymol)
f.close()

In [ ]:
#@title ### **Download ModelFree Results**
!cd {job_dir}; cd ..; zip -ru mf_{job_name}.zip {job_name}; mv mf_{job_name}.zip {job_dir}
compressed_file = job_dir + '/mf_' + job_name + '.zip'
files.download(compressed_file)